# Trading Desk Analytics Platform (All-in-One Standalone)

This is a **true standalone notebook**: the full data engine and analytics library are embedded directly in this file.

## Quick Start (3 steps)
1. Edit the **Master Configuration** cell only.
2. Run **Cell -> Run All** once.
3. Use the **Section Launcher** to open only what you need (or run everything).


## Why This Notebook Exists

The full platform is modular across multiple notebooks for maintainability, but this file wraps the same engine and analytics into one guided workflow so onboarding is easy.


In [ ]:
from __future__ import annotations

import json
from pathlib import Path
import types

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

try:
    import seaborn as sns
except Exception:
    class _SnsFallback:
        def set_theme(self, *args, **kwargs):
            return None

        def barplot(self, x=None, y=None, palette=None, **kwargs):
            plt.bar(x, y)
            return plt.gca()

    sns = _SnsFallback()

try:
    import plotly.express as px
except Exception:
    class _FallbackFigure:
        def update_layout(self, *args, **kwargs):
            return self

        def show(self):
            print('Plotly not installed; skipping interactive chart render.')

    class _PlotlyExpressFallback:
        def line(self, *args, **kwargs):
            return _FallbackFigure()

        def bar(self, *args, **kwargs):
            return _FallbackFigure()

        def treemap(self, *args, **kwargs):
            return _FallbackFigure()

    px = _PlotlyExpressFallback()

try:
    import ipywidgets as widgets
    from IPython.display import display, Markdown
except Exception:
    widgets = None
    display = print
    Markdown = lambda x: x

# ------------------------------------------------------------------
# Embedded standalone source modules (no local package import needed)
# ------------------------------------------------------------------
_EMBEDDED_NOTEBOOK_DEFAULTS = '"""Default notebook configuration for the Trading Desk Analytics Platform."""\n\nfrom __future__ import annotations\n\nimport os\nfrom typing import Any\n\n\ndef build_master_config() -> dict[str, Any]:\n    """Return the default master configuration dictionary."""\n    return {\n        "DATA_MODE": "synthetic",\n        "DB_CONNECTIONS": {\n            "risk_db": {\n                "type": "mssql",\n                "server": "your-server.database.windows.net",\n                "database": "RiskDB",\n                "schema": "dbo",\n                "auth": "windows",\n                "username": None,\n                "password": None,\n                "port": 1433,\n                "driver": "ODBC Driver 17 for SQL Server",\n                "trusted_connection": True,\n                "connection_string_override": None,\n            },\n            "market_data_db": {\n                "type": "postgresql",\n                "server": "market-data-prod.internal.com",\n                "database": "MarketDataDB",\n                "schema": "public",\n                "auth": "sql",\n                "username": os.environ.get("MKT_DB_USER"),\n                "password": os.environ.get("MKT_DB_PASS"),\n                "port": 5432,\n            },\n        },\n        "SQL_QUERIES": {\n            "positions": {\n                "connection": "risk_db",\n                "query": """\n                    SELECT * FROM dbo.Positions\n                    WHERE AsOfDate = {as_of_date}\n                    AND Book IN ({books})\n                """,\n                "params": {\n                    "as_of_date": "2025-01-31",\n                    "books": ["FX_DESK", "RATES_DESK", "EQ_VOL"],\n                },\n                "stored_proc": None,\n            },\n            "market_data": {\n                "connection": "market_data_db",\n                "query": "SELECT * FROM prices WHERE date >= {start_date}",\n                "params": {"start_date": "2024-01-01"},\n            },\n            "pnl_daily": {\n                "connection": "risk_db",\n                "query": "SELECT * FROM dbo.DailyPnL WHERE TradeDate BETWEEN {start} AND {end}",\n                "params": {"start": "2024-01-01", "end": "2025-01-31"},\n            },\n            "risk_sensitivities": {\n                "connection": "risk_db",\n                "query": "SELECT * FROM dbo.RiskSensitivities WHERE AsOfDate = {as_of_date}",\n                "params": {"as_of_date": "2025-01-31"},\n            },\n            "var_results": {\n                "connection": "risk_db",\n                "query": "SELECT * FROM dbo.VaR_Results WHERE RunDate = {as_of_date}",\n                "params": {"as_of_date": "2025-01-31"},\n            },\n        },\n        "FILE_INPUTS": {\n            "positions": "output/jupyter-notebook/data/my_positions.csv",\n            "market_data": "output/jupyter-notebook/data/market_data.xlsx",\n            "pnl_daily": "output/jupyter-notebook/data/daily_pnl.csv",\n        },\n        "MANUAL_POSITIONS": [\n            {\n                "product": "equity_cash",\n                "ticker": "AAPL",\n                "quantity": 1000,\n                "entry_price": 185.50,\n                "currency": "USD",\n                "desk": "EQ_LONG_SHORT",\n            },\n            {\n                "product": "equity_cash",\n                "ticker": "NVDA",\n                "quantity": -500,\n                "entry_price": 890.00,\n                "currency": "USD",\n                "desk": "EQ_LONG_SHORT",\n            },\n            {\n                "product": "equity_option",\n                "ticker": "SPY",\n                "option_type": "call",\n                "strike": 500,\n                "expiry": "2025-06-20",\n                "quantity": 100,\n                "entry_price": 12.50,\n                "iv": 0.18,\n                "desk": "EQ_VOL",\n            },\n            {\n                "product": "fx_spot",\n                "pair": "EURUSD",\n                "notional": 10_000_000,\n                "entry_rate": 1.0850,\n                "direction": "long",\n                "desk": "FX_G10",\n            },\n            {\n                "product": "irs",\n                "currency": "USD",\n                "notional": 25_000_000,\n                "fixed_rate": 3.85,\n                "float_index": "SOFR",\n                "tenor": "5Y",\n                "direction": "pay_fixed",\n                "desk": "RATES",\n            },\n        ],\n        "API_CONFIG": {\n            "yfinance": {"enabled": True},\n            "fred": {"enabled": True, "api_key": os.environ.get("FRED_API_KEY")},\n            "alpha_vantage": {"enabled": False, "api_key": None},\n            "bloomberg": {"enabled": False, "host": None, "port": None},\n            "refinitiv": {"enabled": False},\n        },\n        "SYNTHETIC_CONFIG": {\n            "num_positions": 240,\n            "seed": 42,\n            "as_of_date": "2025-01-31",\n            "history_days": 756,\n            "asset_class_weights": {\n                "equities": 0.30,\n                "fixed_income": 0.20,\n                "fx": 0.15,\n                "commodities": 0.10,\n                "equity_options": 0.15,\n                "credit": 0.05,\n                "exotics": 0.05,\n            },\n            "desks": [\n                "EQ_LONG_SHORT",\n                "EQ_VOL",\n                "RATES",\n                "FX_G10",\n                "FX_EM",\n                "CREDIT",\n                "COMMODITIES",\n                "MACRO",\n                "RELATIVE_VALUE",\n                "STRUCTURED_PRODUCTS",\n            ],\n            "include_pnl_history": True,\n            "include_risk_sensitivities": True,\n            "include_greeks": True,\n            "realistic_correlations": True,\n        },\n        "COLUMN_MAPPINGS": {\n            "positions": {\n                "TradeID": "position_id",\n                "InstrumentType": "product_type",\n                "AssetCls": "asset_class",\n                "Notl": "quantity_notional",\n                "Px": "current_price",\n            }\n        },\n        "BASE_CURRENCY": "USD",\n        "AS_OF_DATE": "2025-01-31",\n        "START_DATE": "2022-01-01",\n        "CONFIDENCE_LEVEL": 0.99,\n        "RISK_FREE_RATE": 0.0435,\n        "TRADING_DAYS_PER_YEAR": 252,\n        "CACHE_DIR": "output/jupyter-notebook/cache",\n        "REFRESH_CACHE": False,\n        "QUERY_TIMEOUT": 120,\n        "DB_MAX_RETRIES": 3,\n        "DB_BACKOFF_SECONDS": 1.0,\n        "DB_POOL_SIZE": 5,\n        "DB_MAX_OVERFLOW": 10,\n        "STALE_DAYS": 5,\n        "OUTLIER_SIGMA": 5.0,\n    }\n'
_EMBEDDED_CORE = '"""Core components for the trading desk analytics notebook platform.\n\nThis module is intentionally self-contained so the notebooks can be thin, readable,\nand source-agnostic. It provides:\n- Multi-database connectivity with retry logic and pooling\n- Synthetic multi-asset portfolio + market/risk/PnL generation\n- Canonical schema normalization across all data sources\n- Data quality validation checks\n- Caching/export/snapshot helpers\n- A unified DataEngine abstraction used by notebook sections\n"""\n\nfrom __future__ import annotations\n\nimport datetime as dt\nimport json\nimport logging\nimport math\nimport os\nimport random\nimport re\nimport time\nfrom dataclasses import dataclass\nfrom pathlib import Path\nfrom typing import Any, Iterable\nfrom urllib.parse import quote_plus\n\nimport numpy as np\nimport pandas as pd\nfrom numpy.typing import NDArray\nfrom pandas import DataFrame\nfrom scipy import stats\n\ntry:\n    from sqlalchemy import create_engine, text\n    from sqlalchemy.engine import Engine\nexcept Exception:  # pragma: no cover - optional dependency for non-db mode\n    Engine = Any  # type: ignore[misc,assignment]\n    create_engine = None  # type: ignore[assignment]\n    text = None  # type: ignore[assignment]\n\ntry:  # pragma: no cover - optional\n    import yfinance as yf\nexcept Exception:  # pragma: no cover\n    yf = None\n\ntry:  # pragma: no cover - optional\n    from fredapi import Fred\nexcept Exception:  # pragma: no cover\n    Fred = None\n\nCANONICAL_POSITION_COLUMNS = [\n    "position_id",\n    "product_type",\n    "asset_class",\n    "sub_class",\n    "ticker_identifier",\n    "desk",\n    "book",\n    "strategy",\n    "direction",\n    "quantity_notional",\n    "entry_price",\n    "current_price",\n    "entry_date",\n    "maturity_expiry",\n    "currency",\n    "counterparty",\n    "trade_date",\n    "settlement_date",\n    "pnl_inception",\n    "pnl_daily",\n]\n\nCANONICAL_MARKET_COLUMNS = [\n    "date",\n    "identifier",\n    "asset_class",\n    "open",\n    "high",\n    "low",\n    "close",\n    "volume",\n    "adjusted_close",\n    "return_1d",\n    "return_log",\n]\n\nCANONICAL_RISK_COLUMNS = [\n    "position_id",\n    "delta",\n    "gamma",\n    "vega",\n    "theta",\n    "rho",\n    "dv01",\n    "cs01",\n    "convexity",\n    "vanna",\n    "volga",\n    "charm",\n    "speed",\n    "color",\n    "zomma",\n    "ultima",\n    "var_contribution",\n]\n\nCANONICAL_PNL_COLUMNS = [\n    "date",\n    "position_id",\n    "desk",\n    "asset_class",\n    "total_pnl",\n    "delta_pnl",\n    "gamma_pnl",\n    "theta_pnl",\n    "vega_pnl",\n    "carry_pnl",\n    "roll_pnl",\n    "fx_pnl",\n    "residual_pnl",\n    "new_trade_pnl",\n    "existing_position_pnl",\n]\n\nDATASET_ALIASES = {\n    "positions": "positions",\n    "position": "positions",\n    "market_data": "market_data",\n    "prices": "market_data",\n    "risk_sensitivities": "risk_sensitivities",\n    "greeks": "risk_sensitivities",\n    "pnl": "pnl_daily",\n    "pnl_daily": "pnl_daily",\n    "var": "var_results",\n    "var_results": "var_results",\n    "yield_curves": "yield_curves",\n    "curves": "yield_curves",\n    "vol_surface": "vol_surface",\n    "vol_surfaces": "vol_surface",\n    "credit_curves": "credit_curves",\n}\n\nEQUITY_TICKERS = [\n    "AAPL",\n    "MSFT",\n    "NVDA",\n    "AMZN",\n    "META",\n    "TSLA",\n    "GOOGL",\n    "JPM",\n    "XOM",\n    "UNH",\n    "PG",\n    "JNJ",\n    "V",\n    "MA",\n    "HD",\n    "NVO",\n    "ASML",\n    "SAP",\n    "NESN.SW",\n    "RDSA.AS",\n    "BABA",\n    "TSM",\n    "SONY",\n    "SHOP",\n    "NFLX",\n]\n\nINDEX_TICKERS = ["SPX", "NDX", "RUT", "SX5E", "FTSE", "N225", "HSI"]\nFX_PAIRS_G10 = ["EURUSD", "USDJPY", "GBPUSD", "USDCHF", "AUDUSD", "USDCAD", "NZDUSD", "EURJPY"]\nFX_PAIRS_EM = ["USDBRL", "USDZAR", "USDMXN", "USDTRY", "USDINR", "USDCNH", "USDKRW"]\nCOMMODITIES = ["CL", "NG", "GC", "SI", "HG", "ZC", "ZW", "ZS"]\nCREDIT_NAMES_IG = ["AAPL", "MSFT", "JPM", "V", "PG", "KO", "PEP", "IBM"]\nCREDIT_NAMES_HY = ["F", "CCL", "M", "AAL", "LCID", "RBLX", "RIVN"]\nCOUNTERPARTIES = [\n    "GS",\n    "JPM",\n    "MS",\n    "CITI",\n    "BAML",\n    "BARX",\n    "DB",\n    "UBS",\n    "SOCGEN",\n    "BNPP",\n]\n\nDEFAULT_DESKS = [\n    "EQ_LONG_SHORT",\n    "EQ_VOL",\n    "RATES",\n    "FX_G10",\n    "FX_EM",\n    "CREDIT",\n    "COMMODITIES",\n    "MACRO",\n    "RELATIVE_VALUE",\n    "STRUCTURED_PRODUCTS",\n]\n\n\ndef get_logger(name: str = "trading_desk_analytics", level: int = logging.INFO) -> logging.Logger:\n    """Return a configured module logger."""\n    logger = logging.getLogger(name)\n    if not logger.handlers:\n        handler = logging.StreamHandler()\n        handler.setFormatter(\n            logging.Formatter("%(asctime)s | %(name)s | %(levelname)s | %(message)s")\n        )\n        logger.addHandler(handler)\n    logger.setLevel(level)\n    logger.propagate = False\n    return logger\n\n\ndef _coerce_datetime(series: pd.Series) -> pd.Series:\n    return pd.to_datetime(series, errors="coerce", utc=False)\n\n\ndef _safe_float(value: Any, default: float = 0.0) -> float:\n    try:\n        if value is None:\n            return default\n        return float(value)\n    except Exception:\n        return default\n\n\ndef _direction_to_sign(value: Any) -> int:\n    if isinstance(value, (int, float)):\n        return 1 if float(value) >= 0 else -1\n    text_value = str(value).strip().lower()\n    if text_value in {"long", "buy", "receive_fixed", "buy_protection", "payer", "+1"}:\n        return 1\n    if text_value in {"short", "sell", "pay_fixed", "sell_protection", "receiver", "-1"}:\n        return -1\n    return 1\n\n\n@dataclass\nclass RetryConfig:\n    max_retries: int = 3\n    backoff_seconds: float = 1.0\n\n\nclass DatabaseConnector:\n    """Database connectivity wrapper with pooling, retries, and safe execution."""\n\n    def __init__(\n        self,\n        connections: dict[str, dict[str, Any]],\n        query_timeout: int = 120,\n        retry_config: RetryConfig | None = None,\n        pool_size: int = 5,\n        max_overflow: int = 10,\n        echo: bool = False,\n    ) -> None:\n        self.connections = connections or {}\n        self.query_timeout = query_timeout\n        self.retry_config = retry_config or RetryConfig()\n        self.pool_size = pool_size\n        self.max_overflow = max_overflow\n        self.echo = echo\n        self.logger = get_logger("trading_desk_analytics.db")\n        self._engines: dict[str, Engine] = {}\n\n    def close(self) -> None:\n        """Dispose all cached SQLAlchemy engines."""\n        for name, engine in self._engines.items():\n            try:\n                engine.dispose()\n                self.logger.info("Disposed engine for connection \'%s\'", name)\n            except Exception as exc:  # pragma: no cover - defensive\n                self.logger.warning("Failed to dispose engine %s: %s", name, exc)\n        self._engines.clear()\n\n    def _resolve_credentials(self, config: dict[str, Any]) -> tuple[str | None, str | None]:\n        auth = str(config.get("auth", "sql")).lower()\n        username = config.get("username")\n        password = config.get("password")\n\n        if auth == "env_var":\n            username_env = config.get("username_env", "DB_USER")\n            password_env = config.get("password_env", "DB_PASS")\n            username = os.environ.get(str(username_env))\n            password = os.environ.get(str(password_env))\n\n        if auth in {"windows", "kerberos"}:\n            return None, None\n\n        return username, password\n\n    def _build_url(self, connection_name: str) -> str:\n        if create_engine is None:\n            raise ImportError(\n                "sqlalchemy is not installed. Install it to use database mode."\n            )\n\n        if connection_name not in self.connections:\n            raise KeyError(f"Unknown connection \'{connection_name}\'.")\n\n        config = self.connections[connection_name]\n        override = config.get("connection_string_override")\n        if override:\n            return str(override)\n\n        db_type = str(config.get("type", "")).lower()\n        server = str(config.get("server", ""))\n        database = str(config.get("database", ""))\n        port = int(config.get("port", 0) or 0)\n        driver = str(config.get("driver", ""))\n        username, password = self._resolve_credentials(config)\n        auth = str(config.get("auth", "sql")).lower()\n\n        if db_type in {"mssql", "sqlserver"}:\n            if auth in {"windows", "kerberos"} or bool(config.get("trusted_connection")):\n                odbc = (\n                    f"DRIVER={{{driver or \'ODBC Driver 17 for SQL Server\'}}};"\n                    f"SERVER={server},{port or 1433};"\n                    f"DATABASE={database};"\n                    "Trusted_Connection=yes;"\n                )\n                return f"mssql+pyodbc:///?odbc_connect={quote_plus(odbc)}"\n\n            if auth == "aad_token":\n                # SQLAlchemy token flows are environment-specific; callers can supply an\n                # override string via `connection_string_override` for production token auth.\n                self.logger.warning(\n                    "AAD token auth requested for %s. Using SQL auth fields as fallback. "\n                    "Prefer connection_string_override with a pre-configured token flow.",\n                    connection_name,\n                )\n\n            if not username:\n                raise ValueError(f"Missing username for MSSQL connection \'{connection_name}\'.")\n            password_enc = quote_plus(str(password or ""))\n            driver_enc = quote_plus(driver or "ODBC Driver 17 for SQL Server")\n            return (\n                f"mssql+pyodbc://{quote_plus(str(username))}:{password_enc}@"\n                f"{server}:{port or 1433}/{database}?driver={driver_enc}"\n            )\n\n        if db_type in {"postgres", "postgresql"}:\n            if not username:\n                raise ValueError(f"Missing username for PostgreSQL connection \'{connection_name}\'.")\n            password_enc = quote_plus(str(password or ""))\n            return (\n                f"postgresql+psycopg2://{quote_plus(str(username))}:{password_enc}@"\n                f"{server}:{port or 5432}/{database}"\n            )\n\n        if db_type == "oracle":\n            if not username:\n                raise ValueError(f"Missing username for Oracle connection \'{connection_name}\'.")\n            password_enc = quote_plus(str(password or ""))\n            return (\n                f"oracle+cx_oracle://{quote_plus(str(username))}:{password_enc}@"\n                f"{server}:{port or 1521}/?service_name={database}"\n            )\n\n        if db_type in {"mysql", "mariadb"}:\n            if not username:\n                raise ValueError(f"Missing username for MySQL connection \'{connection_name}\'.")\n            password_enc = quote_plus(str(password or ""))\n            return (\n                f"mysql+pymysql://{quote_plus(str(username))}:{password_enc}@"\n                f"{server}:{port or 3306}/{database}"\n            )\n\n        if db_type == "sqlite":\n            path = config.get("database", ":memory:")\n            path_str = str(path)\n            if path_str != ":memory:" and not Path(path_str).is_absolute():\n                path_str = str((Path.cwd() / path_str).resolve())\n            return f"sqlite:///{path_str}"\n\n        if db_type in {"odbc", "jdbc"}:\n            dsn = config.get("dsn")\n            if dsn:\n                return f"mssql+pyodbc:///?odbc_connect={quote_plus(str(dsn))}"\n            raise ValueError(\n                f"Generic ODBC/JDBC connection \'{connection_name}\' requires `dsn` "\n                "or `connection_string_override`."\n            )\n\n        raise ValueError(f"Unsupported database type: \'{db_type}\'")\n\n    def get_engine(self, connection_name: str) -> Engine:\n        """Get or build a pooled SQLAlchemy engine."""\n        if connection_name in self._engines:\n            return self._engines[connection_name]\n\n        url = self._build_url(connection_name)\n        engine = create_engine(  # type: ignore[misc]\n            url,\n            echo=self.echo,\n            pool_pre_ping=True,\n            pool_size=self.pool_size,\n            max_overflow=self.max_overflow,\n            pool_recycle=1800,\n            future=True,\n        )\n        self._engines[connection_name] = engine\n        self.logger.info("Created engine for \'%s\'", connection_name)\n        return engine\n\n    def health_check(self) -> dict[str, bool]:\n        """Run a lightweight connectivity check (`SELECT 1`) per configured connection."""\n        results: dict[str, bool] = {}\n        for name in self.connections:\n            try:\n                engine = self.get_engine(name)\n                with engine.connect() as conn:\n                    conn.execute(text("SELECT 1"))  # type: ignore[misc]\n                results[name] = True\n            except Exception as exc:\n                self.logger.warning("Health check failed for %s: %s", name, exc)\n                results[name] = False\n        return results\n\n    def _execute_with_retries(self, fn: Any, description: str) -> DataFrame:\n        attempts = self.retry_config.max_retries + 1\n        wait = self.retry_config.backoff_seconds\n        last_error: Exception | None = None\n\n        for attempt in range(1, attempts + 1):\n            try:\n                return fn()\n            except Exception as exc:  # pragma: no cover - depends on driver\n                if isinstance(exc, (ImportError, ModuleNotFoundError)):\n                    raise RuntimeError(\n                        f"{description} failed due to missing dependency: {exc}"\n                    ) from exc\n                last_error = exc\n                if attempt >= attempts:\n                    break\n                self.logger.warning(\n                    "%s failed on attempt %s/%s: %s. Retrying in %.1fs.",\n                    description,\n                    attempt,\n                    attempts,\n                    exc,\n                    wait,\n                )\n                time.sleep(wait)\n                wait *= 2\n\n        raise RuntimeError(f"{description} failed after {attempts} attempts: {last_error}")\n\n    def _coerce_result_dtypes(self, df: DataFrame) -> DataFrame:\n        output = df.copy()\n        for column in output.columns:\n            if any(keyword in column.lower() for keyword in ["date", "time", "expiry", "maturity"]):\n                output[column] = pd.to_datetime(output[column], errors="ignore")\n        return output\n\n    def run_query(\n        self,\n        connection_name: str,\n        query: str,\n        params: dict[str, Any] | None = None,\n        timeout_seconds: int | None = None,\n    ) -> DataFrame:\n        """Execute a parameterized SQL query and return a DataFrame."""\n\n        def _runner() -> DataFrame:\n            engine = self.get_engine(connection_name)\n            start = time.perf_counter()\n            with engine.connect() as conn:\n                if timeout_seconds is not None:\n                    conn = conn.execution_options(timeout=timeout_seconds)\n                frame = pd.read_sql(text(query), conn, params=params)  # type: ignore[misc]\n            elapsed = time.perf_counter() - start\n            self.logger.info(\n                "Query on \'%s\' finished in %.2fs with %s rows",\n                connection_name,\n                elapsed,\n                len(frame),\n            )\n            return self._coerce_result_dtypes(frame)\n\n        return self._execute_with_retries(\n            _runner,\n            description=f"Query execution on {connection_name}",\n        )\n\n    def run_stored_procedure(\n        self,\n        connection_name: str,\n        proc_name: str,\n        params: dict[str, Any] | None = None,\n    ) -> DataFrame:\n        """Execute a stored procedure as a parameterized `EXEC` statement."""\n        params = params or {}\n        param_sql = ", ".join([f"@{key}=:{key}" for key in params])\n        sql = f"EXEC {proc_name} {param_sql}" if param_sql else f"EXEC {proc_name}"\n        return self.run_query(connection_name=connection_name, query=sql, params=params)\n\n    def run_batch(\n        self,\n        batch_queries: dict[str, dict[str, Any]],\n    ) -> dict[str, DataFrame]:\n        """Run a dictionary of batch queries and return keyed DataFrames."""\n        results: dict[str, DataFrame] = {}\n        for name, cfg in batch_queries.items():\n            connection = cfg["connection"]\n            query = cfg.get("query")\n            params = cfg.get("params", {})\n            stored_proc = cfg.get("stored_proc")\n            if stored_proc:\n                results[name] = self.run_stored_procedure(connection, stored_proc, params=params)\n            else:\n                if not query:\n                    raise ValueError(f"Batch query \'{name}\' missing `query` or `stored_proc`.")\n                results[name] = self.run_query(connection, query, params=params)\n        return results\n\n\nclass SyntheticDataGenerator:\n    """Generate realistic, internally-consistent synthetic cross-asset datasets."""\n\n    def __init__(\n        self,\n        config: dict[str, Any],\n        base_currency: str = "USD",\n        trading_days_per_year: int = 252,\n    ) -> None:\n        self.config = config\n        self.base_currency = base_currency\n        self.trading_days_per_year = trading_days_per_year\n        self.num_positions = int(config.get("num_positions", 200))\n        self.seed = int(config.get("seed", 42))\n        self.as_of_date = pd.Timestamp(config.get("as_of_date", dt.date.today()))\n        self.history_days = int(config.get("history_days", 756))\n        self.asset_class_weights = config.get(\n            "asset_class_weights",\n            {\n                "equities": 0.30,\n                "fixed_income": 0.20,\n                "fx": 0.15,\n                "commodities": 0.10,\n                "equity_options": 0.15,\n                "credit": 0.05,\n                "exotics": 0.05,\n            },\n        )\n        self.desks = config.get("desks", DEFAULT_DESKS)\n\n        self.logger = get_logger("trading_desk_analytics.synthetic")\n        self.rng = np.random.default_rng(self.seed)\n        random.seed(self.seed)\n\n    def _allocate_counts(self) -> dict[str, int]:\n        keys = list(self.asset_class_weights.keys())\n        weights = np.array([float(self.asset_class_weights[k]) for k in keys], dtype=float)\n        weights = np.clip(weights, 0, None)\n        if weights.sum() == 0:\n            weights = np.ones(len(weights))\n        weights = weights / weights.sum()\n        raw = weights * self.num_positions\n        counts = np.floor(raw).astype(int)\n        remainder = self.num_positions - int(counts.sum())\n        if remainder > 0:\n            idx = np.argsort(-(raw - counts))\n            for i in idx[:remainder]:\n                counts[i] += 1\n        return dict(zip(keys, counts.tolist()))\n\n    def _rand_date(self, start_days_back: int = 720, end_days_back: int = 1) -> pd.Timestamp:\n        days = int(self.rng.integers(end_days_back, start_days_back + 1))\n        return self.as_of_date - pd.Timedelta(days=days)\n\n    def _rand_maturity(self, min_days: int = 30, max_days: int = 3650) -> pd.Timestamp:\n        days = int(self.rng.integers(min_days, max_days + 1))\n        return self.as_of_date + pd.Timedelta(days=days)\n\n    def _pick(self, values: list[str] | tuple[str, ...]) -> str:\n        idx = int(self.rng.integers(0, len(values)))\n        return str(values[idx])\n\n    def _build_position_common(self, idx: int, product: str, asset_class: str) -> dict[str, Any]:\n        entry_date = self._rand_date()\n        maturity = self._rand_maturity()\n        direction = "long" if self.rng.random() > 0.45 else "short"\n        quantity = float(np.round(self.rng.lognormal(mean=8, sigma=1.0), 2))\n        entry_price = float(np.round(max(0.01, self.rng.normal(100, 25)), 4))\n        current_price = float(np.round(entry_price * (1 + self.rng.normal(0, 0.1)), 4))\n\n        return {\n            "position_id": f"POS{idx:06d}",\n            "product_type": product,\n            "asset_class": asset_class,\n            "sub_class": "",\n            "ticker_identifier": "",\n            "desk": self._pick(self.desks),\n            "book": f"BOOK_{self._pick([\'A\', \'B\', \'C\', \'D\', \'E\'])}",\n            "strategy": self._pick(\n                [\n                    "carry",\n                    "momentum",\n                    "mean_reversion",\n                    "relative_value",\n                    "event_driven",\n                    "volatility",\n                    "macro",\n                    "dispersion",\n                ]\n            ),\n            "direction": direction,\n            "quantity_notional": quantity if direction == "long" else -quantity,\n            "entry_price": entry_price,\n            "current_price": current_price,\n            "entry_date": entry_date,\n            "maturity_expiry": maturity,\n            "currency": self.base_currency,\n            "counterparty": self._pick(COUNTERPARTIES),\n            "trade_date": entry_date,\n            "settlement_date": entry_date + pd.Timedelta(days=2),\n            "pnl_inception": float(np.round(self.rng.normal(0, 1), 6)),\n            "pnl_daily": float(np.round(self.rng.normal(0, 0.25), 6)),\n        }\n\n    def _generate_equity_like(self, idx: int, product: str) -> dict[str, Any]:\n        position = self._build_position_common(idx, product, "equities")\n        ticker = self._pick(EQUITY_TICKERS)\n        country = self._pick(["US", "UK", "JP", "DE", "FR", "CH", "CN"])\n        sector = self._pick(\n            [\n                "Technology",\n                "Financials",\n                "Energy",\n                "Healthcare",\n                "Industrials",\n                "Consumer",\n                "Materials",\n                "Utilities",\n            ]\n        )\n\n        position.update(\n            {\n                "sub_class": "single_stock",\n                "ticker_identifier": ticker,\n                "country": country,\n                "sector": sector,\n                "dividend_yield": float(np.round(np.clip(self.rng.normal(0.018, 0.01), 0, 0.08), 4)),\n                "beta": float(np.round(np.clip(self.rng.normal(1.0, 0.25), 0.4, 2.0), 4)),\n            }\n        )\n\n        if product in {"equity_index_future", "etf"}:\n            index = self._pick(INDEX_TICKERS)\n            position["sub_class"] = "index_linked"\n            position["ticker_identifier"] = index\n            position["contract_multiplier"] = int(self.rng.choice([50, 100, 250]))\n            position["expiry_cycle"] = self._pick(["H", "M", "U", "Z"])\n\n        if product in {"equity_swap", "total_return_swap"}:\n            position["financing_spread_bps"] = float(np.round(self.rng.normal(45, 15), 2))\n            position["reset_frequency"] = self._pick(["daily", "weekly", "monthly"])\n\n        if product == "adr":\n            position["adr_ratio"] = float(np.round(self.rng.choice([0.5, 1.0, 2.0, 5.0]), 2))\n            position["local_ticker"] = ticker + "-LOCAL"\n\n        return position\n\n    def _generate_equity_option_like(self, idx: int, product: str) -> dict[str, Any]:\n        position = self._build_position_common(idx, product, "equity_options")\n        underlying = self._pick(EQUITY_TICKERS + INDEX_TICKERS)\n        spot = float(np.round(max(5.0, self.rng.normal(120, 45)), 4))\n        tte_days = int(self.rng.integers(15, 540))\n        expiry = self.as_of_date + pd.Timedelta(days=tte_days)\n        moneyness = float(np.round(self.rng.choice([0.8, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2]), 2))\n\n        position.update(\n            {\n                "sub_class": "vanilla_option",\n                "ticker_identifier": underlying,\n                "underlying": underlying,\n                "spot": spot,\n                "strike": float(np.round(spot * moneyness, 4)),\n                "option_type": self._pick(["call", "put"]),\n                "style": self._pick(["European", "American"]),\n                "expiry": expiry,\n                "maturity_expiry": expiry,\n                "implied_vol": float(np.round(np.clip(self.rng.normal(0.24, 0.09), 0.07, 1.25), 4)),\n                "contract_multiplier": int(self.rng.choice([50, 100, 250])),\n            }\n        )\n\n        if product in {"variance_swap", "volatility_swap"}:\n            position["sub_class"] = "vol_derivative"\n            position["vega_notional"] = float(np.round(self.rng.lognormal(10.5, 0.75), 2))\n            position["strike_variance"] = float(np.round(np.clip(self.rng.normal(0.05, 0.02), 0.01, 0.20), 6))\n\n        return position\n\n    def _generate_rates_like(self, idx: int, product: str) -> dict[str, Any]:\n        position = self._build_position_common(idx, product, "fixed_income")\n        currency = self._pick(["USD", "EUR", "GBP", "JPY"])\n        tenors = [0.5, 1, 2, 3, 5, 7, 10, 20, 30]\n        tenor = float(self.rng.choice(tenors))\n\n        position.update(\n            {\n                "currency": currency,\n                "tenor_years": tenor,\n                "day_count": self._pick(["ACT/360", "ACT/365", "30/360"]),\n                "payment_frequency": self._pick(["1M", "3M", "6M", "1Y"]),\n                "rate_index": self._pick(["SOFR", "ESTR", "SONIA", "TONAR"]),\n                "curve_id": f"{currency}_OIS",\n            }\n        )\n\n        if product in {"govt_bond", "corporate_bond", "tips", "mbs", "muni_bond"}:\n            coupon = float(np.round(np.clip(self.rng.normal(0.04, 0.015), 0.0, 0.09), 5))\n            ytm = float(np.round(np.clip(coupon + self.rng.normal(0, 0.01), -0.01, 0.15), 5))\n            position.update(\n                {\n                    "sub_class": "cash_bond",\n                    "identifier": f"{currency}_{int(tenor)}Y_{idx}",\n                    "ticker_identifier": f"{currency}_{int(tenor)}Y",\n                    "coupon": coupon,\n                    "yield_to_maturity": ytm,\n                    "accrued_interest": float(np.round(self.rng.uniform(0, 2), 6)),\n                }\n            )\n            if product == "corporate_bond":\n                position["credit_rating"] = self._pick(["AAA", "AA", "A", "BBB", "BB", "B"])\n                position["spread_bps"] = float(np.round(np.clip(self.rng.normal(135, 85), 20, 800), 2))\n            if product == "tips":\n                position["inflation_index_ratio"] = float(\n                    np.round(np.clip(self.rng.normal(1.05, 0.06), 0.8, 1.35), 6)\n                )\n            if product == "mbs":\n                position["cpr"] = float(np.round(np.clip(self.rng.normal(0.08, 0.03), 0.01, 0.35), 4))\n                position["oas_bps"] = float(np.round(np.clip(self.rng.normal(65, 20), 10, 200), 2))\n            if product == "muni_bond":\n                position["tax_rate"] = float(np.round(np.clip(self.rng.normal(0.28, 0.05), 0.1, 0.45), 4))\n\n        elif product in {"irs", "ois"}:\n            fixed_rate = float(np.round(np.clip(self.rng.normal(0.038, 0.012), -0.002, 0.1), 6))\n            position.update(\n                {\n                    "sub_class": "swap",\n                    "ticker_identifier": f"{currency}_{int(tenor)}Y_{product.upper()}",\n                    "fixed_rate": fixed_rate,\n                    "float_spread_bps": float(np.round(np.clip(self.rng.normal(5, 7), -20, 50), 2)),\n                    "direction": self._pick(["pay_fixed", "receive_fixed"]),\n                }\n            )\n\n        elif product == "fra":\n            position.update(\n                {\n                    "sub_class": "fra",\n                    "ticker_identifier": f"{currency}_FRA_{int(tenor * 12)}M",\n                    "fra_rate": float(np.round(np.clip(self.rng.normal(0.04, 0.012), 0.0, 0.11), 6)),\n                    "start_in_months": int(self.rng.integers(1, 12)),\n                }\n            )\n\n        elif product in {"cap", "floor", "swaption", "bond_future"}:\n            position.update(\n                {\n                    "sub_class": "rates_derivative",\n                    "ticker_identifier": f"{currency}_{product}_{int(tenor)}Y",\n                    "strike": float(np.round(np.clip(self.rng.normal(0.04, 0.01), 0.0, 0.1), 6)),\n                    "implied_vol": float(np.round(np.clip(self.rng.normal(0.35, 0.1), 0.1, 1.0), 6)),\n                    "option_type": "cap" if product == "cap" else "floor",\n                }\n            )\n            if product == "swaption":\n                position["option_type"] = self._pick(["payer", "receiver"])\n            if product == "bond_future":\n                position["ctd_candidate"] = self._pick([f"{currency}_2Y", f"{currency}_5Y", f"{currency}_10Y"])\n                position["conversion_factor"] = float(np.round(np.clip(self.rng.normal(0.87, 0.05), 0.7, 1.2), 5))\n\n        return position\n\n    def _generate_credit_like(self, idx: int, product: str) -> dict[str, Any]:\n        position = self._build_position_common(idx, product, "credit")\n        is_hy = self.rng.random() < 0.35\n        name = self._pick(CREDIT_NAMES_HY if is_hy else CREDIT_NAMES_IG)\n        tenor = self._pick(["1Y", "3Y", "5Y", "7Y", "10Y"])\n        spread = float(np.round(np.clip(self.rng.normal(95 if not is_hy else 320, 90), 20, 1200), 2))\n\n        position.update(\n            {\n                "sub_class": "cds" if product == "cds" else "credit_bond",\n                "ticker_identifier": name,\n                "reference_entity": name,\n                "tenor": tenor,\n                "spread_bps": spread,\n                "direction": self._pick(["buy_protection", "sell_protection"]),\n                "recovery_rate": float(np.round(np.clip(self.rng.normal(0.4, 0.1), 0.1, 0.75), 4)),\n                "rating_bucket": "HY" if is_hy else "IG",\n            }\n        )\n\n        if product == "credit_index_tranche":\n            position["attachment"] = float(np.round(self.rng.choice([0.0, 0.03, 0.07]), 4))\n            position["detachment"] = float(np.round(position["attachment"] + self.rng.choice([0.03, 0.05, 0.1]), 4))\n\n        return position\n\n    def _generate_fx_like(self, idx: int, product: str) -> dict[str, Any]:\n        position = self._build_position_common(idx, product, "fx")\n        pair = self._pick(FX_PAIRS_G10 + FX_PAIRS_EM)\n        base = pair[:3]\n        quote = pair[3:]\n        spot = float(np.round(np.clip(self.rng.normal(1.2, 0.45), 0.25, 190), 6))\n        tenor_days = int(self.rng.integers(7, 365))\n        forward_points = float(np.round(self.rng.normal(0.001, 0.005), 6))\n\n        position.update(\n            {\n                "sub_class": "spot" if product == "fx_spot" else "derivative",\n                "pair": pair,\n                "base_currency": base,\n                "quote_currency": quote,\n                "ticker_identifier": pair,\n                "spot_rate": spot,\n                "entry_rate": float(np.round(spot * (1 + self.rng.normal(0, 0.02)), 6)),\n                "forward_points": forward_points,\n                "forward_rate": float(np.round(spot + forward_points, 6)),\n                "tenor_days": tenor_days,\n            }\n        )\n\n        if product == "fx_option":\n            position["option_type"] = self._pick(["call", "put"])\n            position["strike"] = float(np.round(spot * self.rng.choice([0.9, 1.0, 1.1]), 6))\n            position["implied_vol"] = float(np.round(np.clip(self.rng.normal(0.12, 0.05), 0.04, 0.6), 6))\n\n        if product == "fx_swap":\n            position["near_leg_days"] = int(self.rng.integers(2, 30))\n            position["far_leg_days"] = int(self.rng.integers(31, 365))\n\n        return position\n\n    def _generate_commodity_like(self, idx: int, product: str) -> dict[str, Any]:\n        position = self._build_position_common(idx, product, "commodities")\n        commodity = self._pick(COMMODITIES)\n        month_code = self._pick(["F", "G", "H", "J", "K", "M", "N", "Q", "U", "V", "X", "Z"])\n        year = self.as_of_date.year + int(self.rng.integers(0, 3))\n        contract = f"{commodity}{month_code}{year}"\n\n        position.update(\n            {\n                "sub_class": "future" if product == "commodity_future" else "option",\n                "ticker_identifier": commodity,\n                "commodity": commodity,\n                "contract": contract,\n                "storage_cost": float(np.round(np.clip(self.rng.normal(0.02, 0.01), 0.0, 0.08), 5)),\n                "convenience_yield": float(np.round(np.clip(self.rng.normal(0.015, 0.01), 0.0, 0.07), 5)),\n            }\n        )\n\n        if product == "commodity_option":\n            position["option_type"] = self._pick(["call", "put"])\n            position["strike"] = float(np.round(position["current_price"] * self.rng.choice([0.85, 1.0, 1.15]), 4))\n            position["implied_vol"] = float(np.round(np.clip(self.rng.normal(0.28, 0.12), 0.08, 1.4), 6))\n\n        return position\n\n    def _generate_exotic_like(self, idx: int, product: str) -> dict[str, Any]:\n        position = self._build_position_common(idx, product, "exotics")\n        underlying = self._pick(EQUITY_TICKERS + INDEX_TICKERS + FX_PAIRS_G10 + COMMODITIES)\n        spot = float(np.round(np.clip(self.rng.normal(120, 60), 0.5, 450), 5))\n        strike = float(np.round(spot * self.rng.choice([0.9, 1.0, 1.1]), 5))\n\n        position.update(\n            {\n                "sub_class": "exotic_option",\n                "ticker_identifier": underlying,\n                "underlying": underlying,\n                "spot": spot,\n                "strike": strike,\n                "implied_vol": float(np.round(np.clip(self.rng.normal(0.35, 0.15), 0.12, 1.5), 6)),\n                "barrier": float(np.round(spot * self.rng.choice([0.75, 0.8, 1.2, 1.25]), 5)),\n                "observation_freq": self._pick(["daily", "weekly", "monthly"]),\n            }\n        )\n\n        if product == "convertible_bond":\n            position.update(\n                {\n                    "asset_class": "structured_products",\n                    "sub_class": "convertible_bond",\n                    "conversion_ratio": float(np.round(np.clip(self.rng.normal(12, 4), 3, 40), 5)),\n                    "bond_floor": float(np.round(np.clip(self.rng.normal(92, 6), 60, 110), 4)),\n                    "credit_spread_bps": float(np.round(np.clip(self.rng.normal(220, 90), 40, 1000), 2)),\n                    "coupon": float(np.round(np.clip(self.rng.normal(0.02, 0.01), 0.0, 0.07), 6)),\n                }\n            )\n\n        if product in {"variance_swap", "correlation_swap"}:\n            position["sub_class"] = "vol_swap"\n            position["vega_notional"] = float(np.round(self.rng.lognormal(10, 0.8), 2))\n\n        if product == "digital_option":\n            position["payout"] = float(np.round(np.clip(self.rng.normal(100, 30), 10, 300), 5))\n\n        return position\n\n    def generate_positions(self) -> DataFrame:\n        """Generate synthetic positions across all requested product families."""\n        counts = self._allocate_counts()\n        products_by_asset = {\n            "equities": ["equity_cash", "equity_index_future", "etf", "adr", "equity_swap", "total_return_swap"],\n            "equity_options": ["equity_option", "index_option", "variance_swap", "volatility_swap"],\n            "fixed_income": ["govt_bond", "corporate_bond", "irs", "ois", "fra", "cap", "floor", "swaption", "tips", "mbs", "muni_bond", "bond_future"],\n            "fx": ["fx_spot", "fx_forward", "fx_option", "fx_swap"],\n            "commodities": ["commodity_future", "commodity_option"],\n            "credit": ["cds", "credit_index_tranche"],\n            "exotics": [\n                "barrier_option",\n                "asian_option",\n                "digital_option",\n                "lookback_option",\n                "compound_option",\n                "quanto_option",\n                "chooser_option",\n                "cliquet_option",\n                "convertible_bond",\n                "correlation_swap",\n            ],\n        }\n\n        rows: list[dict[str, Any]] = []\n        idx = 1\n        for asset_class, count in counts.items():\n            products = products_by_asset.get(asset_class, [])\n            if not products:\n                continue\n\n            for _ in range(count):\n                product = self._pick(products)\n                if asset_class == "equities":\n                    row = self._generate_equity_like(idx, product)\n                elif asset_class == "equity_options":\n                    row = self._generate_equity_option_like(idx, product)\n                elif asset_class == "fixed_income":\n                    row = self._generate_rates_like(idx, product)\n                elif asset_class == "fx":\n                    row = self._generate_fx_like(idx, product)\n                elif asset_class == "commodities":\n                    row = self._generate_commodity_like(idx, product)\n                elif asset_class == "credit":\n                    row = self._generate_credit_like(idx, product)\n                else:\n                    row = self._generate_exotic_like(idx, product)\n                rows.append(row)\n                idx += 1\n\n        df = pd.DataFrame(rows)\n        if len(df) < self.num_positions:\n            deficit = self.num_positions - len(df)\n            extra = [self._generate_equity_like(idx + i, "equity_cash") for i in range(deficit)]\n            df = pd.concat([df, pd.DataFrame(extra)], ignore_index=True)\n\n        self.logger.info(\n            "Generated %s synthetic positions across %s asset classes",\n            len(df),\n            df["asset_class"].nunique(),\n        )\n        return df\n\n    def _generate_regime_series(self, n: int) -> NDArray[np.int_]:\n        """Simple 3-state Markov chain (calm, normal, stress)."""\n        transition = np.array(\n            [\n                [0.90, 0.09, 0.01],\n                [0.08, 0.84, 0.08],\n                [0.05, 0.20, 0.75],\n            ]\n        )\n        states = np.zeros(n, dtype=int)\n        for i in range(1, n):\n            prev = states[i - 1]\n            states[i] = int(self.rng.choice([0, 1, 2], p=transition[prev]))\n        return states\n\n    def _factor_returns(self, dates: pd.DatetimeIndex) -> DataFrame:\n        factors = ["equities", "rates", "fx", "commodities", "credit", "volatility", "crypto"]\n        n = len(dates)\n        regimes = self._generate_regime_series(n)\n\n        corr = np.array(\n            [\n                [1.00, -0.25, -0.08, 0.35, 0.55, -0.60, 0.45],\n                [-0.25, 1.00, 0.18, -0.20, -0.32, 0.24, -0.10],\n                [-0.08, 0.18, 1.00, 0.10, -0.14, 0.05, 0.18],\n                [0.35, -0.20, 0.10, 1.00, 0.32, -0.25, 0.28],\n                [0.55, -0.32, -0.14, 0.32, 1.00, -0.40, 0.22],\n                [-0.60, 0.24, 0.05, -0.25, -0.40, 1.00, -0.35],\n                [0.45, -0.10, 0.18, 0.28, 0.22, -0.35, 1.00],\n            ]\n        )\n        chol = np.linalg.cholesky(corr)\n\n        base_vol = np.array([0.010, 0.004, 0.005, 0.012, 0.008, 0.020, 0.030])\n        regime_mult = np.array([0.7, 1.0, 2.2])\n        mu = np.array([0.0003, -0.00002, 0.00005, 0.0002, 0.00015, 0.0001, 0.0005])\n\n        raw = np.zeros((n, len(factors)))\n        for t in range(n):\n            z = self.rng.standard_t(df=6, size=len(factors))\n            correlated = chol @ z\n            shock = np.zeros(len(factors))\n            if self.rng.random() < 0.012:\n                shock += self.rng.normal(0, 0.035, size=len(factors))\n            raw[t, :] = mu + correlated * base_vol * regime_mult[regimes[t]] + shock\n\n        # Add volatility clustering via a simple EWMA amplification.\n        ewma = np.zeros_like(raw)\n        alpha = 0.08\n        for t in range(1, n):\n            ewma[t] = alpha * np.abs(raw[t - 1]) + (1 - alpha) * ewma[t - 1]\n        adjusted = raw * (1 + 3.5 * ewma)\n\n        frame = pd.DataFrame(adjusted, index=dates, columns=factors)\n        frame["regime"] = regimes\n        return frame.reset_index(names="date")\n\n    def _identifier_return_series(\n        self,\n        positions: DataFrame,\n        factor_returns: DataFrame,\n    ) -> DataFrame:\n        dates = pd.to_datetime(factor_returns["date"])\n        factor_cols = [c for c in factor_returns.columns if c not in {"date", "regime"}]\n        factor_mat = factor_returns[factor_cols].to_numpy()\n\n        id_rows: list[dict[str, Any]] = []\n        grouped = positions.dropna(subset=["ticker_identifier"]).groupby("ticker_identifier", as_index=False)\n\n        for _, row in grouped.first().iterrows():\n            identifier = str(row["ticker_identifier"])\n            asset_class = str(row["asset_class"])\n\n            loading = np.zeros(len(factor_cols))\n            idio_vol = 0.006\n            if asset_class in {"equities", "equity_options", "structured_products"}:\n                loading[:] = [0.95, -0.12, -0.06, 0.12, 0.28, -0.42, 0.10]\n                idio_vol = 0.018\n            elif asset_class == "fixed_income":\n                loading[:] = [-0.18, 0.92, 0.15, -0.09, -0.22, 0.15, 0.03]\n                idio_vol = 0.005\n            elif asset_class == "fx":\n                loading[:] = [-0.08, 0.22, 0.96, 0.07, -0.11, 0.08, 0.04]\n                idio_vol = 0.008\n            elif asset_class == "commodities":\n                loading[:] = [0.26, -0.12, 0.08, 1.0, 0.12, -0.20, 0.06]\n                idio_vol = 0.016\n            elif asset_class == "credit":\n                loading[:] = [0.40, -0.25, -0.11, 0.16, 1.05, -0.30, 0.05]\n                idio_vol = 0.012\n            else:\n                loading[:] = [0.35, -0.20, -0.03, 0.10, 0.25, -0.45, 0.12]\n                idio_vol = 0.02\n\n            idio_noise = self.rng.normal(0, idio_vol, size=len(dates))\n            returns = factor_mat @ loading + idio_noise\n\n            start_price = float(np.clip(self.rng.normal(100, 35), 5, 1200))\n            if any(key in identifier for key in ["USDJPY", "USDKRW", "USDINR", "USDTRY"]):\n                start_price = float(np.clip(self.rng.normal(120, 25), 20, 220))\n            if identifier in COMMODITIES:\n                start_price = float(np.clip(self.rng.normal(70, 20), 8, 250))\n\n            close = start_price * np.exp(np.cumsum(returns))\n            open_px = close * (1 + self.rng.normal(0, 0.0012, size=len(close)))\n            high = np.maximum(open_px, close) * (1 + np.abs(self.rng.normal(0, 0.004, size=len(close))))\n            low = np.minimum(open_px, close) * (1 - np.abs(self.rng.normal(0, 0.004, size=len(close))))\n            volume = np.clip(self.rng.lognormal(mean=12.0, sigma=0.8, size=len(close)), 1000, 3e8)\n\n            frame = pd.DataFrame(\n                {\n                    "date": dates,\n                    "identifier": identifier,\n                    "asset_class": asset_class,\n                    "open": open_px,\n                    "high": high,\n                    "low": low,\n                    "close": close,\n                    "volume": volume,\n                }\n            )\n            frame["adjusted_close"] = frame["close"]\n            frame["return_1d"] = frame["close"].pct_change().fillna(0.0)\n            frame["return_log"] = np.log(frame["close"]).diff().fillna(0.0)\n            id_rows.append(frame)\n\n        market = pd.concat(id_rows, ignore_index=True)\n        return market\n\n    def _generate_yield_curves(self, dates: pd.DatetimeIndex) -> DataFrame:\n        tenors = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30])\n        currencies = ["USD", "EUR", "GBP", "JPY"]\n\n        rows: list[dict[str, Any]] = []\n        for ccy in currencies:\n            b0 = {\n                "USD": 0.041,\n                "EUR": 0.029,\n                "GBP": 0.038,\n                "JPY": 0.011,\n            }[ccy]\n\n            for date in dates[:: max(1, len(dates) // 80)]:\n                beta0 = b0 + self.rng.normal(0, 0.004)\n                beta1 = -0.02 + self.rng.normal(0, 0.003)\n                beta2 = 0.015 + self.rng.normal(0, 0.003)\n                beta3 = 0.005 + self.rng.normal(0, 0.002)\n                lam1 = 1.2\n                lam2 = 4.0\n\n                for t in tenors:\n                    x1 = (1 - np.exp(-t / lam1)) / (t / lam1)\n                    x2 = x1 - np.exp(-t / lam1)\n                    x3 = (1 - np.exp(-t / lam2)) / (t / lam2) - np.exp(-t / lam2)\n                    y = beta0 + beta1 * x1 + beta2 * x2 + beta3 * x3\n                    rows.append(\n                        {\n                            "date": pd.Timestamp(date),\n                            "currency": ccy,\n                            "tenor_years": float(t),\n                            "zero_rate": float(np.clip(y, -0.01, 0.16)),\n                        }\n                    )\n\n        curve_df = pd.DataFrame(rows)\n        curve_df["forward_rate"] = (\n            curve_df.sort_values(["currency", "tenor_years", "date"])\n            .groupby(["currency", "date"])["zero_rate"]\n            .diff()\n            .fillna(curve_df["zero_rate"])\n        )\n        return curve_df\n\n    def _generate_credit_curves(self, dates: pd.DatetimeIndex) -> DataFrame:\n        tenors = [1, 3, 5, 7, 10]\n        names = CREDIT_NAMES_IG + CREDIT_NAMES_HY\n        rows: list[dict[str, Any]] = []\n\n        for name in names:\n            base = 85 if name in CREDIT_NAMES_IG else 340\n            for date in dates[:: max(1, len(dates) // 80)]:\n                slope = self.rng.normal(8, 4)\n                for tenor in tenors:\n                    spread = float(np.clip(base + slope * tenor + self.rng.normal(0, 12), 25, 1400))\n                    rows.append(\n                        {\n                            "date": pd.Timestamp(date),\n                            "reference_entity": name,\n                            "tenor_years": float(tenor),\n                            "spread_bps": spread,\n                            "rating_bucket": "HY" if name in CREDIT_NAMES_HY else "IG",\n                        }\n                    )\n\n        return pd.DataFrame(rows)\n\n    def _generate_vol_surface(self) -> DataFrame:\n        underlyings = ["SPX", "NDX", "AAPL", "NVDA", "CL", "GC", "EURUSD", "USDJPY", "BTCUSD"]\n        expiries = np.array([7, 14, 30, 60, 90, 180, 365, 730], dtype=float)\n        moneyness_grid = np.array([0.7, 0.8, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3], dtype=float)\n\n        rows: list[dict[str, Any]] = []\n        for under in underlyings:\n            base = {\n                "SPX": 0.19,\n                "NDX": 0.23,\n                "AAPL": 0.31,\n                "NVDA": 0.45,\n                "CL": 0.38,\n                "GC": 0.22,\n                "EURUSD": 0.11,\n                "USDJPY": 0.12,\n                "BTCUSD": 0.65,\n            }.get(under, 0.25)\n\n            skew = -0.20 if under in {"SPX", "NDX", "AAPL", "NVDA"} else -0.08\n            term = 0.05\n            smile = 0.32\n\n            total_var_prev = np.zeros_like(moneyness_grid)\n            for expiry_days in sorted(expiries):\n                t = expiry_days / 365.0\n                for i, m in enumerate(moneyness_grid):\n                    raw_iv = base + skew * (m - 1.0) + smile * (m - 1.0) ** 2 + term * math.sqrt(t)\n                    raw_iv = float(np.clip(raw_iv + self.rng.normal(0, 0.01), 0.04, 2.5))\n                    total_var = raw_iv**2 * t\n                    total_var = max(total_var, total_var_prev[i] + 1e-6)\n                    iv = math.sqrt(total_var / t)\n                    total_var_prev[i] = total_var\n\n                    rows.append(\n                        {\n                            "as_of_date": self.as_of_date,\n                            "underlying": under,\n                            "expiry_days": int(expiry_days),\n                            "moneyness": float(m),\n                            "strike": float(100.0 * m),\n                            "implied_vol": float(iv),\n                            "total_variance": float(total_var),\n                        }\n                    )\n\n        return pd.DataFrame(rows)\n\n    def _generate_risk_sensitivities(self, positions: DataFrame) -> DataFrame:\n        rows: list[dict[str, Any]] = []\n        for _, row in positions.iterrows():\n            product = str(row.get("product_type", ""))\n            qty = abs(_safe_float(row.get("quantity_notional"), 1.0))\n            direction = _direction_to_sign(row.get("direction", 1))\n            price = max(_safe_float(row.get("current_price"), 100.0), 0.01)\n\n            is_option = any(\n                keyword in product\n                for keyword in [\n                    "option",\n                    "cap",\n                    "floor",\n                    "swaption",\n                    "variance",\n                    "volatility",\n                    "digital",\n                    "barrier",\n                    "asian",\n                    "lookback",\n                    "compound",\n                    "quanto",\n                    "chooser",\n                    "cliquet",\n                ]\n            )\n            is_rates = row.get("asset_class") == "fixed_income" or product in {\n                "irs",\n                "ois",\n                "fra",\n                "govt_bond",\n                "corporate_bond",\n                "muni_bond",\n                "mbs",\n                "tips",\n                "bond_future",\n            }\n            is_credit = row.get("asset_class") == "credit" or "cds" in product\n\n            scale = qty * price / 1000.0\n            delta = direction * scale * self.rng.normal(0.7, 0.25)\n            gamma = scale * self.rng.normal(0.0, 0.08) if is_option else scale * self.rng.normal(0.0, 0.01)\n            vega = scale * self.rng.normal(0.35, 0.20) if is_option else scale * self.rng.normal(0.02, 0.01)\n            theta = -abs(scale * self.rng.normal(0.05, 0.03)) if is_option else -abs(\n                scale * self.rng.normal(0.005, 0.003)\n            )\n            rho = scale * self.rng.normal(0.08, 0.04) if (is_option or is_rates) else scale * self.rng.normal(0.01, 0.005)\n            dv01 = scale * self.rng.normal(1.5, 0.8) if is_rates else scale * self.rng.normal(0.02, 0.01)\n            cs01 = scale * self.rng.normal(1.1, 0.5) if is_credit else scale * self.rng.normal(0.01, 0.005)\n            convexity = scale * self.rng.normal(0.5, 0.3) if is_rates else scale * self.rng.normal(0.02, 0.01)\n\n            row_out = {\n                "position_id": row["position_id"],\n                "delta": float(delta),\n                "gamma": float(gamma),\n                "vega": float(vega),\n                "theta": float(theta),\n                "rho": float(rho),\n                "dv01": float(dv01),\n                "cs01": float(cs01),\n                "convexity": float(convexity),\n                "vanna": float(vega * self.rng.normal(0.08, 0.03)),\n                "volga": float(vega * self.rng.normal(0.12, 0.04)),\n                "charm": float(delta * self.rng.normal(-0.05, 0.02)),\n                "speed": float(gamma * self.rng.normal(0.15, 0.08)),\n                "color": float(gamma * self.rng.normal(-0.10, 0.05)),\n                "zomma": float(gamma * self.rng.normal(0.12, 0.05)),\n                "ultima": float(vega * self.rng.normal(0.07, 0.04)),\n                "var_contribution": float(abs(scale * self.rng.normal(0.12, 0.07))),\n            }\n            rows.append(row_out)\n\n        risk = pd.DataFrame(rows)\n        total = risk["var_contribution"].sum()\n        if total > 0:\n            risk["var_contribution"] = risk["var_contribution"] / total\n        return risk\n\n    def _generate_position_pnl(\n        self,\n        positions: DataFrame,\n        market_data: DataFrame,\n        risk: DataFrame,\n    ) -> DataFrame:\n        market_sorted = market_data.sort_values(["identifier", "date"])\n        returns_map: dict[str, pd.Series] = {\n            identifier: grp.set_index("date")["return_1d"]\n            for identifier, grp in market_sorted.groupby("identifier")\n        }\n\n        risk_lookup = risk.set_index("position_id")\n\n        rows: list[dict[str, Any]] = []\n        for _, pos in positions.iterrows():\n            pid = pos["position_id"]\n            identifier = pos.get("ticker_identifier")\n            if identifier not in returns_map:\n                continue\n\n            returns = returns_map[str(identifier)]\n            notional = abs(_safe_float(pos.get("quantity_notional"), 1.0))\n            px = max(_safe_float(pos.get("current_price"), 1.0), 0.01)\n            direction = _direction_to_sign(pos.get("direction", 1))\n            exposure = notional * px * 0.01\n\n            greeks = risk_lookup.loc[pid] if pid in risk_lookup.index else None\n            gamma_scale = float(greeks["gamma"]) if greeks is not None else 0.0\n            theta_scale = float(greeks["theta"]) if greeks is not None else -0.01\n            vega_scale = float(greeks["vega"]) if greeks is not None else 0.0\n\n            vol_shock = returns.rolling(5).std().fillna(returns.std())\n            carry = exposure * 0.00002\n\n            delta_component = direction * exposure * returns\n            gamma_component = 0.5 * gamma_scale * (returns**2)\n            theta_component = theta_scale / self.trading_days_per_year\n            vega_component = vega_scale * (vol_shock - vol_shock.mean()) * 0.1\n            roll_component = exposure * self.rng.normal(0, 0.00003, size=len(returns))\n            fx_component = exposure * self.rng.normal(0, 0.00004, size=len(returns))\n            residual = exposure * self.rng.normal(0, 0.00025, size=len(returns))\n\n            total = (\n                delta_component\n                + gamma_component\n                + theta_component\n                + vega_component\n                + carry\n                + roll_component\n                + fx_component\n                + residual\n            )\n\n            entry_date = pd.Timestamp(pos.get("entry_date", self.as_of_date - pd.Timedelta(days=120)))\n            is_new_trade = returns.index <= (entry_date + pd.Timedelta(days=5))\n            new_trade_pnl = np.where(is_new_trade, total, 0.0)\n            existing_trade_pnl = total - new_trade_pnl\n\n            position_rows = pd.DataFrame(\n                {\n                    "date": returns.index,\n                    "position_id": pid,\n                    "desk": pos.get("desk"),\n                    "asset_class": pos.get("asset_class"),\n                    "total_pnl": total,\n                    "delta_pnl": delta_component,\n                    "gamma_pnl": gamma_component,\n                    "theta_pnl": theta_component,\n                    "vega_pnl": vega_component,\n                    "carry_pnl": carry,\n                    "roll_pnl": roll_component,\n                    "fx_pnl": fx_component,\n                    "residual_pnl": residual,\n                    "new_trade_pnl": new_trade_pnl,\n                    "existing_position_pnl": existing_trade_pnl,\n                }\n            )\n            rows.append(position_rows)\n\n        pnl = pd.concat(rows, ignore_index=True)\n        return pnl\n\n    def _generate_var_results(self, pnl_daily: DataFrame) -> DataFrame:\n        if pnl_daily.empty:\n            return pd.DataFrame(\n                columns=["run_date", "desk", "var_95", "var_99", "cvar_99", "observations"]\n            )\n\n        rows: list[dict[str, Any]] = []\n        for desk, grp in pnl_daily.groupby("desk"):\n            daily = grp.groupby("date")["total_pnl"].sum().dropna()\n            if len(daily) < 30:\n                continue\n            var_95 = -np.percentile(daily, 5)\n            var_99 = -np.percentile(daily, 1)\n            cvar_99 = -daily[daily <= np.percentile(daily, 1)].mean()\n            rows.append(\n                {\n                    "run_date": self.as_of_date,\n                    "desk": desk,\n                    "var_95": float(var_95),\n                    "var_99": float(var_99),\n                    "cvar_99": float(cvar_99),\n                    "observations": int(len(daily)),\n                }\n            )\n        return pd.DataFrame(rows)\n\n    def generate(self) -> dict[str, DataFrame]:\n        """Generate the complete synthetic data pack."""\n        positions = self.generate_positions()\n        dates = pd.bdate_range(end=self.as_of_date, periods=self.history_days)\n        factors = self._factor_returns(dates)\n        market_data = self._identifier_return_series(positions, factors)\n        yield_curves = self._generate_yield_curves(dates)\n        credit_curves = self._generate_credit_curves(dates)\n        vol_surface = self._generate_vol_surface()\n        risk = self._generate_risk_sensitivities(positions)\n        pnl_daily = self._generate_position_pnl(positions, market_data, risk)\n        var_results = self._generate_var_results(pnl_daily)\n\n        return {\n            "positions": positions,\n            "market_data": market_data,\n            "risk_sensitivities": risk,\n            "pnl_daily": pnl_daily,\n            "var_results": var_results,\n            "yield_curves": yield_curves,\n            "credit_curves": credit_curves,\n            "vol_surface": vol_surface,\n            "factor_returns": factors,\n        }\n\n\nclass DataNormalizer:\n    """Normalize arbitrary source datasets into canonical schemas."""\n\n    def __init__(self, column_mappings: dict[str, dict[str, str]] | None = None) -> None:\n        self.column_mappings = column_mappings or {}\n\n    def _rename(self, df: DataFrame, dataset: str) -> DataFrame:\n        mapping = self.column_mappings.get(dataset, {})\n        if not mapping:\n            return df.copy()\n        return df.rename(columns=mapping).copy()\n\n    def _ensure_columns(self, df: DataFrame, required: list[str]) -> DataFrame:\n        output = df.copy()\n        for col in required:\n            if col not in output.columns:\n                output[col] = np.nan\n        return output\n\n    def normalize_positions(self, df: DataFrame) -> DataFrame:\n        out = self._rename(df, "positions")\n        out = self._ensure_columns(out, CANONICAL_POSITION_COLUMNS)\n        out = out.loc[:, ~out.columns.duplicated()].copy()\n\n        for date_col in ["entry_date", "maturity_expiry", "trade_date", "settlement_date"]:\n            out[date_col] = _coerce_datetime(out[date_col])\n\n        out["position_id"] = out["position_id"].astype("object")\n        out["product_type"] = out["product_type"].astype(str).str.strip().str.lower()\n        out["asset_class"] = out["asset_class"].astype(str).str.strip().str.lower()\n        out["sub_class"] = out["sub_class"].fillna("").astype(str)\n        out["ticker_identifier"] = out["ticker_identifier"].astype(str)\n        out["direction"] = out["direction"].fillna("long")\n\n        numeric_cols = ["quantity_notional", "entry_price", "current_price", "pnl_inception", "pnl_daily"]\n        for col in numeric_cols:\n            out[col] = pd.to_numeric(out[col], errors="coerce")\n\n        if out["position_id"].isna().any() or (out["position_id"].astype(str).str.strip() == "").any():\n            missing_mask = out["position_id"].isna() | (out["position_id"].astype(str).str.strip() == "")\n            generated = [f"AUTO_POS_{i:06d}" for i in range(missing_mask.sum())]\n            out.loc[missing_mask, "position_id"] = generated\n\n        return out\n\n    def normalize_market_data(self, df: DataFrame) -> DataFrame:\n        out = self._rename(df, "market_data")\n        out = self._ensure_columns(out, CANONICAL_MARKET_COLUMNS)\n        out = out.loc[:, ~out.columns.duplicated()].copy()\n        out["date"] = _coerce_datetime(out["date"])\n\n        for col in ["open", "high", "low", "close", "volume", "adjusted_close", "return_1d", "return_log"]:\n            values = out[col]\n            if isinstance(values, pd.DataFrame):\n                values = values.iloc[:, 0]\n            out[col] = pd.to_numeric(values, errors="coerce")\n\n        out["identifier"] = out["identifier"].astype(str)\n        out["asset_class"] = out["asset_class"].astype(str).str.lower()\n        out = out.sort_values(["identifier", "date"])\n        return out\n\n    def normalize_risk(self, df: DataFrame) -> DataFrame:\n        out = self._rename(df, "risk_sensitivities")\n        out = self._ensure_columns(out, CANONICAL_RISK_COLUMNS)\n        for col in [c for c in CANONICAL_RISK_COLUMNS if c != "position_id"]:\n            out[col] = pd.to_numeric(out[col], errors="coerce")\n        out["position_id"] = out["position_id"].astype(str)\n        return out\n\n    def normalize_pnl(self, df: DataFrame) -> DataFrame:\n        out = self._rename(df, "pnl_daily")\n        out = self._ensure_columns(out, CANONICAL_PNL_COLUMNS)\n        out["date"] = _coerce_datetime(out["date"])\n        for col in [c for c in CANONICAL_PNL_COLUMNS if c not in {"date", "position_id", "desk", "asset_class"}]:\n            out[col] = pd.to_numeric(out[col], errors="coerce")\n        out["position_id"] = out["position_id"].astype(str)\n        out["desk"] = out["desk"].astype(str)\n        out["asset_class"] = out["asset_class"].astype(str).str.lower()\n        return out\n\n    def normalize_any(self, name: str, df: DataFrame) -> DataFrame:\n        canonical_name = DATASET_ALIASES.get(name, name)\n        if canonical_name == "positions":\n            return self.normalize_positions(df)\n        if canonical_name == "market_data":\n            return self.normalize_market_data(df)\n        if canonical_name == "risk_sensitivities":\n            return self.normalize_risk(df)\n        if canonical_name == "pnl_daily":\n            return self.normalize_pnl(df)\n        return self._rename(df, canonical_name)\n\n\nclass DataQualityChecker:\n    """Data quality checks and schema compliance reporting."""\n\n    def __init__(self, stale_days: int = 5, outlier_sigma: float = 5.0) -> None:\n        self.stale_days = stale_days\n        self.outlier_sigma = outlier_sigma\n\n    def _schema_compliance(self, df: DataFrame, required: list[str]) -> dict[str, Any]:\n        missing = [c for c in required if c not in df.columns]\n        return {\n            "required_columns": len(required),\n            "present_columns": len(required) - len(missing),\n            "missing_columns": missing,\n            "is_compliant": len(missing) == 0,\n        }\n\n    def _missing_report(self, df: DataFrame) -> DataFrame:\n        miss = df.isna().sum().sort_values(ascending=False)\n        return pd.DataFrame(\n            {\n                "column": miss.index,\n                "missing_count": miss.values,\n                "missing_pct": (miss.values / max(len(df), 1)) * 100,\n            }\n        )\n\n    def _stale_price_report(self, market_data: DataFrame) -> DataFrame:\n        stale_rows: list[dict[str, Any]] = []\n        if market_data.empty:\n            return pd.DataFrame(columns=["identifier", "stale_days", "last_change_date"])\n\n        market_sorted = market_data.sort_values(["identifier", "date"])\n        for identifier, grp in market_sorted.groupby("identifier"):\n            tail = grp.tail(self.stale_days + 1)\n            if len(tail) < self.stale_days + 1:\n                continue\n            close_values = tail["close"].to_numpy()\n            if np.allclose(close_values, close_values[0], equal_nan=True):\n                stale_rows.append(\n                    {\n                        "identifier": identifier,\n                        "stale_days": self.stale_days,\n                        "last_change_date": tail.iloc[0]["date"],\n                    }\n                )\n\n        return pd.DataFrame(stale_rows)\n\n    def _return_outliers(self, market_data: DataFrame) -> DataFrame:\n        outliers: list[dict[str, Any]] = []\n        if market_data.empty or "return_1d" not in market_data.columns:\n            return pd.DataFrame(columns=["identifier", "date", "return_1d", "z_score"])\n\n        for identifier, grp in market_data.groupby("identifier"):\n            series = grp["return_1d"].dropna()\n            if len(series) < 20:\n                continue\n            mu = series.mean()\n            sigma = series.std(ddof=0)\n            if sigma <= 0:\n                continue\n            z = (series - mu) / sigma\n            flagged = grp.loc[series.index][np.abs(z) > self.outlier_sigma]\n            for idx in flagged.index:\n                outliers.append(\n                    {\n                        "identifier": identifier,\n                        "date": grp.loc[idx, "date"],\n                        "return_1d": float(grp.loc[idx, "return_1d"]),\n                        "z_score": float(z.loc[idx]),\n                    }\n                )\n\n        return pd.DataFrame(outliers)\n\n    def _position_consistency(self, positions: DataFrame, as_of_date: pd.Timestamp) -> DataFrame:\n        checks: list[dict[str, Any]] = []\n        if positions.empty:\n            return pd.DataFrame(columns=["position_id", "issue", "severity"])\n\n        option_like = positions[positions["product_type"].str.contains("option|cap|floor|swaption", na=False)]\n        if "maturity_expiry" in option_like.columns:\n            bad_expiry = option_like[pd.to_datetime(option_like["maturity_expiry"], errors="coerce") < as_of_date]\n            for _, row in bad_expiry.iterrows():\n                checks.append(\n                    {\n                        "position_id": row["position_id"],\n                        "issue": "Expiry/maturity is before as-of date",\n                        "severity": "high",\n                    }\n                )\n\n        huge_notionals = positions[positions["quantity_notional"].abs() > 1e11]\n        for _, row in huge_notionals.iterrows():\n            checks.append(\n                {\n                    "position_id": row["position_id"],\n                    "issue": "Notional magnitude is extreme (>1e11)",\n                    "severity": "medium",\n                }\n            )\n\n        missing_core = positions[\n            positions[["position_id", "product_type", "asset_class", "quantity_notional"]].isna().any(axis=1)\n        ]\n        for _, row in missing_core.iterrows():\n            checks.append(\n                {\n                    "position_id": row.get("position_id", "UNKNOWN"),\n                    "issue": "Missing core position fields",\n                    "severity": "high",\n                }\n            )\n\n        return pd.DataFrame(checks)\n\n    def run(\n        self,\n        positions: DataFrame,\n        market_data: DataFrame,\n        risk: DataFrame,\n        pnl: DataFrame,\n        as_of_date: str | pd.Timestamp,\n    ) -> dict[str, Any]:\n        as_of = pd.Timestamp(as_of_date)\n\n        missing_positions = self._missing_report(positions)\n        missing_market = self._missing_report(market_data)\n        missing_risk = self._missing_report(risk)\n        missing_pnl = self._missing_report(pnl)\n\n        stale_prices = self._stale_price_report(market_data)\n        outliers = self._return_outliers(market_data)\n        consistency = self._position_consistency(positions, as_of)\n\n        dup_positions = positions[positions["position_id"].duplicated(keep=False)]\n\n        schema = {\n            "positions": self._schema_compliance(positions, CANONICAL_POSITION_COLUMNS),\n            "market_data": self._schema_compliance(market_data, CANONICAL_MARKET_COLUMNS),\n            "risk_sensitivities": self._schema_compliance(risk, CANONICAL_RISK_COLUMNS),\n            "pnl_daily": self._schema_compliance(pnl, CANONICAL_PNL_COLUMNS),\n        }\n\n        has_warning = any(\n            [\n                len(stale_prices) > 0,\n                len(outliers) > 0,\n                len(consistency) > 0,\n                len(dup_positions) > 0,\n                not all(v["is_compliant"] for v in schema.values()),\n            ]\n        )\n\n        summary = (\n            f"Loaded {len(positions)} positions across "\n            f"{positions[\'asset_class\'].nunique() if not positions.empty else 0} asset classes from "\n            f"{positions[\'desk\'].nunique() if \'desk\' in positions.columns and not positions.empty else 0} desks. "\n            f"Data quality: {\'WARN\' if has_warning else \'PASS\'}"\n        )\n\n        return {\n            "summary": summary,\n            "status": "WARN" if has_warning else "PASS",\n            "missing_values": {\n                "positions": missing_positions,\n                "market_data": missing_market,\n                "risk_sensitivities": missing_risk,\n                "pnl_daily": missing_pnl,\n            },\n            "stale_prices": stale_prices,\n            "return_outliers": outliers,\n            "position_consistency": consistency,\n            "duplicate_positions": dup_positions,\n            "schema": schema,\n        }\n\n\nclass CacheManager:\n    """Cache and export utility for DataFrames and full portfolio snapshots."""\n\n    def __init__(self, cache_dir: str | Path = "output/jupyter-notebook/cache") -> None:\n        self.cache_dir = Path(cache_dir)\n        self.cache_dir.mkdir(parents=True, exist_ok=True)\n\n    def _base_path(self, dataset_name: str, as_of_date: str, suffix: str) -> Path:\n        safe_date = str(as_of_date).replace("/", "-")\n        return self.cache_dir / f"{dataset_name}_{safe_date}.{suffix}"\n\n    def save(self, dataset_name: str, df: DataFrame, as_of_date: str) -> Path:\n        parquet_path = self._base_path(dataset_name, as_of_date, "parquet")\n        pickle_path = self._base_path(dataset_name, as_of_date, "pkl")\n\n        try:\n            df.to_parquet(parquet_path, index=False)\n            return parquet_path\n        except Exception:\n            df.to_pickle(pickle_path)\n            return pickle_path\n\n    def load(self, dataset_name: str, as_of_date: str) -> DataFrame | None:\n        parquet_path = self._base_path(dataset_name, as_of_date, "parquet")\n        pickle_path = self._base_path(dataset_name, as_of_date, "pkl")\n\n        if parquet_path.exists():\n            try:\n                return pd.read_parquet(parquet_path)\n            except Exception:\n                pass\n        if pickle_path.exists():\n            try:\n                return pd.read_pickle(pickle_path)\n            except Exception:\n                pass\n        return None\n\n    def export_dataframe(\n        self,\n        df: DataFrame,\n        output_path: str | Path,\n        fmt: str | None = None,\n    ) -> Path:\n        path = Path(output_path)\n        path.parent.mkdir(parents=True, exist_ok=True)\n        format_guess = (fmt or path.suffix.lstrip(".") or "csv").lower()\n\n        if format_guess in {"csv"}:\n            df.to_csv(path, index=False)\n        elif format_guess in {"xlsx", "xls", "excel"}:\n            df.to_excel(path, index=False)\n        elif format_guess in {"parquet"}:\n            df.to_parquet(path, index=False)\n        elif format_guess in {"json"}:\n            df.to_json(path, orient="records", date_format="iso")\n        elif format_guess in {"feather"}:\n            df.to_feather(path)\n        else:\n            raise ValueError(f"Unsupported export format: {format_guess}")\n\n        return path\n\n    def snapshot(\n        self,\n        state: dict[str, DataFrame],\n        as_of_date: str,\n        label: str | None = None,\n    ) -> Path:\n        timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")\n        label = label or f"snapshot_{as_of_date}_{timestamp}"\n        snapshot_dir = self.cache_dir / "snapshots" / label\n        snapshot_dir.mkdir(parents=True, exist_ok=True)\n\n        manifest = {\n            "created_at": dt.datetime.utcnow().isoformat(),\n            "as_of_date": as_of_date,\n            "datasets": {},\n        }\n\n        for name, frame in state.items():\n            file_path = snapshot_dir / f"{name}.parquet"\n            try:\n                frame.to_parquet(file_path, index=False)\n                manifest["datasets"][name] = str(file_path)\n            except Exception:\n                file_path = snapshot_dir / f"{name}.pkl"\n                frame.to_pickle(file_path)\n                manifest["datasets"][name] = str(file_path)\n\n        with (snapshot_dir / "manifest.json").open("w", encoding="utf-8") as handle:\n            json.dump(manifest, handle, indent=2)\n\n        return snapshot_dir\n\n\nclass DataEngine:\n    """Unified source-agnostic data access layer for all notebook analytics."""\n\n    def __init__(self, config: dict[str, Any]) -> None:\n        self.config = config\n        self.data_mode = str(config.get("DATA_MODE", "synthetic")).lower()\n        self.base_currency = str(config.get("BASE_CURRENCY", "USD"))\n        self.as_of_date = str(config.get("AS_OF_DATE", dt.date.today()))\n        self.start_date = str(config.get("START_DATE", dt.date.today() - dt.timedelta(days=365)))\n        self.refresh_cache = bool(config.get("REFRESH_CACHE", False))\n\n        self.logger = get_logger("trading_desk_analytics.engine")\n        self.cache = CacheManager(config.get("CACHE_DIR", "output/jupyter-notebook/cache"))\n        self.normalizer = DataNormalizer(config.get("COLUMN_MAPPINGS", {}))\n        self.quality_checker = DataQualityChecker(\n            stale_days=int(config.get("STALE_DAYS", 5)),\n            outlier_sigma=float(config.get("OUTLIER_SIGMA", 5.0)),\n        )\n\n        self.synthetic_generator = SyntheticDataGenerator(\n            config=config.get("SYNTHETIC_CONFIG", {}),\n            base_currency=self.base_currency,\n            trading_days_per_year=int(config.get("TRADING_DAYS_PER_YEAR", 252)),\n        )\n\n        self.db_connector: DatabaseConnector | None = None\n        if self.data_mode in {"database", "hybrid"} and config.get("DB_CONNECTIONS"):\n            self.db_connector = DatabaseConnector(\n                connections=config.get("DB_CONNECTIONS", {}),\n                query_timeout=int(config.get("QUERY_TIMEOUT", 120)),\n                retry_config=RetryConfig(\n                    max_retries=int(config.get("DB_MAX_RETRIES", 3)),\n                    backoff_seconds=float(config.get("DB_BACKOFF_SECONDS", 1.0)),\n                ),\n                pool_size=int(config.get("DB_POOL_SIZE", 5)),\n                max_overflow=int(config.get("DB_MAX_OVERFLOW", 10)),\n            )\n\n        self._synthetic_pack: dict[str, DataFrame] | None = None\n        self._store: dict[str, DataFrame] = {}\n        self._source_by_dataset: dict[str, str] = {}\n\n    def switch_mode(self, new_mode: str) -> None:\n        self.data_mode = new_mode.lower().strip()\n        self.logger.info("Switched DATA_MODE to %s", self.data_mode)\n\n    def _canonical_name(self, name: str) -> str:\n        canonical = DATASET_ALIASES.get(name.lower().strip(), name.lower().strip())\n        return canonical\n\n    def _prepare_template_query(\n        self,\n        query_template: str,\n        params: dict[str, Any] | None,\n    ) -> tuple[str, dict[str, Any]]:\n        """Convert `{param}` placeholders into SQLAlchemy named binds.\n\n        Supports list expansion safely for `IN ({books})` style placeholders.\n        """\n        params = params or {}\n        query_out = query_template\n        bind_params: dict[str, Any] = {}\n\n        placeholders = sorted(set(re.findall(r"\\{([A-Za-z_][A-Za-z0-9_]*)\\}", query_template)))\n        for key in placeholders:\n            if key not in params:\n                raise KeyError(f"Missing query parameter \'{key}\'.")\n            value = params[key]\n            if isinstance(value, (list, tuple, set, np.ndarray)):\n                placeholder_names = []\n                for i, item in enumerate(list(value)):\n                    bind_key = f"{key}_{i}"\n                    bind_params[bind_key] = item\n                    placeholder_names.append(f":{bind_key}")\n                if not placeholder_names:\n                    placeholder_names = ["NULL"]\n                query_out = query_out.replace("{" + key + "}", ",".join(placeholder_names))\n            else:\n                bind_params[key] = value\n                query_out = query_out.replace("{" + key + "}", f":{key}")\n\n        return query_out, bind_params\n\n    def _ensure_synthetic_pack(self) -> dict[str, DataFrame]:\n        if self._synthetic_pack is None:\n            self._synthetic_pack = self.synthetic_generator.generate()\n        return self._synthetic_pack\n\n    def _load_database(self, dataset_name: str) -> DataFrame:\n        if self.db_connector is None:\n            raise RuntimeError("Database connector is not configured.")\n\n        sql_queries = self.config.get("SQL_QUERIES", {})\n        if dataset_name not in sql_queries:\n            raise KeyError(f"Dataset \'{dataset_name}\' not found in SQL_QUERIES config.")\n\n        cfg = sql_queries[dataset_name]\n        connection = cfg["connection"]\n        params = cfg.get("params", {})\n        stored_proc = cfg.get("stored_proc")\n\n        if stored_proc:\n            return self.db_connector.run_stored_procedure(connection, stored_proc, params=params)\n\n        query_template = cfg.get("query", "")\n        if not query_template:\n            raise ValueError(f"SQL query configuration missing `query` for \'{dataset_name}\'.")\n        query, bind_params = self._prepare_template_query(query_template, params)\n        return self.db_connector.run_query(\n            connection_name=connection,\n            query=query,\n            params=bind_params,\n            timeout_seconds=int(self.config.get("QUERY_TIMEOUT", 120)),\n        )\n\n    def _load_file(self, dataset_name: str) -> DataFrame:\n        inputs = self.config.get("FILE_INPUTS", {})\n        path = inputs.get(dataset_name)\n        if path is None:\n            raise KeyError(f"No file configured for dataset \'{dataset_name}\'.")\n\n        data_path = Path(path)\n        if not data_path.is_absolute():\n            data_path = (Path.cwd() / data_path).resolve()\n\n        if not data_path.exists():\n            raise FileNotFoundError(f"File input not found: {data_path}")\n\n        suffix = data_path.suffix.lower()\n        if suffix == ".csv":\n            return pd.read_csv(data_path)\n        if suffix in {".xlsx", ".xls"}:\n            return pd.read_excel(data_path)\n        if suffix == ".parquet":\n            return pd.read_parquet(data_path)\n        if suffix == ".json":\n            return pd.read_json(data_path)\n        if suffix == ".feather":\n            return pd.read_feather(data_path)\n        raise ValueError(f"Unsupported file extension \'{suffix}\' for {data_path}")\n\n    def _load_manual(self, dataset_name: str) -> DataFrame:\n        if dataset_name != "positions":\n            raise KeyError(f"Manual mode currently supports only \'positions\', got \'{dataset_name}\'.")\n        manual_positions = self.config.get("MANUAL_POSITIONS", [])\n        return pd.DataFrame(manual_positions)\n\n    def _map_identifier_to_yahoo(self, identifier: str) -> str | None:\n        """Map canonical identifiers to Yahoo symbols when possible."""\n        if not identifier or identifier == "nan":\n            return None\n\n        mapping = {\n            "SPX": "^GSPC",\n            "NDX": "^NDX",\n            "RUT": "^RUT",\n            "FTSE": "^FTSE",\n            "N225": "^N225",\n            "HSI": "^HSI",\n            "SX5E": "^STOXX50E",\n            "CL": "CL=F",\n            "NG": "NG=F",\n            "GC": "GC=F",\n            "SI": "SI=F",\n            "HG": "HG=F",\n            "ZC": "ZC=F",\n            "ZW": "ZW=F",\n            "ZS": "ZS=F",\n        }\n        if identifier in mapping:\n            return mapping[identifier]\n\n        # FX spot pairs and crosses.\n        if len(identifier) == 6 and identifier.isalpha():\n            return f"{identifier}=X"\n\n        # Skip most synthetic curve identifiers and generated contracts.\n        synthetic_markers = [\n            "_",\n            "FRA",\n            "IRS",\n            "OIS",\n            "BOND_FUTURE",\n            "SWAPTION",\n            "CAP",\n            "FLOOR",\n        ]\n        if any(marker in identifier for marker in synthetic_markers):\n            return None\n\n        return identifier\n\n    def _fetch_api_market_data(self, identifiers: Iterable[str]) -> DataFrame:\n        identifiers = [i for i in identifiers if isinstance(i, str) and i and i != "nan"]\n        start_date = self.start_date\n        end_date = self.as_of_date\n\n        frames: list[DataFrame] = []\n        use_yf = bool(self.config.get("API_CONFIG", {}).get("yfinance", {}).get("enabled", False))\n        if use_yf and yf is not None and identifiers:\n            for identifier in identifiers:\n                ticker = self._map_identifier_to_yahoo(identifier)\n                if ticker is None:\n                    continue\n                try:\n                    hist = yf.download(\n                        ticker,\n                        start=start_date,\n                        end=(pd.Timestamp(end_date) + pd.Timedelta(days=1)).strftime("%Y-%m-%d"),\n                        progress=False,\n                        auto_adjust=False,\n                        group_by="column",\n                        threads=False,\n                    )\n                    if hist.empty:\n                        continue\n\n                    if isinstance(hist.columns, pd.MultiIndex):\n                        hist.columns = hist.columns.get_level_values(0)\n\n                    hist = hist.reset_index()\n                    rename_map = {\n                        "Date": "date",\n                        "Open": "open",\n                        "High": "high",\n                        "Low": "low",\n                        "Close": "close",\n                        "Adj Close": "adjusted_close",\n                        "Volume": "volume",\n                    }\n                    hist = hist.rename(columns=rename_map)\n\n                    if "date" not in hist.columns and "Datetime" in hist.columns:\n                        hist = hist.rename(columns={"Datetime": "date"})\n                    if "adjusted_close" not in hist.columns and "close" in hist.columns:\n                        hist["adjusted_close"] = hist["close"]\n                    if "volume" not in hist.columns:\n                        hist["volume"] = np.nan\n                    if "open" not in hist.columns and "close" in hist.columns:\n                        hist["open"] = hist["close"]\n                    if "high" not in hist.columns and "close" in hist.columns:\n                        hist["high"] = hist["close"]\n                    if "low" not in hist.columns and "close" in hist.columns:\n                        hist["low"] = hist["close"]\n\n                    required = ["date", "open", "high", "low", "close", "adjusted_close", "volume"]\n                    if any(col not in hist.columns for col in required):\n                        continue\n\n                    hist["identifier"] = identifier\n                    hist["asset_class"] = "unknown"\n                    hist["return_1d"] = hist["close"].pct_change().fillna(0.0)\n                    hist["return_log"] = np.log(hist["close"]).diff().fillna(0.0)\n                    frames.append(hist[CANONICAL_MARKET_COLUMNS])\n                except Exception as exc:  # pragma: no cover - network/runtime\n                    self.logger.warning("API fetch failed for %s via yfinance: %s", ticker, exc)\n\n        if not frames:\n            raise RuntimeError("No API market data could be retrieved.")\n\n        return pd.concat(frames, ignore_index=True)\n\n    def _fetch_api_macro(self) -> DataFrame:\n        api_cfg = self.config.get("API_CONFIG", {})\n        fred_cfg = api_cfg.get("fred", {})\n        enabled = bool(fred_cfg.get("enabled", False))\n        if not enabled or Fred is None:\n            return pd.DataFrame(columns=["date", "series", "value"])\n\n        api_key = fred_cfg.get("api_key")\n        if not api_key:\n            self.logger.warning("FRED enabled but no API key found; skipping macro API pull.")\n            return pd.DataFrame(columns=["date", "series", "value"])\n\n        fred = Fred(api_key=api_key)\n        series_map = {\n            "fed_funds": "FEDFUNDS",\n            "cpi_yoy": "CPIAUCSL",\n            "unemployment": "UNRATE",\n            "us10y": "DGS10",\n            "dxy": "DTWEXBGS",\n        }\n        rows: list[DataFrame] = []\n        for logical_name, code in series_map.items():\n            try:\n                ser = fred.get_series(code, observation_start=self.start_date)\n                df = ser.rename("value").reset_index().rename(columns={"index": "date"})\n                df["series"] = logical_name\n                rows.append(df)\n            except Exception as exc:  # pragma: no cover - network/runtime\n                self.logger.warning("FRED fetch failed for %s (%s): %s", logical_name, code, exc)\n\n        if not rows:\n            return pd.DataFrame(columns=["date", "series", "value"])\n        return pd.concat(rows, ignore_index=True)\n\n    def _load_api(self, dataset_name: str) -> DataFrame:\n        if dataset_name == "market_data":\n            positions = self._store.get("positions")\n            if positions is None or positions.empty:\n                positions = self._ensure_synthetic_pack()["positions"]\n            identifiers = positions["ticker_identifier"].dropna().astype(str).unique().tolist()\n            return self._fetch_api_market_data(identifiers)\n        if dataset_name == "macro_data":\n            return self._fetch_api_macro()\n        raise KeyError(f"API mode does not support dataset \'{dataset_name}\'.")\n\n    def _load_hybrid(self, dataset_name: str) -> DataFrame:\n        """Hybrid mode orchestration:\n        - DB is primary for positions/risk/pnl/var\n        - API is primary for market data\n        - Manual positions overlay is applied on top\n        - Synthetic fills missing datasets or data gaps\n        """\n        synthetic = self._ensure_synthetic_pack()\n\n        if dataset_name == "positions":\n            positions_db = pd.DataFrame()\n            try:\n                positions_db = self._load_database("positions")\n                self._source_by_dataset[dataset_name] = "database"\n            except Exception as exc:\n                self.logger.warning("Hybrid DB positions load failed: %s", exc)\n\n            manual_positions = pd.DataFrame(self.config.get("MANUAL_POSITIONS", []))\n            frames = [f for f in [positions_db, manual_positions, synthetic["positions"]] if not f.empty]\n            combined = pd.concat(frames, ignore_index=True)\n            if "position_id" in combined.columns:\n                combined = combined.drop_duplicates(subset=["position_id"], keep="first")\n            return combined\n\n        if dataset_name == "market_data":\n            api_data = pd.DataFrame()\n            try:\n                api_data = self._load_api("market_data")\n                self._source_by_dataset[dataset_name] = "api"\n            except Exception as exc:\n                self.logger.warning("Hybrid API market data load failed: %s", exc)\n\n            db_data = pd.DataFrame()\n            try:\n                db_data = self._load_database("market_data")\n            except Exception:\n                pass\n\n            frames = [f for f in [api_data, db_data, synthetic["market_data"]] if not f.empty]\n            combined = pd.concat(frames, ignore_index=True)\n            combined = combined.sort_values(["identifier", "date"]).drop_duplicates(\n                subset=["identifier", "date"], keep="first"\n            )\n            return combined\n\n        if dataset_name in {"risk_sensitivities", "pnl_daily", "var_results", "yield_curves", "vol_surface", "credit_curves"}:\n            db_frame = pd.DataFrame()\n            try:\n                db_frame = self._load_database(dataset_name)\n                self._source_by_dataset[dataset_name] = "database"\n            except Exception as exc:\n                self.logger.warning("Hybrid DB load failed for %s: %s", dataset_name, exc)\n\n            if db_frame.empty:\n                return synthetic.get(dataset_name, pd.DataFrame())\n\n            if dataset_name in {"risk_sensitivities", "pnl_daily"} and "position_id" in db_frame.columns:\n                synthetic_frame = synthetic.get(dataset_name, pd.DataFrame())\n                if not synthetic_frame.empty and "position_id" in synthetic_frame.columns:\n                    missing_ids = set(synthetic_frame["position_id"]) - set(db_frame["position_id"])\n                    if missing_ids:\n                        fill = synthetic_frame[synthetic_frame["position_id"].isin(missing_ids)]\n                        db_frame = pd.concat([db_frame, fill], ignore_index=True)\n            return db_frame\n\n        return synthetic.get(dataset_name, pd.DataFrame())\n\n    def _load_dataset(self, dataset_name: str) -> DataFrame:\n        mode = self.data_mode\n\n        if mode == "synthetic":\n            self._source_by_dataset[dataset_name] = "synthetic"\n            return self._ensure_synthetic_pack().get(dataset_name, pd.DataFrame())\n\n        if mode == "database":\n            try:\n                frame = self._load_database(dataset_name)\n                self._source_by_dataset[dataset_name] = "database"\n                return frame\n            except Exception as exc:\n                self.logger.warning(\n                    "Database mode load failed for \'%s\': %s. Falling back to synthetic.",\n                    dataset_name,\n                    exc,\n                )\n                self._source_by_dataset[dataset_name] = "synthetic_fallback"\n                return self._ensure_synthetic_pack().get(dataset_name, pd.DataFrame())\n\n        if mode in {"csv", "file"}:\n            try:\n                frame = self._load_file(dataset_name)\n                self._source_by_dataset[dataset_name] = "file"\n                return frame\n            except Exception as exc:\n                self.logger.warning(\n                    "File mode load failed for \'%s\': %s. Falling back to synthetic.",\n                    dataset_name,\n                    exc,\n                )\n                self._source_by_dataset[dataset_name] = "synthetic_fallback"\n                return self._ensure_synthetic_pack().get(dataset_name, pd.DataFrame())\n\n        if mode == "manual":\n            try:\n                frame = self._load_manual(dataset_name)\n                self._source_by_dataset[dataset_name] = "manual"\n                return frame\n            except Exception as exc:\n                self.logger.warning(\n                    "Manual mode load failed for \'%s\': %s. Falling back to synthetic.",\n                    dataset_name,\n                    exc,\n                )\n                self._source_by_dataset[dataset_name] = "synthetic_fallback"\n                return self._ensure_synthetic_pack().get(dataset_name, pd.DataFrame())\n\n        if mode == "api":\n            try:\n                frame = self._load_api(dataset_name)\n                self._source_by_dataset[dataset_name] = "api"\n                return frame\n            except Exception as exc:\n                self.logger.warning(\n                    "API mode load failed for \'%s\': %s. Falling back to synthetic.",\n                    dataset_name,\n                    exc,\n                )\n                self._source_by_dataset[dataset_name] = "synthetic_fallback"\n                return self._ensure_synthetic_pack().get(dataset_name, pd.DataFrame())\n\n        if mode == "hybrid":\n            try:\n                frame = self._load_hybrid(dataset_name)\n                self._source_by_dataset.setdefault(dataset_name, "hybrid")\n                return frame\n            except Exception as exc:\n                self.logger.warning(\n                    "Hybrid mode load failed for \'%s\': %s. Falling back to synthetic.",\n                    dataset_name,\n                    exc,\n                )\n                self._source_by_dataset[dataset_name] = "synthetic_fallback"\n                return self._ensure_synthetic_pack().get(dataset_name, pd.DataFrame())\n\n        self.logger.warning("Unknown DATA_MODE \'%s\'. Falling back to synthetic.", mode)\n        self._source_by_dataset[dataset_name] = "synthetic_fallback"\n        return self._ensure_synthetic_pack().get(dataset_name, pd.DataFrame())\n\n    def get(self, name: str, refresh: bool | None = None) -> DataFrame:\n        """Get a normalized dataset by logical name."""\n        dataset_name = self._canonical_name(name)\n        do_refresh = self.refresh_cache if refresh is None else bool(refresh)\n\n        if dataset_name in self._store and not do_refresh:\n            return self._store[dataset_name]\n\n        if not do_refresh:\n            cached = self.cache.load(dataset_name, self.as_of_date)\n            if cached is not None:\n                normalized = self.normalizer.normalize_any(dataset_name, cached)\n                self._store[dataset_name] = normalized\n                self._source_by_dataset[dataset_name] = "cache"\n                return normalized\n\n        loaded = self._load_dataset(dataset_name)\n        normalized = self.normalizer.normalize_any(dataset_name, loaded)\n        self._store[dataset_name] = normalized\n\n        try:\n            self.cache.save(dataset_name, normalized, self.as_of_date)\n        except Exception as exc:\n            self.logger.warning("Cache save failed for %s: %s", dataset_name, exc)\n\n        return normalized\n\n    def load_all(self, refresh: bool = False) -> dict[str, DataFrame]:\n        datasets = [\n            "positions",\n            "market_data",\n            "risk_sensitivities",\n            "pnl_daily",\n            "var_results",\n            "yield_curves",\n            "credit_curves",\n            "vol_surface",\n            "factor_returns",\n        ]\n        output: dict[str, DataFrame] = {}\n        for name in datasets:\n            output[name] = self.get(name, refresh=refresh)\n        return output\n\n    def run_quality_checks(self) -> dict[str, Any]:\n        positions = self.get("positions")\n        market = self.get("market_data")\n        risk = self.get("risk_sensitivities")\n        pnl = self.get("pnl_daily")\n        return self.quality_checker.run(\n            positions=positions,\n            market_data=market,\n            risk=risk,\n            pnl=pnl,\n            as_of_date=self.as_of_date,\n        )\n\n    def source_of(self, dataset_name: str) -> str:\n        return self._source_by_dataset.get(self._canonical_name(dataset_name), "unknown")\n\n    def run_sql(\n        self,\n        connection_name: str,\n        query: str,\n        params: dict[str, Any] | None = None,\n    ) -> DataFrame:\n        if self.db_connector is None:\n            raise RuntimeError("Database connector is not configured in current mode/config.")\n        return self.db_connector.run_query(connection_name=connection_name, query=query, params=params)\n\n    def export(\n        self,\n        dataset_name: str,\n        output_path: str | Path,\n        fmt: str | None = None,\n    ) -> Path:\n        frame = self.get(dataset_name)\n        return self.cache.export_dataframe(frame, output_path, fmt=fmt)\n\n    def snapshot(self, label: str | None = None) -> Path:\n        if not self._store:\n            self.load_all(refresh=False)\n        return self.cache.snapshot(self._store, as_of_date=self.as_of_date, label=label)\n\n    def close(self) -> None:\n        if self.db_connector is not None:\n            self.db_connector.close()\n\n\n__all__ = [\n    "CANONICAL_MARKET_COLUMNS",\n    "CANONICAL_PNL_COLUMNS",\n    "CANONICAL_POSITION_COLUMNS",\n    "CANONICAL_RISK_COLUMNS",\n    "CacheManager",\n    "DataEngine",\n    "DataNormalizer",\n    "DataQualityChecker",\n    "DatabaseConnector",\n    "RetryConfig",\n    "SyntheticDataGenerator",\n    "get_logger",\n]\n'
_EMBEDDED_ANALYTICS = '"""Analytics toolkit for the Trading Desk Analytics Platform notebooks.\n\nThe functions in this module are designed for notebook use:\n- deterministic and vectorized where practical\n- explicit inputs/outputs with pandas/NumPy data structures\n- graceful fallbacks when optional dependencies are unavailable\n"""\n\nfrom __future__ import annotations\n\nimport datetime as dt\nimport math\nfrom dataclasses import dataclass\nfrom typing import Any, Callable\n\nimport numpy as np\nimport pandas as pd\nfrom numpy.typing import NDArray\nfrom pandas import DataFrame, Series\nfrom scipy import optimize, stats\nfrom sklearn.covariance import LedoitWolf\n\ntry:  # pragma: no cover - optional\n    import statsmodels.api as sm\n    from statsmodels.tsa.api import VAR\n    from statsmodels.tsa.stattools import coint, grangercausalitytests\nexcept Exception:  # pragma: no cover\n    sm = None\n    VAR = None\n    coint = None\n    grangercausalitytests = None\n\ntry:  # pragma: no cover - optional\n    from arch import arch_model\nexcept Exception:  # pragma: no cover\n    arch_model = None\n\ntry:  # pragma: no cover - optional\n    from hmmlearn.hmm import GaussianHMM\nexcept Exception:  # pragma: no cover\n    GaussianHMM = None\n\ntry:  # pragma: no cover - optional\n    from scipy.cluster.hierarchy import linkage, leaves_list\nexcept Exception:  # pragma: no cover\n    linkage = None\n    leaves_list = None\n\n\n@dataclass\nclass BacktestResult:\n    equity_curve: Series\n    returns: Series\n    positions: Series\n    turnover: Series\n    metrics: dict[str, float]\n\n\ndef _to_series(x: Any, index: pd.Index | None = None) -> Series:\n    if isinstance(x, Series):\n        return x\n    return pd.Series(x, index=index)\n\n\ndef annualized_return(returns: Series, periods_per_year: int = 252) -> float:\n    returns = returns.dropna()\n    if returns.empty:\n        return 0.0\n    total = (1 + returns).prod()\n    years = len(returns) / periods_per_year\n    if years <= 0:\n        return 0.0\n    return float(total ** (1 / years) - 1)\n\n\ndef annualized_volatility(returns: Series, periods_per_year: int = 252) -> float:\n    returns = returns.dropna()\n    if returns.empty:\n        return 0.0\n    return float(returns.std(ddof=0) * np.sqrt(periods_per_year))\n\n\ndef downside_deviation(returns: Series, target: float = 0.0, periods_per_year: int = 252) -> float:\n    downside = np.minimum(returns - target, 0)\n    if len(downside) == 0:\n        return 0.0\n    return float(np.sqrt(np.mean(downside**2)) * np.sqrt(periods_per_year))\n\n\ndef sharpe_ratio(returns: Series, risk_free_rate: float = 0.0, periods_per_year: int = 252) -> float:\n    rf_per_period = risk_free_rate / periods_per_year\n    excess = returns.dropna() - rf_per_period\n    vol = annualized_volatility(excess, periods_per_year)\n    if vol == 0:\n        return 0.0\n    return float(annualized_return(excess, periods_per_year) / vol)\n\n\ndef sortino_ratio(returns: Series, risk_free_rate: float = 0.0, periods_per_year: int = 252) -> float:\n    rf_per_period = risk_free_rate / periods_per_year\n    excess = returns.dropna() - rf_per_period\n    dd = downside_deviation(excess, 0.0, periods_per_year)\n    if dd == 0:\n        return 0.0\n    return float(annualized_return(excess, periods_per_year) / dd)\n\n\ndef omega_ratio(returns: Series, threshold: float = 0.0) -> float:\n    r = returns.dropna()\n    if r.empty:\n        return 0.0\n    gains = (r - threshold).clip(lower=0).sum()\n    losses = (threshold - r).clip(lower=0).sum()\n    if losses == 0:\n        return float("inf")\n    return float(gains / losses)\n\n\ndef drawdown_series(returns: Series) -> Series:\n    wealth = (1 + returns.fillna(0)).cumprod()\n    peaks = wealth.cummax()\n    return wealth / peaks - 1\n\n\ndef max_drawdown(returns: Series) -> float:\n    dd = drawdown_series(returns)\n    return float(dd.min()) if not dd.empty else 0.0\n\n\ndef calmar_ratio(returns: Series, periods_per_year: int = 252) -> float:\n    mdd = abs(max_drawdown(returns))\n    if mdd == 0:\n        return 0.0\n    return float(annualized_return(returns, periods_per_year) / mdd)\n\n\ndef information_ratio(strategy_returns: Series, benchmark_returns: Series, periods_per_year: int = 252) -> float:\n    aligned = pd.concat([strategy_returns, benchmark_returns], axis=1).dropna()\n    if aligned.empty:\n        return 0.0\n    active = aligned.iloc[:, 0] - aligned.iloc[:, 1]\n    te = active.std(ddof=0) * np.sqrt(periods_per_year)\n    if te == 0:\n        return 0.0\n    return float((active.mean() * periods_per_year) / te)\n\n\ndef tracking_error(strategy_returns: Series, benchmark_returns: Series, periods_per_year: int = 252) -> float:\n    aligned = pd.concat([strategy_returns, benchmark_returns], axis=1).dropna()\n    if aligned.empty:\n        return 0.0\n    active = aligned.iloc[:, 0] - aligned.iloc[:, 1]\n    return float(active.std(ddof=0) * np.sqrt(periods_per_year))\n\n\ndef beta(strategy_returns: Series, market_returns: Series) -> float:\n    aligned = pd.concat([strategy_returns, market_returns], axis=1).dropna()\n    if len(aligned) < 3:\n        return 0.0\n    cov = np.cov(aligned.iloc[:, 0], aligned.iloc[:, 1], ddof=0)[0, 1]\n    var_mkt = np.var(aligned.iloc[:, 1], ddof=0)\n    if var_mkt == 0:\n        return 0.0\n    return float(cov / var_mkt)\n\n\ndef treynor_ratio(strategy_returns: Series, market_returns: Series, risk_free_rate: float = 0.0, periods_per_year: int = 252) -> float:\n    b = beta(strategy_returns, market_returns)\n    if b == 0:\n        return 0.0\n    rf_per_period = risk_free_rate / periods_per_year\n    excess = strategy_returns.dropna() - rf_per_period\n    return float((excess.mean() * periods_per_year) / b)\n\n\n# ---------------------------------------------------------------------------\n# Options and derivatives pricing\n# ---------------------------------------------------------------------------\n\n\ndef _d1_d2(spot: float, strike: float, ttm: float, rate: float, vol: float, carry: float = 0.0) -> tuple[float, float]:\n    if ttm <= 0 or vol <= 0 or spot <= 0 or strike <= 0:\n        return 0.0, 0.0\n    d1 = (math.log(spot / strike) + (carry + 0.5 * vol**2) * ttm) / (vol * math.sqrt(ttm))\n    d2 = d1 - vol * math.sqrt(ttm)\n    return d1, d2\n\n\ndef black_scholes_merton(\n    spot: float,\n    strike: float,\n    ttm: float,\n    rate: float,\n    vol: float,\n    option_type: str = "call",\n    dividend_yield: float = 0.0,\n) -> dict[str, float]:\n    """Black-Scholes-Merton price and broad Greek set."""\n    cp = 1 if option_type.lower() == "call" else -1\n    if ttm <= 0:\n        intrinsic = max(cp * (spot - strike), 0.0)\n        return {\n            "price": float(intrinsic),\n            "delta": float(cp * (1.0 if cp * (spot - strike) > 0 else 0.0)),\n            "gamma": 0.0,\n            "vega": 0.0,\n            "theta": 0.0,\n            "rho": 0.0,\n            "vanna": 0.0,\n            "volga": 0.0,\n            "charm": 0.0,\n            "speed": 0.0,\n            "color": 0.0,\n            "zomma": 0.0,\n            "ultima": 0.0,\n        }\n\n    d1, d2 = _d1_d2(spot, strike, ttm, rate, vol, carry=rate - dividend_yield)\n    n1 = stats.norm.cdf(cp * d1)\n    n2 = stats.norm.cdf(cp * d2)\n    pdf_d1 = stats.norm.pdf(d1)\n\n    disc_q = math.exp(-dividend_yield * ttm)\n    disc_r = math.exp(-rate * ttm)\n\n    price = cp * (spot * disc_q * n1 - strike * disc_r * n2)\n    delta = cp * disc_q * stats.norm.cdf(cp * d1)\n    gamma = disc_q * pdf_d1 / (spot * vol * math.sqrt(ttm))\n    vega = spot * disc_q * pdf_d1 * math.sqrt(ttm)\n\n    theta_1 = -(spot * disc_q * pdf_d1 * vol) / (2 * math.sqrt(ttm))\n    theta_2 = cp * dividend_yield * spot * disc_q * stats.norm.cdf(cp * d1)\n    theta_3 = -cp * rate * strike * disc_r * stats.norm.cdf(cp * d2)\n    theta = theta_1 + theta_2 + theta_3\n\n    rho = cp * strike * ttm * disc_r * stats.norm.cdf(cp * d2)\n\n    # Higher-order Greeks (continuous-time approximations)\n    vanna = disc_q * pdf_d1 * (math.sqrt(ttm) - d1 / vol)\n    volga = vega * d1 * d2 / max(vol, 1e-10)\n    charm = -disc_q * pdf_d1 * ((2 * (rate - dividend_yield) * ttm - d2 * vol * math.sqrt(ttm)) / (2 * ttm * vol * math.sqrt(ttm)))\n    speed = -gamma / spot * (d1 / (vol * math.sqrt(ttm)) + 1)\n    color = -disc_q * pdf_d1 / (2 * spot * ttm * vol * math.sqrt(ttm)) * (\n        2 * dividend_yield * ttm + 1 + ((2 * (rate - dividend_yield) * ttm - d2 * vol * math.sqrt(ttm)) / (vol * math.sqrt(ttm))) * d1\n    )\n    zomma = gamma * (d1 * d2 - 1) / max(vol, 1e-10)\n    ultima = -vega / (vol**2 + 1e-10) * (d1 * d2 * (1 - d1 * d2) + d1**2 + d2**2)\n\n    return {\n        "price": float(price),\n        "delta": float(delta),\n        "gamma": float(gamma),\n        "vega": float(vega),\n        "theta": float(theta),\n        "rho": float(rho),\n        "vanna": float(vanna),\n        "volga": float(volga),\n        "charm": float(charm),\n        "speed": float(speed),\n        "color": float(color),\n        "zomma": float(zomma),\n        "ultima": float(ultima),\n    }\n\n\ndef black_76(\n    futures_price: float,\n    strike: float,\n    ttm: float,\n    rate: float,\n    vol: float,\n    option_type: str = "call",\n) -> float:\n    cp = 1 if option_type.lower() == "call" else -1\n    if ttm <= 0 or vol <= 0:\n        return max(cp * (futures_price - strike), 0.0)\n    d1 = (math.log(futures_price / strike) + 0.5 * vol**2 * ttm) / (vol * math.sqrt(ttm))\n    d2 = d1 - vol * math.sqrt(ttm)\n    return float(math.exp(-rate * ttm) * cp * (futures_price * stats.norm.cdf(cp * d1) - strike * stats.norm.cdf(cp * d2)))\n\n\ndef bachelier(\n    forward: float,\n    strike: float,\n    ttm: float,\n    rate: float,\n    normal_vol: float,\n    option_type: str = "call",\n) -> float:\n    cp = 1 if option_type.lower() == "call" else -1\n    if ttm <= 0 or normal_vol <= 0:\n        return max(cp * (forward - strike), 0.0)\n    sigma_t = normal_vol * math.sqrt(ttm)\n    d = (forward - strike) / sigma_t\n    price = math.exp(-rate * ttm) * (cp * (forward - strike) * stats.norm.cdf(cp * d) + sigma_t * stats.norm.pdf(d))\n    return float(price)\n\n\ndef binomial_american_option(\n    spot: float,\n    strike: float,\n    ttm: float,\n    rate: float,\n    vol: float,\n    steps: int = 200,\n    option_type: str = "call",\n) -> float:\n    cp = 1 if option_type.lower() == "call" else -1\n    dt_step = ttm / steps\n    u = math.exp(vol * math.sqrt(dt_step))\n    d = 1 / u\n    p = (math.exp(rate * dt_step) - d) / (u - d)\n    discount = math.exp(-rate * dt_step)\n\n    prices = np.array([spot * (u ** (steps - i)) * (d**i) for i in range(steps + 1)])\n    values = np.maximum(cp * (prices - strike), 0.0)\n\n    for step in range(steps - 1, -1, -1):\n        prices = prices[:-1] / u\n        continuation = discount * (p * values[:-1] + (1 - p) * values[1:])\n        exercise = np.maximum(cp * (prices - strike), 0.0)\n        values = np.maximum(continuation, exercise)\n\n    return float(values[0])\n\n\ndef monte_carlo_option(\n    spot: float,\n    strike: float,\n    ttm: float,\n    rate: float,\n    vol: float,\n    paths: int = 50_000,\n    steps: int = 252,\n    option_type: str = "call",\n    exotic: str = "vanilla",\n    barrier: float | None = None,\n    seed: int = 42,\n) -> float:\n    """Monte Carlo pricer for vanilla and selected path-dependent exotics."""\n    cp = 1 if option_type.lower() == "call" else -1\n    rng = np.random.default_rng(seed)\n    dt_step = ttm / steps\n\n    shocks = rng.standard_normal((paths, steps))\n    drift = (rate - 0.5 * vol**2) * dt_step\n    diffusion = vol * math.sqrt(dt_step) * shocks\n    log_paths = np.log(spot) + np.cumsum(drift + diffusion, axis=1)\n    price_paths = np.exp(log_paths)\n\n    final = price_paths[:, -1]\n    if exotic == "vanilla":\n        payoff = np.maximum(cp * (final - strike), 0)\n    elif exotic == "asian_arithmetic":\n        avg = price_paths.mean(axis=1)\n        payoff = np.maximum(cp * (avg - strike), 0)\n    elif exotic == "asian_geometric":\n        avg = np.exp(np.mean(np.log(np.maximum(price_paths, 1e-8)), axis=1))\n        payoff = np.maximum(cp * (avg - strike), 0)\n    elif exotic == "digital":\n        payoff = (cp * (final - strike) > 0).astype(float)\n    elif exotic == "lookback":\n        payoff = np.maximum(cp * (price_paths.max(axis=1) - strike), 0)\n    elif exotic == "barrier_knock_out":\n        if barrier is None:\n            raise ValueError("Barrier level required for barrier options.")\n        knocked = (price_paths.max(axis=1) >= barrier) if barrier > spot else (price_paths.min(axis=1) <= barrier)\n        payoff = np.where(knocked, 0.0, np.maximum(cp * (final - strike), 0))\n    else:\n        raise ValueError(f"Unsupported exotic type: {exotic}")\n\n    return float(np.exp(-rate * ttm) * payoff.mean())\n\n\ndef implied_vol_newton(\n    option_price: float,\n    spot: float,\n    strike: float,\n    ttm: float,\n    rate: float,\n    option_type: str = "call",\n    dividend_yield: float = 0.0,\n    initial_guess: float = 0.2,\n    tol: float = 1e-8,\n    max_iter: int = 100,\n) -> float:\n    vol = max(initial_guess, 1e-4)\n    for _ in range(max_iter):\n        greeks = black_scholes_merton(\n            spot=spot,\n            strike=strike,\n            ttm=ttm,\n            rate=rate,\n            vol=vol,\n            option_type=option_type,\n            dividend_yield=dividend_yield,\n        )\n        diff = greeks["price"] - option_price\n        vega = greeks["vega"]\n        if abs(diff) < tol:\n            break\n        if abs(vega) < 1e-12:\n            break\n        vol = max(1e-6, vol - diff / vega)\n    return float(vol)\n\n\ndef implied_vol_bisection(\n    option_price: float,\n    spot: float,\n    strike: float,\n    ttm: float,\n    rate: float,\n    option_type: str = "call",\n    dividend_yield: float = 0.0,\n    lower: float = 1e-4,\n    upper: float = 5.0,\n    tol: float = 1e-8,\n    max_iter: int = 200,\n) -> float:\n    for _ in range(max_iter):\n        mid = 0.5 * (lower + upper)\n        price_mid = black_scholes_merton(\n            spot=spot,\n            strike=strike,\n            ttm=ttm,\n            rate=rate,\n            vol=mid,\n            option_type=option_type,\n            dividend_yield=dividend_yield,\n        )["price"]\n        if abs(price_mid - option_price) < tol:\n            return float(mid)\n        if price_mid > option_price:\n            upper = mid\n        else:\n            lower = mid\n    return float(mid)\n\n\ndef option_strategy_payoff(spot_grid: NDArray[np.float64], legs: list[dict[str, Any]]) -> NDArray[np.float64]:\n    """Compute payoff profile for multi-leg option strategy.\n\n    Each leg supports keys: type (call/put/underlying), strike, premium, qty.\n    """\n    payoff = np.zeros_like(spot_grid, dtype=float)\n    for leg in legs:\n        leg_type = leg.get("type", "call").lower()\n        strike = float(leg.get("strike", 0))\n        premium = float(leg.get("premium", 0))\n        qty = float(leg.get("qty", 1))\n\n        if leg_type == "call":\n            leg_payoff = np.maximum(spot_grid - strike, 0) - premium\n        elif leg_type == "put":\n            leg_payoff = np.maximum(strike - spot_grid, 0) - premium\n        elif leg_type == "underlying":\n            leg_payoff = spot_grid - strike - premium\n        else:\n            raise ValueError(f"Unsupported leg type: {leg_type}")\n\n        payoff += qty * leg_payoff\n    return payoff\n\n\n# ---------------------------------------------------------------------------\n# Volatility estimators and surface helpers\n# ---------------------------------------------------------------------------\n\n\ndef realized_vol_close_to_close(close: Series, window: int = 21, annualization: int = 252) -> Series:\n    returns = np.log(close).diff()\n    return returns.rolling(window).std(ddof=0) * np.sqrt(annualization)\n\n\ndef realized_vol_parkinson(high: Series, low: Series, window: int = 21, annualization: int = 252) -> Series:\n    rs = (np.log(high / low) ** 2) / (4 * np.log(2))\n    return np.sqrt(rs.rolling(window).mean() * annualization)\n\n\ndef realized_vol_garman_klass(open_px: Series, high: Series, low: Series, close: Series, window: int = 21, annualization: int = 252) -> Series:\n    term1 = 0.5 * (np.log(high / low) ** 2)\n    term2 = (2 * np.log(2) - 1) * (np.log(close / open_px) ** 2)\n    var = (term1 - term2).rolling(window).mean()\n    return np.sqrt(np.maximum(var, 0) * annualization)\n\n\ndef realized_vol_rogers_satchell(open_px: Series, high: Series, low: Series, close: Series, window: int = 21, annualization: int = 252) -> Series:\n    x = np.log(high / close) * np.log(high / open_px) + np.log(low / close) * np.log(low / open_px)\n    return np.sqrt(np.maximum(x.rolling(window).mean(), 0) * annualization)\n\n\ndef realized_vol_yang_zhang(open_px: Series, high: Series, low: Series, close: Series, window: int = 21, annualization: int = 252) -> Series:\n    log_oc = np.log(open_px / close.shift(1))\n    log_co = np.log(close / open_px)\n    rs = np.log(high / close) * np.log(high / open_px) + np.log(low / close) * np.log(low / open_px)\n    k = 0.34 / (1.34 + (window + 1) / (window - 1))\n    var = (log_oc.rolling(window).var(ddof=0) + k * log_co.rolling(window).var(ddof=0) + (1 - k) * rs.rolling(window).mean())\n    return np.sqrt(np.maximum(var, 0) * annualization)\n\n\ndef volatility_risk_premium(implied_vol: Series, realized_vol: Series) -> Series:\n    aligned = pd.concat([implied_vol, realized_vol], axis=1).dropna()\n    return aligned.iloc[:, 0] - aligned.iloc[:, 1]\n\n\n# ---------------------------------------------------------------------------\n# Rates and fixed-income analytics\n# ---------------------------------------------------------------------------\n\n\ndef bond_price_from_yield(face: float, coupon_rate: float, ytm: float, maturity_years: float, freq: int = 2) -> float:\n    periods = int(round(maturity_years * freq))\n    if periods <= 0:\n        return face\n    coupon = face * coupon_rate / freq\n    discount = 1 + ytm / freq\n    cashflows = np.array([coupon] * periods)\n    cashflows[-1] += face\n    times = np.arange(1, periods + 1)\n    pv = np.sum(cashflows / (discount**times))\n    return float(pv)\n\n\ndef yield_from_bond_price(price: float, face: float, coupon_rate: float, maturity_years: float, freq: int = 2) -> float:\n    def objective(y: float) -> float:\n        return bond_price_from_yield(face, coupon_rate, y, maturity_years, freq) - price\n\n    return float(optimize.brentq(objective, -0.10, 1.50))\n\n\ndef macaulay_duration(face: float, coupon_rate: float, ytm: float, maturity_years: float, freq: int = 2) -> float:\n    periods = int(round(maturity_years * freq))\n    coupon = face * coupon_rate / freq\n    discount = 1 + ytm / freq\n    times = np.arange(1, periods + 1)\n    cashflows = np.array([coupon] * periods)\n    cashflows[-1] += face\n    pv_cashflows = cashflows / (discount**times)\n    price = pv_cashflows.sum()\n    if price == 0:\n        return 0.0\n    duration_periods = np.sum(times * pv_cashflows) / price\n    return float(duration_periods / freq)\n\n\ndef modified_duration(face: float, coupon_rate: float, ytm: float, maturity_years: float, freq: int = 2) -> float:\n    mac = macaulay_duration(face, coupon_rate, ytm, maturity_years, freq)\n    return float(mac / (1 + ytm / freq))\n\n\ndef convexity(face: float, coupon_rate: float, ytm: float, maturity_years: float, freq: int = 2) -> float:\n    periods = int(round(maturity_years * freq))\n    coupon = face * coupon_rate / freq\n    discount = 1 + ytm / freq\n    times = np.arange(1, periods + 1)\n    cashflows = np.array([coupon] * periods)\n    cashflows[-1] += face\n    pv_cashflows = cashflows / (discount**times)\n    price = pv_cashflows.sum()\n    if price == 0:\n        return 0.0\n    conv = np.sum(pv_cashflows * times * (times + 1)) / (price * (freq**2) * (discount**2))\n    return float(conv)\n\n\ndef dv01(face: float, coupon_rate: float, ytm: float, maturity_years: float, freq: int = 2) -> float:\n    bump = 0.0001\n    p_up = bond_price_from_yield(face, coupon_rate, ytm + bump, maturity_years, freq)\n    p_dn = bond_price_from_yield(face, coupon_rate, ytm - bump, maturity_years, freq)\n    return float((p_dn - p_up) / 2)\n\n\ndef nelson_siegel(tenor_years: NDArray[np.float64], beta0: float, beta1: float, beta2: float, tau: float) -> NDArray[np.float64]:\n    t = np.maximum(tenor_years, 1e-6)\n    x = (1 - np.exp(-t / tau)) / (t / tau)\n    return beta0 + beta1 * x + beta2 * (x - np.exp(-t / tau))\n\n\ndef nelson_siegel_svensson(\n    tenor_years: NDArray[np.float64],\n    beta0: float,\n    beta1: float,\n    beta2: float,\n    beta3: float,\n    tau1: float,\n    tau2: float,\n) -> NDArray[np.float64]:\n    t = np.maximum(tenor_years, 1e-6)\n    x1 = (1 - np.exp(-t / tau1)) / (t / tau1)\n    x2 = x1 - np.exp(-t / tau1)\n    x3 = (1 - np.exp(-t / tau2)) / (t / tau2) - np.exp(-t / tau2)\n    return beta0 + beta1 * x1 + beta2 * x2 + beta3 * x3\n\n\ndef fit_nelson_siegel(tenors: Series, yields: Series) -> dict[str, float]:\n    x = tenors.astype(float).to_numpy()\n    y = yields.astype(float).to_numpy()\n\n    def objective(params: NDArray[np.float64]) -> float:\n        b0, b1, b2, tau = params\n        fitted = nelson_siegel(x, b0, b1, b2, tau)\n        return float(np.mean((y - fitted) ** 2))\n\n    result = optimize.minimize(\n        objective,\n        x0=np.array([0.03, -0.02, 0.02, 1.2]),\n        bounds=[(-0.05, 0.2), (-0.2, 0.2), (-0.2, 0.2), (0.1, 10.0)],\n    )\n    b0, b1, b2, tau = result.x\n    return {"beta0": float(b0), "beta1": float(b1), "beta2": float(b2), "tau": float(tau)}\n\n\ndef key_rate_duration(\n    tenors: NDArray[np.float64],\n    zero_rates: NDArray[np.float64],\n    cashflow_times: NDArray[np.float64],\n    cashflows: NDArray[np.float64],\n    bump: float = 1e-4,\n) -> DataFrame:\n    base_discount = np.exp(-np.interp(cashflow_times, tenors, zero_rates) * cashflow_times)\n    base_price = np.sum(cashflows * base_discount)\n    rows = []\n    for i, t in enumerate(tenors):\n        bumped = zero_rates.copy()\n        bumped[i] += bump\n        discount = np.exp(-np.interp(cashflow_times, tenors, bumped) * cashflow_times)\n        bumped_price = np.sum(cashflows * discount)\n        krd = (base_price - bumped_price) / (base_price * bump)\n        rows.append({"tenor": float(t), "key_rate_duration": float(krd)})\n    return pd.DataFrame(rows)\n\n\ndef tax_equivalent_yield(muni_yield: float, tax_rate: float) -> float:\n    if tax_rate >= 1:\n        return np.nan\n    return float(muni_yield / max(1 - tax_rate, 1e-10))\n\n\ndef breakeven_inflation(nominal_yield: float, real_yield: float) -> float:\n    return float(nominal_yield - real_yield)\n\n\ndef ctd_basis(cash_price: float, futures_price: float, conversion_factor: float, accrued_interest: float = 0.0) -> float:\n    invoice = futures_price * conversion_factor + accrued_interest\n    return float(cash_price - invoice)\n\n\n# ---------------------------------------------------------------------------\n# FX and commodities\n# ---------------------------------------------------------------------------\n\n\ndef fx_forward_rate(spot: float, domestic_rate: float, foreign_rate: float, ttm: float) -> float:\n    return float(spot * np.exp((domestic_rate - foreign_rate) * ttm))\n\n\ndef forward_points(spot: float, forward: float, pips_factor: float = 10_000) -> float:\n    return float((forward - spot) * pips_factor)\n\n\ndef cross_rate(pair_ab: float, pair_bc: float) -> float:\n    return float(pair_ab * pair_bc)\n\n\ndef garman_kohlhagen(\n    spot: float,\n    strike: float,\n    ttm: float,\n    domestic_rate: float,\n    foreign_rate: float,\n    vol: float,\n    option_type: str = "call",\n) -> dict[str, float]:\n    return black_scholes_merton(\n        spot=spot,\n        strike=strike,\n        ttm=ttm,\n        rate=domestic_rate,\n        vol=vol,\n        option_type=option_type,\n        dividend_yield=foreign_rate,\n    )\n\n\ndef carry_trade_pnl(spot_return: Series, rate_diff_series: Series) -> Series:\n    aligned = pd.concat([spot_return, rate_diff_series], axis=1).dropna()\n    return aligned.iloc[:, 0] + aligned.iloc[:, 1] / 252\n\n\ndef futures_cost_of_carry(spot: float, rate: float, storage_cost: float, convenience_yield: float, ttm: float) -> float:\n    return float(spot * np.exp((rate + storage_cost - convenience_yield) * ttm))\n\n\ndef roll_yield(front_contract_price: float, next_contract_price: float, days_to_roll: int = 30) -> float:\n    if front_contract_price <= 0:\n        return 0.0\n    return float((next_contract_price - front_contract_price) / front_contract_price * (365 / max(days_to_roll, 1)))\n\n\ndef term_structure_state(front: float, deferred: float) -> str:\n    if deferred > front:\n        return "contango"\n    if deferred < front:\n        return "backwardation"\n    return "flat"\n\n\n# ---------------------------------------------------------------------------\n# Portfolio construction and optimization\n# ---------------------------------------------------------------------------\n\n\ndef mean_variance_optimization(\n    exp_returns: Series,\n    cov_matrix: DataFrame,\n    target_return: float | None = None,\n    risk_free_rate: float = 0.0,\n    long_only: bool = True,\n    max_weight: float = 1.0,\n) -> Series:\n    assets = exp_returns.index.tolist()\n    mu = exp_returns.values\n    cov = cov_matrix.loc[assets, assets].values\n    n = len(assets)\n\n    def portfolio_vol(w: NDArray[np.float64]) -> float:\n        return float(np.sqrt(w @ cov @ w))\n\n    constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1}]\n    if target_return is not None:\n        constraints.append({"type": "eq", "fun": lambda w: w @ mu - target_return})\n\n    bounds = [(0.0, max_weight) if long_only else (-max_weight, max_weight) for _ in range(n)]\n    w0 = np.ones(n) / n\n\n    result = optimize.minimize(portfolio_vol, w0, method="SLSQP", bounds=bounds, constraints=constraints)\n    if not result.success:\n        raise RuntimeError(f"Optimization failed: {result.message}")\n    return pd.Series(result.x, index=assets)\n\n\ndef max_sharpe_portfolio(\n    exp_returns: Series,\n    cov_matrix: DataFrame,\n    risk_free_rate: float = 0.0,\n    long_only: bool = True,\n    max_weight: float = 1.0,\n) -> Series:\n    assets = exp_returns.index.tolist()\n    mu = exp_returns.values\n    cov = cov_matrix.loc[assets, assets].values\n    n = len(assets)\n\n    def neg_sharpe(w: NDArray[np.float64]) -> float:\n        ret = w @ mu\n        vol = np.sqrt(w @ cov @ w)\n        if vol == 0:\n            return 1e6\n        return -(ret - risk_free_rate) / vol\n\n    constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1}]\n    bounds = [(0.0, max_weight) if long_only else (-max_weight, max_weight) for _ in range(n)]\n    w0 = np.ones(n) / n\n\n    result = optimize.minimize(neg_sharpe, w0, method="SLSQP", bounds=bounds, constraints=constraints)\n    if not result.success:\n        raise RuntimeError(f"Optimization failed: {result.message}")\n    return pd.Series(result.x, index=assets)\n\n\ndef minimum_variance_portfolio(cov_matrix: DataFrame, long_only: bool = True, max_weight: float = 1.0) -> Series:\n    assets = cov_matrix.index.tolist()\n    cov = cov_matrix.values\n    n = len(assets)\n\n    def objective(w: NDArray[np.float64]) -> float:\n        return float(w @ cov @ w)\n\n    constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1}]\n    bounds = [(0.0, max_weight) if long_only else (-max_weight, max_weight) for _ in range(n)]\n    w0 = np.ones(n) / n\n\n    result = optimize.minimize(objective, w0, method="SLSQP", bounds=bounds, constraints=constraints)\n    if not result.success:\n        raise RuntimeError(f"Optimization failed: {result.message}")\n    return pd.Series(result.x, index=assets)\n\n\ndef risk_contributions(weights: Series, cov_matrix: DataFrame) -> Series:\n    w = weights.values\n    cov = cov_matrix.loc[weights.index, weights.index].values\n    port_vol = np.sqrt(w @ cov @ w)\n    if port_vol <= 0:\n        return pd.Series(0.0, index=weights.index)\n    mrc = cov @ w / port_vol\n    rc = w * mrc\n    return pd.Series(rc, index=weights.index)\n\n\ndef risk_parity_portfolio(cov_matrix: DataFrame, long_only: bool = True, max_weight: float = 1.0) -> Series:\n    assets = cov_matrix.index.tolist()\n    cov = cov_matrix.values\n    n = len(assets)\n    target = np.ones(n) / n\n\n    def objective(w: NDArray[np.float64]) -> float:\n        port_vol = np.sqrt(w @ cov @ w)\n        if port_vol <= 0:\n            return 1e6\n        rc = w * (cov @ w) / port_vol\n        rc_pct = rc / rc.sum()\n        return float(np.sum((rc_pct - target) ** 2))\n\n    constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1}]\n    bounds = [(0.0, max_weight) if long_only else (-max_weight, max_weight) for _ in range(n)]\n    w0 = np.ones(n) / n\n\n    result = optimize.minimize(objective, w0, method="SLSQP", bounds=bounds, constraints=constraints)\n    if not result.success:\n        raise RuntimeError(f"Risk parity optimization failed: {result.message}")\n    return pd.Series(result.x, index=assets)\n\n\ndef black_litterman(\n    cov_matrix: DataFrame,\n    market_weights: Series,\n    risk_aversion: float,\n    views_matrix_p: DataFrame,\n    views_q: Series,\n    tau: float = 0.05,\n    omega: DataFrame | None = None,\n) -> dict[str, Any]:\n    assets = cov_matrix.index.tolist()\n    sigma = cov_matrix.loc[assets, assets].values\n    w_mkt = market_weights.loc[assets].values.reshape(-1, 1)\n\n    pi = risk_aversion * sigma @ w_mkt\n    p = views_matrix_p.loc[:, assets].values\n    q = views_q.values.reshape(-1, 1)\n\n    if omega is None:\n        omega_mat = np.diag(np.diag(p @ (tau * sigma) @ p.T))\n    else:\n        omega_mat = omega.values\n\n    middle = np.linalg.inv(np.linalg.inv(tau * sigma) + p.T @ np.linalg.inv(omega_mat) @ p)\n    posterior_mean = middle @ (np.linalg.inv(tau * sigma) @ pi + p.T @ np.linalg.inv(omega_mat) @ q)\n    posterior_cov = sigma + middle\n\n    weights = np.linalg.inv(risk_aversion * posterior_cov) @ posterior_mean\n    weights = weights / weights.sum()\n\n    return {\n        "implied_returns": pd.Series(pi.flatten(), index=assets),\n        "posterior_returns": pd.Series(posterior_mean.flatten(), index=assets),\n        "posterior_covariance": pd.DataFrame(posterior_cov, index=assets, columns=assets),\n        "optimal_weights": pd.Series(weights.flatten(), index=assets),\n    }\n\n\ndef efficient_frontier(\n    exp_returns: Series,\n    cov_matrix: DataFrame,\n    points: int = 40,\n    long_only: bool = True,\n    max_weight: float = 1.0,\n) -> DataFrame:\n    min_ret = exp_returns.min()\n    max_ret = exp_returns.max()\n    targets = np.linspace(min_ret, max_ret, points)\n    rows = []\n\n    for target in targets:\n        try:\n            w = mean_variance_optimization(\n                exp_returns,\n                cov_matrix,\n                target_return=float(target),\n                long_only=long_only,\n                max_weight=max_weight,\n            )\n            ret = float(w @ exp_returns)\n            vol = float(np.sqrt(w.values @ cov_matrix.loc[w.index, w.index].values @ w.values))\n            rows.append({"target_return": target, "return": ret, "volatility": vol, "weights": w.to_dict()})\n        except Exception:\n            continue\n\n    return pd.DataFrame(rows)\n\n\n# ---------------------------------------------------------------------------\n# Risk metrics and stress testing\n# ---------------------------------------------------------------------------\n\n\ndef ewma_volatility(returns: Series, lam: float = 0.94, annualization: int = 252) -> Series:\n    r = returns.fillna(0).to_numpy()\n    var = np.zeros_like(r)\n    if len(r) == 0:\n        return pd.Series(dtype=float)\n    var[0] = np.var(r)\n    for i in range(1, len(r)):\n        var[i] = lam * var[i - 1] + (1 - lam) * r[i - 1] ** 2\n    return pd.Series(np.sqrt(var) * np.sqrt(annualization), index=returns.index)\n\n\ndef garch_11_volatility(returns: Series, annualization: int = 252) -> Series:\n    r = returns.dropna()\n    if r.empty:\n        return pd.Series(dtype=float)\n    if arch_model is None:\n        return ewma_volatility(r, annualization=annualization)\n    model = arch_model(r * 100, vol="GARCH", p=1, q=1, dist="normal")\n    fit = model.fit(disp="off")\n    cond_vol = fit.conditional_volatility / 100 * np.sqrt(annualization)\n    return cond_vol.reindex(returns.index)\n\n\ndef var_parametric(returns: Series, confidence: float = 0.99, horizon_days: int = 1) -> float:\n    mu = returns.mean()\n    sigma = returns.std(ddof=0)\n    z = stats.norm.ppf(1 - confidence)\n    var = -(mu * horizon_days + sigma * np.sqrt(horizon_days) * z)\n    return float(var)\n\n\ndef var_historical(returns: Series, confidence: float = 0.99) -> float:\n    return float(-np.percentile(returns.dropna(), (1 - confidence) * 100))\n\n\ndef var_monte_carlo(\n    returns: Series,\n    confidence: float = 0.99,\n    horizon_days: int = 1,\n    simulations: int = 200_000,\n    seed: int = 42,\n) -> float:\n    rng = np.random.default_rng(seed)\n    mu = returns.mean() * horizon_days\n    sigma = returns.std(ddof=0) * np.sqrt(horizon_days)\n    sims = rng.normal(mu, sigma, size=simulations)\n    return float(-np.percentile(sims, (1 - confidence) * 100))\n\n\ndef var_cornish_fisher(returns: Series, confidence: float = 0.99) -> float:\n    r = returns.dropna()\n    z = stats.norm.ppf(confidence)\n    s = stats.skew(r)\n    k = stats.kurtosis(r, fisher=True)\n    z_cf = z + (1 / 6) * (z**2 - 1) * s + (1 / 24) * (z**3 - 3 * z) * k - (1 / 36) * (2 * z**3 - 5 * z) * s**2\n    mu = r.mean()\n    sigma = r.std(ddof=0)\n    return float(-(mu - z_cf * sigma))\n\n\ndef cvar(returns: Series, confidence: float = 0.99) -> float:\n    r = returns.dropna()\n    threshold = np.percentile(r, (1 - confidence) * 100)\n    tail = r[r <= threshold]\n    if tail.empty:\n        return 0.0\n    return float(-tail.mean())\n\n\ndef stress_test_portfolio(\n    exposures: Series,\n    scenario_shocks: DataFrame,\n) -> DataFrame:\n    """Apply scenario shocks to factor exposures.\n\n    exposures index: factor names\n    scenario_shocks columns: factor names, index: scenario labels\n    """\n    common = [c for c in scenario_shocks.columns if c in exposures.index]\n    pnl = scenario_shocks[common].mul(exposures[common], axis=1).sum(axis=1)\n    return pd.DataFrame({"scenario": scenario_shocks.index, "stress_pnl": pnl.values})\n\n\ndef skew_kurtosis_jb(returns: Series) -> dict[str, float]:\n    r = returns.dropna()\n    if r.empty:\n        return {"skew": 0.0, "kurtosis": 0.0, "jb_stat": 0.0, "jb_pvalue": 1.0}\n    jb_stat, jb_pvalue = stats.jarque_bera(r)\n    return {\n        "skew": float(stats.skew(r)),\n        "kurtosis": float(stats.kurtosis(r, fisher=False)),\n        "jb_stat": float(jb_stat),\n        "jb_pvalue": float(jb_pvalue),\n    }\n\n\ndef sample_covariance(returns_df: DataFrame) -> DataFrame:\n    return returns_df.cov()\n\n\ndef ledoit_wolf_covariance(returns_df: DataFrame) -> DataFrame:\n    lw = LedoitWolf().fit(returns_df.dropna().values)\n    return pd.DataFrame(lw.covariance_, index=returns_df.columns, columns=returns_df.columns)\n\n\ndef exponentially_weighted_covariance(returns_df: DataFrame, lam: float = 0.94) -> DataFrame:\n    x = returns_df.dropna().values\n    if len(x) == 0:\n        return pd.DataFrame(index=returns_df.columns, columns=returns_df.columns, data=np.nan)\n    n = x.shape[1]\n    s = np.zeros((n, n))\n    for row in x:\n        row_vec = row.reshape(-1, 1)\n        s = lam * s + (1 - lam) * (row_vec @ row_vec.T)\n    return pd.DataFrame(s, index=returns_df.columns, columns=returns_df.columns)\n\n\ndef clustered_correlation(corr_matrix: DataFrame) -> tuple[DataFrame, list[str]]:\n    if linkage is None or leaves_list is None:\n        return corr_matrix, corr_matrix.index.tolist()\n    dist = np.sqrt(0.5 * (1 - corr_matrix.clip(-1, 1).values))\n    link = linkage(dist, method="average")\n    order_idx = leaves_list(link)\n    ordered = corr_matrix.iloc[order_idx, order_idx]\n    return ordered, ordered.index.tolist()\n\n\ndef rolling_beta(asset_returns: Series, benchmark_returns: Series, window: int = 63) -> Series:\n    aligned = pd.concat([asset_returns, benchmark_returns], axis=1).dropna()\n    if aligned.empty:\n        return pd.Series(dtype=float)\n    cov = aligned.iloc[:, 0].rolling(window).cov(aligned.iloc[:, 1])\n    var = aligned.iloc[:, 1].rolling(window).var()\n    return cov / var.replace(0, np.nan)\n\n\ndef regime_detection(returns: Series, n_states: int = 3, seed: int = 42) -> Series:\n    r = returns.dropna()\n    if len(r) < 40:\n        return pd.Series(index=returns.index, dtype=float)\n\n    values = r.values.reshape(-1, 1)\n    if GaussianHMM is not None:\n        model = GaussianHMM(n_components=n_states, covariance_type="diag", random_state=seed, n_iter=500)\n        states = model.fit_predict(values)\n    else:\n        # fallback: volatility-quantile regimes\n        vol = r.rolling(20).std().fillna(r.std())\n        bins = np.quantile(vol, np.linspace(0, 1, n_states + 1))\n        states = np.digitize(vol, bins[1:-1], right=True)\n\n    out = pd.Series(states, index=r.index)\n    return out.reindex(returns.index)\n\n\ndef dcc_like_dynamic_correlation(asset_a: Series, asset_b: Series, lam: float = 0.97) -> Series:\n    aligned = pd.concat([asset_a, asset_b], axis=1).dropna()\n    if aligned.empty:\n        return pd.Series(dtype=float)\n    z = (aligned - aligned.mean()) / aligned.std(ddof=0)\n    q11 = q22 = q12 = 0.0\n    corr = []\n    for x, y in z.itertuples(index=False):\n        q11 = lam * q11 + (1 - lam) * x * x\n        q22 = lam * q22 + (1 - lam) * y * y\n        q12 = lam * q12 + (1 - lam) * x * y\n        denom = math.sqrt(max(q11 * q22, 1e-12))\n        corr.append(q12 / denom)\n    return pd.Series(corr, index=aligned.index)\n\n\n# ---------------------------------------------------------------------------\n# Attribution and factor modeling\n# ---------------------------------------------------------------------------\n\n\ndef capm_regression(asset_returns: Series, market_returns: Series, risk_free_rate: float = 0.0, periods_per_year: int = 252) -> dict[str, float]:\n    aligned = pd.concat([asset_returns, market_returns], axis=1).dropna()\n    if aligned.empty:\n        return {"alpha": 0.0, "beta": 0.0, "r_squared": 0.0}\n\n    y = aligned.iloc[:, 0] - risk_free_rate / periods_per_year\n    x = aligned.iloc[:, 1] - risk_free_rate / periods_per_year\n    if sm is None:\n        b = beta(y, x)\n        alpha = (y.mean() - b * x.mean()) * periods_per_year\n        return {"alpha": float(alpha), "beta": float(b), "r_squared": np.nan}\n\n    X = sm.add_constant(x)\n    model = sm.OLS(y, X, missing="drop").fit()\n    return {\n        "alpha": float(model.params.get("const", 0.0) * periods_per_year),\n        "beta": float(model.params.iloc[1]),\n        "r_squared": float(model.rsquared),\n        "p_value_beta": float(model.pvalues.iloc[1]),\n    }\n\n\ndef multi_factor_regression(asset_returns: Series, factor_returns: DataFrame) -> dict[str, Any]:\n    aligned = pd.concat([asset_returns, factor_returns], axis=1).dropna()\n    if aligned.empty:\n        return {"alpha": 0.0, "betas": {}, "r_squared": 0.0}\n    y = aligned.iloc[:, 0]\n    x = aligned.iloc[:, 1:]\n\n    if sm is None:\n        betas = np.linalg.lstsq(x.values, y.values, rcond=None)[0]\n        fitted = x.values @ betas\n        resid = y.values - fitted\n        r2 = 1 - np.var(resid) / np.var(y.values)\n        return {\n            "alpha": float(0.0),\n            "betas": dict(zip(x.columns, betas.astype(float))),\n            "r_squared": float(r2),\n        }\n\n    X = sm.add_constant(x)\n    model = sm.OLS(y, X, missing="drop").fit()\n    betas = model.params.drop("const", errors="ignore")\n    return {\n        "alpha": float(model.params.get("const", 0.0)),\n        "betas": betas.to_dict(),\n        "r_squared": float(model.rsquared),\n        "p_values": model.pvalues.to_dict(),\n    }\n\n\ndef brinson_fachler(\n    portfolio_weights: Series,\n    benchmark_weights: Series,\n    portfolio_returns: Series,\n    benchmark_returns: Series,\n) -> dict[str, float]:\n    """Single-period Brinson-Fachler attribution (allocation, selection, interaction)."""\n    idx = sorted(\n        set(portfolio_weights.index)\n        & set(benchmark_weights.index)\n        & set(portfolio_returns.index)\n        & set(benchmark_returns.index)\n    )\n    wp = portfolio_weights.loc[idx]\n    wb = benchmark_weights.loc[idx]\n    rp = portfolio_returns.loc[idx]\n    rb = benchmark_returns.loc[idx]\n\n    benchmark_total = float((wb * rb).sum())\n    allocation = float(((wp - wb) * (rb - benchmark_total)).sum())\n    selection = float((wb * (rp - rb)).sum())\n    interaction = float(((wp - wb) * (rp - rb)).sum())\n    active = allocation + selection + interaction\n\n    return {\n        "allocation": allocation,\n        "selection": selection,\n        "interaction": interaction,\n        "active_return": active,\n    }\n\n\ndef greeks_pnl_attribution(greeks: DataFrame, market_moves: DataFrame, option_meta: DataFrame | None = None) -> DataFrame:\n    """Approximate daily Greeks-based PnL decomposition."""\n    merged = greeks.merge(market_moves, on="position_id", how="inner")\n    out = pd.DataFrame()\n    out["position_id"] = merged["position_id"]\n    out["delta_pnl"] = merged.get("delta", 0) * merged.get("dS", 0)\n    out["gamma_pnl"] = 0.5 * merged.get("gamma", 0) * merged.get("dS", 0) ** 2\n    out["vega_pnl"] = merged.get("vega", 0) * merged.get("dVol", 0)\n    out["theta_pnl"] = merged.get("theta", 0) * merged.get("dT", 1 / 252)\n    out["rho_pnl"] = merged.get("rho", 0) * merged.get("dR", 0)\n    out["predicted_pnl"] = out[["delta_pnl", "gamma_pnl", "vega_pnl", "theta_pnl", "rho_pnl"]].sum(axis=1)\n    if "actual_pnl" in merged.columns:\n        out["residual_pnl"] = merged["actual_pnl"] - out["predicted_pnl"]\n    return out\n\n\ndef factor_pnl_attribution(exposures: DataFrame, factor_returns: DataFrame) -> DataFrame:\n    """Position-level factor PnL attribution for each date."""\n    fac_cols = [c for c in factor_returns.columns if c != "date"]\n    rows: list[dict[str, Any]] = []\n\n    for _, exp_row in exposures.iterrows():\n        pid = exp_row["position_id"]\n        for _, ret_row in factor_returns.iterrows():\n            pnl_components = {}\n            total = 0.0\n            for fac in fac_cols:\n                beta_col = f"beta_{fac}"\n                beta_val = float(exp_row.get(beta_col, 0.0))\n                comp = beta_val * float(ret_row.get(fac, 0.0))\n                pnl_components[f"{fac}_pnl"] = comp\n                total += comp\n            rec = {"date": ret_row["date"], "position_id": pid, "factor_pnl": total}\n            rec.update(pnl_components)\n            rows.append(rec)\n\n    return pd.DataFrame(rows)\n\n\n# ---------------------------------------------------------------------------\n# PnL analytics\n# ---------------------------------------------------------------------------\n\n\ndef pnl_summary(pnl_df: DataFrame) -> dict[str, float]:\n    r = pnl_df["total_pnl"].dropna()\n    if r.empty:\n        return {\n            "total_pnl": 0.0,\n            "average_daily_pnl": 0.0,\n            "pnl_volatility": 0.0,\n            "pnl_sharpe": 0.0,\n            "win_rate": 0.0,\n            "profit_factor": 0.0,\n            "expectancy": 0.0,\n        }\n\n    wins = r[r > 0]\n    losses = r[r < 0]\n    win_rate = len(wins) / len(r)\n    avg_win = wins.mean() if len(wins) else 0.0\n    avg_loss = losses.mean() if len(losses) else 0.0\n    profit_factor = abs(wins.sum() / losses.sum()) if losses.sum() != 0 else np.inf\n    expectancy = win_rate * avg_win + (1 - win_rate) * avg_loss\n    sharpe = r.mean() / r.std(ddof=0) * np.sqrt(252) if r.std(ddof=0) > 0 else 0.0\n\n    return {\n        "total_pnl": float(r.sum()),\n        "average_daily_pnl": float(r.mean()),\n        "pnl_volatility": float(r.std(ddof=0)),\n        "pnl_sharpe": float(sharpe),\n        "win_rate": float(win_rate),\n        "average_win": float(avg_win),\n        "average_loss": float(avg_loss),\n        "profit_factor": float(profit_factor),\n        "expectancy": float(expectancy),\n    }\n\n\ndef pnl_heatmap_table(pnl_df: DataFrame, freq: str = "M") -> DataFrame:\n    df = pnl_df.copy()\n    df["date"] = pd.to_datetime(df["date"])\n    grouped = df.groupby(pd.Grouper(key="date", freq=freq))["total_pnl"].sum()\n    out = grouped.to_frame("pnl")\n    out["year"] = out.index.year\n    out["period"] = out.index.month if freq.upper().startswith("M") else out.index.quarter\n    return out.pivot_table(index="year", columns="period", values="pnl", aggfunc="sum")\n\n\ndef pnl_explain(model_pnl: Series, actual_pnl: Series) -> DataFrame:\n    aligned = pd.concat([model_pnl, actual_pnl], axis=1).dropna()\n    aligned.columns = ["model_pnl", "actual_pnl"]\n    aligned["unexplained_pnl"] = aligned["actual_pnl"] - aligned["model_pnl"]\n    aligned["explained_ratio"] = np.where(\n        aligned["actual_pnl"].abs() > 1e-10,\n        aligned["model_pnl"] / aligned["actual_pnl"],\n        np.nan,\n    )\n    return aligned\n\n\ndef desk_level_aggregation(pnl_df: DataFrame) -> DataFrame:\n    return (\n        pnl_df.groupby(["date", "desk", "asset_class"], as_index=False)[\n            ["total_pnl", "delta_pnl", "gamma_pnl", "theta_pnl", "vega_pnl", "carry_pnl", "roll_pnl", "fx_pnl", "residual_pnl"]\n        ]\n        .sum()\n        .sort_values(["date", "desk"])\n    )\n\n\ndef pnl_distribution_stats(pnl_series: Series) -> dict[str, float]:\n    r = pnl_series.dropna()\n    if r.empty:\n        return {"mean": 0.0, "std": 0.0, "skew": 0.0, "kurtosis": 0.0, "jb_pvalue": 1.0}\n    jb = stats.jarque_bera(r)\n    return {\n        "mean": float(r.mean()),\n        "std": float(r.std(ddof=0)),\n        "skew": float(stats.skew(r)),\n        "kurtosis": float(stats.kurtosis(r, fisher=False)),\n        "jb_stat": float(jb.statistic),\n        "jb_pvalue": float(jb.pvalue),\n    }\n\n\n# ---------------------------------------------------------------------------\n# Cross-asset / macro analytics\n# ---------------------------------------------------------------------------\n\n\ndef rolling_correlation_matrix(returns_df: DataFrame, window: int = 63) -> dict[pd.Timestamp, DataFrame]:\n    out: dict[pd.Timestamp, DataFrame] = {}\n    for i in range(window, len(returns_df) + 1):\n        end_date = returns_df.index[i - 1]\n        out[end_date] = returns_df.iloc[i - window : i].corr()\n    return out\n\n\ndef regime_conditional_correlation(returns_df: DataFrame, regime_series: Series) -> dict[Any, DataFrame]:\n    aligned = returns_df.join(regime_series.rename("regime"), how="inner")\n    out: dict[Any, DataFrame] = {}\n    for regime, grp in aligned.groupby("regime"):\n        out[regime] = grp.drop(columns=["regime"]).corr()\n    return out\n\n\ndef granger_causality_table(data: DataFrame, max_lag: int = 3) -> DataFrame:\n    if grangercausalitytests is None:\n        return pd.DataFrame(columns=["cause", "effect", "min_pvalue"])\n    cols = data.columns.tolist()\n    rows = []\n    for cause in cols:\n        for effect in cols:\n            if cause == effect:\n                continue\n            subset = data[[effect, cause]].dropna()\n            if len(subset) < max_lag + 20:\n                continue\n            try:\n                gc = grangercausalitytests(subset, maxlag=max_lag, verbose=False)\n                pvals = [gc[lag][0]["ssr_chi2test"][1] for lag in range(1, max_lag + 1)]\n                rows.append({"cause": cause, "effect": effect, "min_pvalue": float(min(pvals))})\n            except Exception:\n                continue\n    return pd.DataFrame(rows)\n\n\ndef impulse_response(data: DataFrame, steps: int = 10) -> dict[str, DataFrame]:\n    if VAR is None:\n        return {}\n    clean = data.dropna()\n    if len(clean) < 40:\n        return {}\n    model = VAR(clean)\n    fit = model.fit(maxlags=3)\n    irf = fit.irf(steps)\n\n    out: dict[str, DataFrame] = {}\n    for i, impulse_var in enumerate(clean.columns):\n        for j, response_var in enumerate(clean.columns):\n            name = f"impulse_{impulse_var}_response_{response_var}"\n            out[name] = pd.DataFrame(\n                {\n                    "horizon": np.arange(steps + 1),\n                    "response": irf.irfs[:, j, i],\n                }\n            )\n    return out\n\n\ndef risk_on_off_composite(signals: DataFrame, weights: Series | None = None) -> Series:\n    """Build composite risk-on/risk-off score from normalized input signals."""\n    z = (signals - signals.mean()) / signals.std(ddof=0)\n    z = z.replace([np.inf, -np.inf], np.nan).fillna(0.0)\n    if weights is None:\n        weights = pd.Series(1 / z.shape[1], index=z.columns)\n    weights = weights.reindex(z.columns).fillna(0.0)\n    composite = z.mul(weights, axis=1).sum(axis=1)\n    return composite\n\n\ndef dollar_smile_buckets(usd_index: Series, global_growth: Series, risk_aversion: Series) -> DataFrame:\n    df = pd.concat([usd_index, global_growth, risk_aversion], axis=1).dropna()\n    df.columns = ["usd", "growth", "risk"]\n    growth_z = (df["growth"] - df["growth"].mean()) / df["growth"].std(ddof=0)\n    risk_z = (df["risk"] - df["risk"].mean()) / df["risk"].std(ddof=0)\n\n    regimes = np.where((growth_z > 0.5) & (risk_z < 0.5), "middle", np.where((growth_z < -0.5) | (risk_z > 0.5), "stress", "transition"))\n    out = df.copy()\n    out["regime"] = regimes\n    return out.groupby("regime")["usd"].agg(["mean", "std", "count"]).reset_index()\n\n\ndef dispersion_metrics(index_iv: Series, component_ivs: DataFrame, weights: Series | None = None) -> DataFrame:\n    if weights is None:\n        weights = pd.Series(1 / component_ivs.shape[1], index=component_ivs.columns)\n    w = weights.reindex(component_ivs.columns).fillna(0.0)\n    weighted_var = (component_ivs**2).mul(w, axis=1).sum(axis=1)\n    implied_corr = ((index_iv**2) - weighted_var) / (2 * weighted_var.abs().replace(0, np.nan))\n    out = pd.DataFrame({"index_iv": index_iv, "weighted_component_var": weighted_var, "implied_correlation": implied_corr})\n    return out\n\n\n# ---------------------------------------------------------------------------\n# Strategy signals and backtesting\n# ---------------------------------------------------------------------------\n\n\ndef backtest_signal(\n    prices: Series,\n    signal: Series,\n    transaction_cost_bps: float = 1.0,\n    slippage_bps: float = 1.0,\n    financing_rate: float = 0.0,\n    periods_per_year: int = 252,\n) -> BacktestResult:\n    prices = prices.dropna()\n    signal = signal.reindex(prices.index).fillna(0.0)\n    returns = prices.pct_change().fillna(0.0)\n\n    positions = signal.shift(1).fillna(0.0)\n    turnover = positions.diff().abs().fillna(0.0)\n    costs = turnover * (transaction_cost_bps + slippage_bps) / 10_000\n    financing = abs(positions) * financing_rate / periods_per_year\n\n    strategy_returns = positions * returns - costs - financing\n    equity = (1 + strategy_returns).cumprod()\n\n    metrics = {\n        "cagr": annualized_return(strategy_returns, periods_per_year),\n        "volatility": annualized_volatility(strategy_returns, periods_per_year),\n        "sharpe": sharpe_ratio(strategy_returns, 0.0, periods_per_year),\n        "sortino": sortino_ratio(strategy_returns, 0.0, periods_per_year),\n        "max_drawdown": max_drawdown(strategy_returns),\n        "calmar": calmar_ratio(strategy_returns, periods_per_year),\n        "hit_rate": float((strategy_returns > 0).mean()),\n        "turnover": float(turnover.mean() * periods_per_year),\n    }\n\n    return BacktestResult(equity_curve=equity, returns=strategy_returns, positions=positions, turnover=turnover, metrics=metrics)\n\n\ndef signal_moving_average_crossover(prices: Series, fast: int = 20, slow: int = 100) -> Series:\n    fast_ma = prices.rolling(fast).mean()\n    slow_ma = prices.rolling(slow).mean()\n    return np.sign(fast_ma - slow_ma).fillna(0.0)\n\n\ndef signal_breakout(prices: Series, lookback: int = 55) -> Series:\n    high = prices.rolling(lookback).max().shift(1)\n    low = prices.rolling(lookback).min().shift(1)\n    sig = pd.Series(0.0, index=prices.index)\n    sig[prices > high] = 1.0\n    sig[prices < low] = -1.0\n    return sig.ffill().fillna(0.0)\n\n\ndef signal_bollinger_mean_reversion(prices: Series, window: int = 20, num_std: float = 2.0) -> Series:\n    ma = prices.rolling(window).mean()\n    sd = prices.rolling(window).std(ddof=0)\n    z = (prices - ma) / sd\n    sig = pd.Series(0.0, index=prices.index)\n    sig[z > num_std] = -1.0\n    sig[z < -num_std] = 1.0\n    sig[(z.abs() < 0.5)] = 0.0\n    return sig.ffill().fillna(0.0)\n\n\ndef signal_rsi(prices: Series, window: int = 14, oversold: float = 30, overbought: float = 70) -> Series:\n    delta = prices.diff()\n    up = delta.clip(lower=0).rolling(window).mean()\n    down = -delta.clip(upper=0).rolling(window).mean()\n    rs = up / down.replace(0, np.nan)\n    rsi = 100 - (100 / (1 + rs))\n\n    sig = pd.Series(0.0, index=prices.index)\n    sig[rsi < oversold] = 1.0\n    sig[rsi > overbought] = -1.0\n    return sig.ffill().fillna(0.0)\n\n\ndef signal_timeseries_momentum(prices: Series, lookback: int = 63) -> Series:\n    ret = prices.pct_change(lookback)\n    return np.sign(ret).fillna(0.0)\n\n\ndef signal_fx_carry(rate_diff: Series) -> Series:\n    return np.sign(rate_diff).fillna(0.0)\n\n\ndef signal_pairs_cointegration(y: Series, x: Series, lookback: int = 120, z_entry: float = 2.0, z_exit: float = 0.5) -> Series:\n    aligned = pd.concat([y, x], axis=1).dropna()\n    aligned.columns = ["y", "x"]\n\n    if coint is None:\n        spread = aligned["y"] - aligned["x"]\n    else:\n        beta_hedge = np.polyfit(aligned["x"], aligned["y"], deg=1)[0]\n        spread = aligned["y"] - beta_hedge * aligned["x"]\n\n    z = (spread - spread.rolling(lookback).mean()) / spread.rolling(lookback).std(ddof=0)\n    sig = pd.Series(0.0, index=aligned.index)\n    sig[z > z_entry] = -1.0\n    sig[z < -z_entry] = 1.0\n    sig[z.abs() < z_exit] = 0.0\n    return sig.ffill().reindex(y.index).fillna(0.0)\n\n\ndef signal_curve_steepener(tenor_short: Series, tenor_long: Series, window: int = 60) -> Series:\n    spread = tenor_long - tenor_short\n    z = (spread - spread.rolling(window).mean()) / spread.rolling(window).std(ddof=0)\n    sig = pd.Series(0.0, index=spread.index)\n    sig[z < -1.0] = 1.0  # expect steepening\n    sig[z > 1.0] = -1.0  # expect flattening\n    return sig.ffill().fillna(0.0)\n\n\ndef signal_vol_risk_premium(implied_vol: Series, realized_vol: Series, threshold: float = 0.02) -> Series:\n    vrp = implied_vol - realized_vol\n    sig = pd.Series(0.0, index=vrp.index)\n    sig[vrp > threshold] = -1.0  # short vol\n    sig[vrp < -threshold] = 1.0  # long vol\n    return sig.ffill().fillna(0.0)\n\n\ndef signal_calendar_spread_roll(front: Series, second: Series, threshold: float = 0.01) -> Series:\n    slope = (second - front) / front.replace(0, np.nan)\n    sig = pd.Series(0.0, index=slope.index)\n    sig[slope > threshold] = 1.0\n    sig[slope < -threshold] = -1.0\n    return sig.ffill().fillna(0.0)\n\n\ndef signal_event_vol(event_dates: list[pd.Timestamp], index: pd.DatetimeIndex, pre_days: int = 3, post_days: int = 2) -> Series:\n    sig = pd.Series(0.0, index=index)\n    for event in event_dates:\n        event = pd.Timestamp(event)\n        pre_window = pd.bdate_range(event - pd.Timedelta(days=pre_days), event - pd.Timedelta(days=1))\n        post_window = pd.bdate_range(event, event + pd.Timedelta(days=post_days))\n        sig.loc[sig.index.intersection(pre_window)] = 1.0\n        sig.loc[sig.index.intersection(post_window)] = -0.5\n    return sig.ffill().fillna(0.0)\n\n\ndef signal_pca_residual_mean_reversion(returns_df: DataFrame, n_components: int = 3, z_entry: float = 2.0) -> DataFrame:\n    """Simple PCA residual strategy signals per asset."""\n    clean = returns_df.dropna()\n    if clean.empty:\n        return pd.DataFrame(index=returns_df.index, columns=returns_df.columns, data=0.0)\n\n    x = clean.values\n    x = (x - x.mean(axis=0)) / x.std(axis=0)\n    u, s, vh = np.linalg.svd(x, full_matrices=False)\n    recon = (u[:, :n_components] * s[:n_components]) @ vh[:n_components, :]\n    resid = x - recon\n    resid_df = pd.DataFrame(resid, index=clean.index, columns=clean.columns)\n    z = (resid_df - resid_df.rolling(60).mean()) / resid_df.rolling(60).std(ddof=0)\n    signals = -np.sign(z.where(z.abs() > z_entry, 0.0)).fillna(0.0)\n    return signals.reindex(returns_df.index).fillna(0.0)\n\n\ndef strategy_report(backtest: BacktestResult) -> DataFrame:\n    return pd.DataFrame([backtest.metrics]).T.rename(columns={0: "value"})\n\n\n# ---------------------------------------------------------------------------\n# Scenario grids and tools for dashboards\n# ---------------------------------------------------------------------------\n\n\ndef scenario_grid_option_pnl(\n    greek_dict: dict[str, float],\n    spot_shocks: NDArray[np.float64],\n    vol_shocks: NDArray[np.float64],\n    rate_shock: float = 0.0,\n    day_theta: float = 1 / 252,\n) -> DataFrame:\n    rows = []\n    for ds in spot_shocks:\n        for dv in vol_shocks:\n            delta = greek_dict.get("delta", 0.0) * ds\n            gamma = 0.5 * greek_dict.get("gamma", 0.0) * ds**2\n            vega = greek_dict.get("vega", 0.0) * dv\n            theta = greek_dict.get("theta", 0.0) * day_theta\n            rho = greek_dict.get("rho", 0.0) * rate_shock\n            total = delta + gamma + vega + theta + rho\n            rows.append({"spot_shock": ds, "vol_shock": dv, "scenario_pnl": total})\n    return pd.DataFrame(rows)\n\n\ndef portfolio_scenario_pnl(\n    sensitivities: DataFrame,\n    factor_shocks: dict[str, float],\n    mapping: dict[str, str] | None = None,\n) -> DataFrame:\n    mapping = mapping or {\n        "delta": "spot",\n        "gamma": "spot_sq",\n        "vega": "vol",\n        "theta": "time",\n        "rho": "rate",\n        "dv01": "rates_bp",\n        "cs01": "credit_bp",\n    }\n\n    rows = []\n    for _, row in sensitivities.iterrows():\n        pnl = 0.0\n        for greek_col, factor_key in mapping.items():\n            shock = factor_shocks.get(factor_key, 0.0)\n            val = float(row.get(greek_col, 0.0))\n            if factor_key == "spot_sq":\n                pnl += 0.5 * val * (factor_shocks.get("spot", 0.0) ** 2)\n            else:\n                pnl += val * shock\n        rows.append({"position_id": row.get("position_id"), "scenario_pnl": pnl})\n\n    out = pd.DataFrame(rows)\n    return out\n\n\n# ---------------------------------------------------------------------------\n# Misc\n# ---------------------------------------------------------------------------\n\n\ndef rolling_returns(returns: Series, windows: dict[str, int] | None = None) -> DataFrame:\n    windows = windows or {"1M": 21, "3M": 63, "6M": 126, "1Y": 252, "3Y": 756}\n    out = pd.DataFrame(index=returns.index)\n    for label, win in windows.items():\n        out[label] = (1 + returns).rolling(win).apply(np.prod, raw=True) - 1\n    return out\n\n\ndef calendar_returns(returns: Series) -> DataFrame:\n    s = returns.dropna()\n    df = pd.DataFrame({"date": s.index, "ret": s.values})\n    df["year"] = df["date"].dt.year\n    df["month"] = df["date"].dt.month\n    monthly = df.groupby(["year", "month"]) ["ret"].apply(lambda x: (1 + x).prod() - 1)\n    return monthly.unstack("month")\n\n\ndef twr(returns: Series) -> float:\n    r = returns.dropna()\n    if r.empty:\n        return 0.0\n    return float((1 + r).prod() - 1)\n\n\ndef xirr(cashflows: DataFrame, date_col: str = "date", amount_col: str = "amount") -> float:\n    """Money-weighted return solved via Newton method."""\n    cf = cashflows[[date_col, amount_col]].copy().dropna()\n    if cf.empty:\n        return 0.0\n    cf[date_col] = pd.to_datetime(cf[date_col])\n    t0 = cf[date_col].min()\n    years = (cf[date_col] - t0).dt.days / 365.25\n\n    def npv(rate: float) -> float:\n        return float(np.sum(cf[amount_col] / ((1 + rate) ** years)))\n\n    def d_npv(rate: float) -> float:\n        return float(np.sum(-years * cf[amount_col] / ((1 + rate) ** (years + 1))))\n\n    rate = 0.10\n    for _ in range(100):\n        f = npv(rate)\n        fp = d_npv(rate)\n        if abs(fp) < 1e-10:\n            break\n        new_rate = rate - f / fp\n        if abs(new_rate - rate) < 1e-8:\n            rate = new_rate\n            break\n        rate = new_rate\n    return float(rate)\n\n\n__all__ = [\n    "BacktestResult",\n    "annualized_return",\n    "annualized_volatility",\n    "sharpe_ratio",\n    "sortino_ratio",\n    "omega_ratio",\n    "drawdown_series",\n    "max_drawdown",\n    "calmar_ratio",\n    "information_ratio",\n    "tracking_error",\n    "beta",\n    "treynor_ratio",\n    "black_scholes_merton",\n    "black_76",\n    "bachelier",\n    "binomial_american_option",\n    "monte_carlo_option",\n    "implied_vol_newton",\n    "implied_vol_bisection",\n    "option_strategy_payoff",\n    "realized_vol_close_to_close",\n    "realized_vol_parkinson",\n    "realized_vol_garman_klass",\n    "realized_vol_rogers_satchell",\n    "realized_vol_yang_zhang",\n    "volatility_risk_premium",\n    "bond_price_from_yield",\n    "yield_from_bond_price",\n    "macaulay_duration",\n    "modified_duration",\n    "convexity",\n    "dv01",\n    "nelson_siegel",\n    "nelson_siegel_svensson",\n    "fit_nelson_siegel",\n    "key_rate_duration",\n    "tax_equivalent_yield",\n    "breakeven_inflation",\n    "ctd_basis",\n    "fx_forward_rate",\n    "forward_points",\n    "cross_rate",\n    "garman_kohlhagen",\n    "carry_trade_pnl",\n    "futures_cost_of_carry",\n    "roll_yield",\n    "term_structure_state",\n    "mean_variance_optimization",\n    "max_sharpe_portfolio",\n    "minimum_variance_portfolio",\n    "risk_contributions",\n    "risk_parity_portfolio",\n    "black_litterman",\n    "efficient_frontier",\n    "ewma_volatility",\n    "garch_11_volatility",\n    "var_parametric",\n    "var_historical",\n    "var_monte_carlo",\n    "var_cornish_fisher",\n    "cvar",\n    "stress_test_portfolio",\n    "skew_kurtosis_jb",\n    "sample_covariance",\n    "ledoit_wolf_covariance",\n    "exponentially_weighted_covariance",\n    "clustered_correlation",\n    "rolling_beta",\n    "regime_detection",\n    "dcc_like_dynamic_correlation",\n    "capm_regression",\n    "multi_factor_regression",\n    "brinson_fachler",\n    "greeks_pnl_attribution",\n    "factor_pnl_attribution",\n    "pnl_summary",\n    "pnl_heatmap_table",\n    "pnl_explain",\n    "desk_level_aggregation",\n    "pnl_distribution_stats",\n    "rolling_correlation_matrix",\n    "regime_conditional_correlation",\n    "granger_causality_table",\n    "impulse_response",\n    "risk_on_off_composite",\n    "dollar_smile_buckets",\n    "dispersion_metrics",\n    "backtest_signal",\n    "signal_moving_average_crossover",\n    "signal_breakout",\n    "signal_bollinger_mean_reversion",\n    "signal_rsi",\n    "signal_timeseries_momentum",\n    "signal_fx_carry",\n    "signal_pairs_cointegration",\n    "signal_curve_steepener",\n    "signal_vol_risk_premium",\n    "signal_calendar_spread_roll",\n    "signal_event_vol",\n    "signal_pca_residual_mean_reversion",\n    "strategy_report",\n    "scenario_grid_option_pnl",\n    "portfolio_scenario_pnl",\n    "rolling_returns",\n    "calendar_returns",\n    "twr",\n    "xirr",\n]\n'

exec(_EMBEDDED_NOTEBOOK_DEFAULTS, globals())
exec(_EMBEDDED_CORE, globals())
exec(_EMBEDDED_ANALYTICS, globals())

# Build analytics namespace used by the notebook sections.
_ta_all = globals().get('__all__', [])
ta = types.SimpleNamespace(**{name: globals()[name] for name in _ta_all if name in globals()})

if 'DataEngine' not in globals() or 'build_master_config' not in globals():
    raise RuntimeError('Embedded bootstrap failed: DataEngine/build_master_config missing.')

sns.set_theme(style='whitegrid', context='talk')
pd.set_option('display.max_columns', 200)
print('Standalone imports + embedded engine/analytics ready.')


## Master Configuration

Edit this one cell. Everything else uses this config.


In [ ]:
MASTER_CONFIG = build_master_config()

# User-friendly defaults
MASTER_CONFIG['DATA_MODE'] = 'synthetic'  # synthetic, database, manual, csv, api, hybrid
MASTER_CONFIG['AS_OF_DATE'] = '2025-01-31'
MASTER_CONFIG['START_DATE'] = '2022-01-01'
MASTER_CONFIG['BASE_CURRENCY'] = 'USD'
MASTER_CONFIG['REFRESH_CACHE'] = False
MASTER_CONFIG['SYNTHETIC_CONFIG']['num_positions'] = 240

# Save runtime config for compatibility with modular notebooks.
runtime_cfg_path = Path('output/jupyter-notebook/runtime_config.json')
runtime_cfg_path.parent.mkdir(parents=True, exist_ok=True)
runtime_cfg_path.write_text(json.dumps(MASTER_CONFIG, indent=2), encoding='utf-8')

print('Configured mode:', MASTER_CONFIG['DATA_MODE'])
print('Runtime config written to:', runtime_cfg_path.resolve())


## Initialize Engine and Load Core Data

This cell runs the unified data layer and quality checks.


In [ ]:
engine = DataEngine(MASTER_CONFIG)
data = engine.load_all(refresh=False)

positions = data['positions']
market = data['market_data']
risk = data['risk_sensitivities']
pnl = data['pnl_daily']
yield_curves = data['yield_curves']
credit_curves = data['credit_curves']
vol_surface = data['vol_surface']
factors = data.get('factor_returns', pd.DataFrame())

quality = engine.run_quality_checks()
print(quality['summary'])

for name, df in data.items():
    print(f"{name:18s} | rows={len(df):8d} | source={engine.source_of(name)}")


## Portfolio Landing Dashboard

This gives the user immediate top-line context before drilling into sections.


In [ ]:
overview = (
    positions.groupby(['desk', 'asset_class'], as_index=False)
    .agg(num_positions=('position_id', 'count'), gross_notional=('quantity_notional', lambda x: np.abs(x).sum()))
    .sort_values('gross_notional', ascending=False)
)

display(overview.head(20))

fig = px.treemap(
    overview,
    path=['desk', 'asset_class'],
    values='gross_notional',
    color='num_positions',
    title='Portfolio Gross Notional by Desk and Asset Class',
)
fig.update_layout(height=650)
fig.show()

pnl_by_desk = pnl.groupby('desk')['total_pnl'].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 5))
sns.barplot(x=pnl_by_desk.index, y=pnl_by_desk.values, palette='viridis')
plt.title('Cumulative PnL by Desk')
plt.xlabel('Desk')
plt.ylabel('PnL')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


## Section Functions

This cell defines every major section as a function, so users can run one area at a time or run everything with one click.


In [ ]:
def section_equities_and_options() -> None:
    eq = positions[positions['asset_class'].isin(['equities', 'equity_options', 'structured_products'])]
    ids = eq['ticker_identifier'].dropna().astype(str).unique().tolist()
    mkt = market[market['identifier'].isin(ids)]
    if mkt.empty:
        print('No equity-like market series available.')
        return
    rets = mkt.pivot_table(index='date', columns='identifier', values='return_1d').fillna(0)
    top = eq.groupby('ticker_identifier')['quantity_notional'].apply(lambda x: np.abs(x).sum()).sort_values(ascending=False).head(10).index
    fig = px.line((1 + rets[top]).cumprod(), title='Equity and Index Cumulative Returns')
    fig.show()

    strikes = np.linspace(70, 130, 41)
    rows = []
    for k in strikes:
        g = ta.black_scholes_merton(100, float(k), 0.5, MASTER_CONFIG['RISK_FREE_RATE'], 0.24)
        rows.append({'strike': k, 'price': g['price'], 'delta': g['delta'], 'gamma': g['gamma'], 'vega': g['vega'], 'theta': g['theta']})
    greek_df = pd.DataFrame(rows)
    fig2 = px.line(greek_df, x='strike', y=['delta', 'gamma', 'vega', 'theta'], title='Greek Profiles vs Strike')
    fig2.show()


def section_rates_credit_fx_commodities() -> None:
    usd = yield_curves[yield_curves['currency'] == 'USD']
    if not usd.empty:
        d = usd['date'].max()
        curve = usd[usd['date'] == d].sort_values('tenor_years')
        fig = px.line(curve, x='tenor_years', y='zero_rate', markers=True, title=f'USD Yield Curve ({pd.Timestamp(d).date()})')
        fig.show()

    if not credit_curves.empty:
        latest = credit_curves['date'].max()
        cc = credit_curves[credit_curves['date'] == latest]
        sample_names = cc['reference_entity'].drop_duplicates().head(6)
        fig2 = px.line(cc[cc['reference_entity'].isin(sample_names)], x='tenor_years', y='spread_bps', color='reference_entity',
                       title='Credit Curves (Sample Names)')
        fig2.show()

    fx = market[market['asset_class'] == 'fx']
    if not fx.empty:
        fx_ret = fx.pivot_table(index='date', columns='identifier', values='return_1d').fillna(0)
        basket = fx_ret.iloc[:, : min(8, fx_ret.shape[1])]
        carry_signal = pd.Series(np.linspace(-1, 1, basket.shape[1]), index=basket.columns)
        carry = basket.mul(np.sign(carry_signal), axis=1).mean(axis=1)
        fig3 = px.line((1 + carry).cumprod(), title='FX Carry Basket (Proxy)')
        fig3.show()

    cmdty = market[market['asset_class'] == 'commodities']
    if not cmdty.empty:
        p = cmdty.pivot_table(index='date', columns='identifier', values='close').dropna(how='all')
        fig4 = px.line(p.iloc[:, : min(4, p.shape[1])], title='Commodity Price Curves')
        fig4.show()


def section_portfolio_risk_and_attribution() -> None:
    returns = market.pivot_table(index='date', columns='identifier', values='return_1d').fillna(0)
    universe = returns.columns[: min(20, returns.shape[1])]
    r = returns[universe]
    cov = ta.ledoit_wolf_covariance(r) * MASTER_CONFIG['TRADING_DAYS_PER_YEAR']

    w_rp = ta.risk_parity_portfolio(cov)
    rp_ret = r.mul(w_rp, axis=1).sum(axis=1)

    metrics = {
        'Sharpe': ta.sharpe_ratio(rp_ret, MASTER_CONFIG['RISK_FREE_RATE']),
        'Sortino': ta.sortino_ratio(rp_ret, MASTER_CONFIG['RISK_FREE_RATE']),
        'MaxDrawdown': ta.max_drawdown(rp_ret),
        'VaR99_hist': ta.var_historical(rp_ret, 0.99),
        'CVaR99': ta.cvar(rp_ret, 0.99),
    }
    display(pd.DataFrame(metrics, index=['value']).T)

    fig = px.line((1 + rp_ret).cumprod(), title='Risk Parity Equity Curve')
    fig.show()

    dd = ta.drawdown_series(rp_ret)
    fig2 = px.line(dd, title='Risk Parity Underwater Plot')
    fig2.show()

    g = risk[['position_id', 'delta', 'gamma', 'vega', 'theta', 'rho']].head(200).copy()
    moves = g[['position_id']].copy()
    moves['dS'] = np.random.normal(0, 0.012, len(moves))
    moves['dVol'] = np.random.normal(0, 0.02, len(moves))
    moves['dR'] = np.random.normal(0, 0.0015, len(moves))
    moves['dT'] = 1 / MASTER_CONFIG['TRADING_DAYS_PER_YEAR']
    moves['actual_pnl'] = np.random.normal(0, 2200, len(moves))
    attr = ta.greeks_pnl_attribution(g, moves)
    s = attr[['delta_pnl', 'gamma_pnl', 'vega_pnl', 'theta_pnl', 'rho_pnl', 'residual_pnl']].sum()
    fig3 = px.bar(x=s.index, y=s.values, title='Greeks-based PnL Explain (Aggregate)')
    fig3.show()


def section_strategy_lab() -> None:
    px_close = market.pivot_table(index='date', columns='identifier', values='close').sort_index()
    if px_close.empty:
        print('No price data for strategies.')
        return

    asset = px_close.columns[0]
    p = px_close[asset].dropna()

    signals = {
        'ma_cross': ta.signal_moving_average_crossover(p, 20, 100),
        'breakout': ta.signal_breakout(p, 55),
        'bollinger_mr': ta.signal_bollinger_mean_reversion(p, 20, 2.0),
        'rsi_mr': ta.signal_rsi(p),
        'ts_momentum': ta.signal_timeseries_momentum(p, 63),
        'vol_rp': ta.signal_vol_risk_premium(
            implied_vol=pd.Series(0.22 + 0.08 * np.sin(np.linspace(0, 8, len(p))), index=p.index),
            realized_vol=p.pct_change().rolling(21).std().fillna(method='bfill') * np.sqrt(252),
        ),
        'calendar_roll': ta.signal_calendar_spread_roll(p, p * (1 + 0.01 * np.sin(np.linspace(0, 10, len(p))))),
    }

    if px_close.shape[1] >= 2:
        signals['pairs'] = ta.signal_pairs_cointegration(px_close.iloc[:, 0], px_close.iloc[:, 1])

    yc = yield_curves[yield_curves['currency'] == 'USD'].pivot_table(index='date', columns='tenor_years', values='zero_rate')
    if 2.0 in yc.columns and 10.0 in yc.columns:
        signals['curve_steepener'] = ta.signal_curve_steepener(yc[2.0].reindex(p.index).fillna(method='ffill'), yc[10.0].reindex(p.index).fillna(method='ffill'))

    if data.get('factor_returns', pd.DataFrame()).shape[0] > 0 and 'fx' in data['factor_returns'].columns:
        fx_proxy = data['factor_returns'].set_index('date')['fx'].reindex(p.index).fillna(0)
        signals['fx_carry_proxy'] = ta.signal_fx_carry(fx_proxy)

    event_dates = [p.index[int(i)] for i in np.linspace(20, len(p) - 20, 6, dtype=int)]
    signals['event_vol'] = ta.signal_event_vol(event_dates, p.index)

    if px_close.shape[1] >= 6:
        ret = px_close.pct_change().fillna(0)
        pca_sig = ta.signal_pca_residual_mean_reversion(ret.iloc[:, :6])
        signals['pca_stat_arb'] = pca_sig.mean(axis=1)

    bt = {}
    for name, sig in signals.items():
        bt[name] = ta.backtest_signal(
            prices=p,
            signal=sig.reindex(p.index).fillna(0),
            transaction_cost_bps=1.5,
            slippage_bps=0.8,
            financing_rate=MASTER_CONFIG['RISK_FREE_RATE'],
        )

    perf = pd.DataFrame({k: v.metrics for k, v in bt.items()}).T.sort_values('sharpe', ascending=False)
    display(perf)

    eq = pd.DataFrame({k: v.equity_curve for k, v in bt.items()})
    fig = px.line(eq, title='Strategy Equity Curves')
    fig.show()


def section_product_deep_dive() -> None:
    if widgets is None:
        print('ipywidgets is not installed; install ipywidgets for product deep dive explorer.')
        return

    product_opts = sorted(positions['product_type'].dropna().astype(str).unique().tolist())
    if not product_opts:
        print('No products found in positions table.')
        return

    product_dd = widgets.Dropdown(options=product_opts, description='Product', layout=widgets.Layout(width='370px'))
    pos_dd = widgets.Dropdown(description='Position', layout=widgets.Layout(width='500px'))
    out = widgets.Output()

    as_of = pd.Timestamp(MASTER_CONFIG.get('AS_OF_DATE', pd.Timestamp.today()))
    rf = float(MASTER_CONFIG.get('RISK_FREE_RATE', 0.04))

    def _hist_vol(identifier: str, window: int = 21) -> float:
        sub = market[market['identifier'].astype(str) == str(identifier)].sort_values('date')
        if sub.empty:
            return np.nan
        r = sub['return_1d'].dropna()
        if len(r) < 10:
            return np.nan
        return float(r.tail(window).std(ddof=0) * np.sqrt(252))

    def _latest_price(identifier: str, default: float = 100.0) -> float:
        sub = market[market['identifier'].astype(str) == str(identifier)].sort_values('date')
        if sub.empty:
            return float(default)
        return float(sub['close'].iloc[-1])

    def _ttm_from_row(row: pd.Series) -> float:
        maturity = pd.to_datetime(row.get('maturity_expiry'), errors='coerce')
        if pd.isna(maturity):
            maturity = pd.to_datetime(row.get('expiry'), errors='coerce')
        if pd.isna(maturity):
            return 0.5
        return max((maturity - as_of).days / 365.0, 1 / 365.0)

    def _update_positions(*_):
        sub = positions[positions['product_type'].astype(str) == str(product_dd.value)].copy()
        if sub.empty:
            pos_dd.options = []
            return
        sub = sub.sort_values('quantity_notional', key=np.abs, ascending=False)
        opts = []
        for _, r in sub.iterrows():
            label = f"{r['position_id']} | {r.get('ticker_identifier', '')} | {r.get('desk', '')}"
            opts.append((label, r['position_id']))
        pos_dd.options = opts
        if opts:
            pos_dd.value = opts[0][1]

    def _render(*_):
        with out:
            out.clear_output()

            if pos_dd.value is None:
                print('No position selected.')
                return

            r = positions[positions['position_id'].astype(str) == str(pos_dd.value)]
            if r.empty:
                print('Position not found.')
                return
            row = r.iloc[0]

            ptype = str(row.get('product_type', 'unknown')).lower()
            pid = str(row.get('position_id'))
            ident = str(row.get('ticker_identifier', ''))
            qty = float(row.get('quantity_notional', 0.0) or 0.0)
            direction = str(row.get('direction', 'long'))
            px_cur = float(row.get('current_price', np.nan) if pd.notna(row.get('current_price', np.nan)) else np.nan)
            px_ent = float(row.get('entry_price', np.nan) if pd.notna(row.get('entry_price', np.nan)) else np.nan)
            ttm = _ttm_from_row(row)
            hist_vol = _hist_vol(ident)
            latest_spot = _latest_price(ident, default=px_cur if pd.notna(px_cur) else 100.0)

            display(Markdown(f"## Product Deep Dive: `{ptype}` | Position `{pid}`"))
            display(Markdown(f"**Identifier:** `{ident}` | **Desk:** `{row.get('desk', '')}` | **Direction:** `{direction}` | **Quantity/Notional:** `{qty:,.2f}`"))

            base_info = pd.DataFrame({
                'field': ['entry_price', 'current_price', 'currency', 'entry_date', 'maturity_expiry', 'counterparty'],
                'value': [
                    row.get('entry_price'),
                    row.get('current_price'),
                    row.get('currency'),
                    row.get('entry_date'),
                    row.get('maturity_expiry'),
                    row.get('counterparty'),
                ],
            })
            display(base_info)

            formula = 'Generic mark-to-market proxy: V ~= Q * P.'
            calc_rows = []

            if ptype in {'equity_cash', 'etf', 'adr', 'equity_index_future', 'equity_swap', 'total_return_swap'}:
                multiplier = float(row.get('contract_multiplier', 1.0) or 1.0)
                value = qty * latest_spot * multiplier
                mtm = qty * (latest_spot - (px_ent if pd.notna(px_ent) else latest_spot)) * multiplier
                formula = (
                    'Linear product valuation: '
                    'V = Q * S * M, '
                    'MTM = Q * (S - S0) * M.'
                )
                calc_rows = [
                    ('spot', latest_spot),
                    ('multiplier', multiplier),
                    ('position_value', value),
                    ('mark_to_market', mtm),
                    ('annualized_hist_vol', hist_vol),
                ]

            elif ('option' in ptype and not any(k in ptype for k in ['asian', 'barrier', 'digital', 'lookback', 'compound', 'quanto', 'chooser', 'cliquet'])):
                strike = float(row.get('strike', latest_spot) or latest_spot)
                iv = float(row.get('implied_vol', row.get('iv', hist_vol if pd.notna(hist_vol) else 0.25)) or 0.25)
                otype = 'call' if str(row.get('option_type', 'call')).lower() in {'call', 'payer'} else 'put'

                if 'fx_' in ptype:
                    domestic = rf
                    foreign = max(rf - 0.01, 0.0)
                    g = ta.garman_kohlhagen(latest_spot, strike, ttm, domestic, foreign, iv, option_type=otype)
                    model_price = float(g['price'])
                elif 'commodity' in ptype:
                    model_price = float(ta.black_76(latest_spot, strike, ttm, rf, iv, option_type=otype))
                    g = ta.black_scholes_merton(latest_spot, strike, ttm, rf, iv, option_type=otype, dividend_yield=0.0)
                else:
                    g = ta.black_scholes_merton(latest_spot, strike, ttm, rf, iv, option_type=otype, dividend_yield=0.0)
                    model_price = float(g['price'])

                mark = px_cur if pd.notna(px_cur) else model_price
                intrinsic = max(latest_spot - strike, 0) if otype == 'call' else max(strike - latest_spot, 0)
                time_value = mark - intrinsic

                formula = (
                    'Black-Scholes/Garman-Kohlhagen style valuation: '
                    'd1 = (ln(S/K) + (r - q + 0.5*sigma^2)*T) / (sigma*sqrt(T)), '
                    'd2 = d1 - sigma*sqrt(T), '
                    'Call = S*exp(-q*T)*N(d1) - K*exp(-r*T)*N(d2).'
                )
                calc_rows = [
                    ('spot', latest_spot),
                    ('strike', strike),
                    ('ttm_years', ttm),
                    ('implied_vol', iv),
                    ('risk_free_rate', rf),
                    ('model_price', model_price),
                    ('mark_price', mark),
                    ('intrinsic_value', intrinsic),
                    ('time_value', time_value),
                    ('delta', float(g.get('delta', np.nan))),
                    ('gamma', float(g.get('gamma', np.nan))),
                    ('vega', float(g.get('vega', np.nan))),
                    ('theta', float(g.get('theta', np.nan))),
                ]

                spot_shocks = np.array([-0.10, -0.05, 0.0, 0.05, 0.10])
                vol_shocks = np.array([-0.05, 0.0, 0.05, 0.10])
                scen = ta.scenario_grid_option_pnl(g, spot_shocks, vol_shocks, rate_shock=0.0)
                heat = scen.pivot_table(index='spot_shock', columns='vol_shock', values='scenario_pnl')
                plt.figure(figsize=(7, 4))
                sns.heatmap(heat, annot=True, fmt='.2f', cmap='RdYlGn', center=0)
                plt.title('Option PnL Sensitivity Grid (Spot x Vol Shocks)')
                plt.tight_layout()
                plt.show()

            elif ptype in {'govt_bond', 'corporate_bond', 'muni_bond', 'tips', 'mbs'}:
                coupon = float(row.get('coupon', 0.04) or 0.04)
                ytm = float(row.get('yield_to_maturity', row.get('entry_yield', rf)) or rf)
                if ytm > 1.0:
                    ytm = ytm / 100.0
                tenor = float(row.get('tenor_years', max(ttm, 0.5)) or max(ttm, 0.5))
                clean_price = ta.bond_price_from_yield(100, coupon, ytm, tenor, freq=2)
                mdur = ta.modified_duration(100, coupon, ytm, tenor, freq=2)
                conv = ta.convexity(100, coupon, ytm, tenor, freq=2)
                pv01 = ta.dv01(100, coupon, ytm, tenor, freq=2)
                notional = abs(qty) if abs(qty) > 0 else 1_000_000.0
                est_value = np.sign(qty if qty != 0 else 1.0) * clean_price / 100 * notional

                formula = (
                    'Bond valuation: '
                    'P = sum_t CF_t / (1 + y/m)^t, '
                    'with duration/convexity and DV01 from price-yield sensitivity.'
                )
                calc_rows = [
                    ('coupon_rate', coupon),
                    ('yield_to_maturity', ytm),
                    ('maturity_years', tenor),
                    ('clean_price_model', clean_price),
                    ('estimated_position_value', est_value),
                    ('modified_duration', mdur),
                    ('convexity', conv),
                    ('dv01_per_100_notional', pv01),
                    ('hist_vol', hist_vol),
                ]

            elif ptype in {'irs', 'ois', 'fra'}:
                notional = abs(qty) if abs(qty) > 0 else float(row.get('notional', 10_000_000.0) or 10_000_000.0)
                fixed = float(row.get('fixed_rate', row.get('fra_rate', rf)) or rf)
                curve_rate = rf
                ccy = str(row.get('currency', 'USD'))
                yc = yield_curves[yield_curves['currency'].astype(str) == ccy].sort_values('date')
                if not yc.empty:
                    latest_dt = yc['date'].max()
                    yc_today = yc[yc['date'] == latest_dt]
                    target_tenor = float(row.get('tenor_years', max(ttm, 1.0)) or max(ttm, 1.0))
                    idx = (yc_today['tenor_years'] - target_tenor).abs().idxmin()
                    curve_rate = float(yc_today.loc[idx, 'zero_rate'])

                if ptype == 'fra':
                    mtm = notional * (curve_rate - fixed) * ttm / (1 + curve_rate * ttm)
                    dv01_est = notional * ttm * 1e-4 / (1 + curve_rate * ttm)
                    formula = 'FRA MTM proxy: V ~= N * ((F - K) * T) / (1 + F*T).'
                else:
                    pay_fixed = str(row.get('direction', '')).lower() == 'pay_fixed'
                    annuity = max(ttm, 0.25)
                    sign = 1.0 if pay_fixed else -1.0
                    mtm = sign * notional * (curve_rate - fixed) * annuity
                    dv01_est = notional * annuity * 1e-4
                    formula = 'Swap MTM proxy: V ~= N * Annuity * (R_mkt - R_fixed).'

                calc_rows = [
                    ('notional', notional),
                    ('contract_rate', fixed),
                    ('market_curve_rate', curve_rate),
                    ('ttm_or_annuity', ttm),
                    ('mtm_estimate', mtm),
                    ('dv01_estimate', dv01_est),
                ]

            elif ptype in {'cds', 'credit_index_tranche'}:
                notional = abs(qty) if abs(qty) > 0 else float(row.get('notional', 10_000_000.0) or 10_000_000.0)
                spread_bps = float(row.get('spread_bps', 150.0) or 150.0)
                spread = spread_bps / 10_000.0
                recovery = float(row.get('recovery_rate', 0.40) or 0.40)
                hazard = max(spread / max(1 - recovery, 1e-6), 0.0)
                premium_leg = notional * spread * max(ttm, 1.0)
                expected_loss = notional * (1 - recovery) * hazard * max(ttm, 1.0)

                formula = 'CDS proxy: hazard ~= spread/(1-recovery); premium leg ~= N*spread*T; default leg ~= N*(1-recovery)*hazard*T.'
                calc_rows = [
                    ('notional', notional),
                    ('spread_bps', spread_bps),
                    ('recovery_rate', recovery),
                    ('implied_hazard_rate', hazard),
                    ('premium_leg_proxy', premium_leg),
                    ('default_leg_proxy', expected_loss),
                    ('net_protection_value_proxy', expected_loss - premium_leg),
                ]

            elif ptype in {'fx_spot', 'fx_forward', 'fx_swap'}:
                pair = str(row.get('pair', ident if ident else 'EURUSD'))
                spot = float(row.get('spot_rate', row.get('entry_rate', latest_spot)) or latest_spot)
                rd = rf
                rf_for = max(rf - 0.01, 0.0)
                fwd_theo = ta.fx_forward_rate(spot, rd, rf_for, ttm)
                fwd_pts = ta.forward_points(spot, fwd_theo)
                entry = float(row.get('entry_rate', spot) or spot)
                notional = abs(qty) if abs(qty) > 0 else float(row.get('notional', 1_000_000.0) or 1_000_000.0)
                sign = 1.0 if str(direction).lower() in {'long', 'buy'} else -1.0
                mtm = sign * notional * (spot - entry)

                formula = 'CIP forward: F = S*exp((r_d-r_f)*T); spot MTM proxy: N*(S-S0).'
                calc_rows = [
                    ('pair', pair),
                    ('spot_rate', spot),
                    ('entry_rate', entry),
                    ('domestic_rate', rd),
                    ('foreign_rate', rf_for),
                    ('theoretical_forward', fwd_theo),
                    ('forward_points', fwd_pts),
                    ('notional', notional),
                    ('mtm_proxy', mtm),
                    ('hist_vol', hist_vol),
                ]

            elif ptype in {'commodity_future', 'commodity_option'}:
                spot = latest_spot
                storage = float(row.get('storage_cost', 0.02) or 0.02)
                conv_y = float(row.get('convenience_yield', 0.01) or 0.01)
                fut_theo = ta.futures_cost_of_carry(spot, rf, storage, conv_y, ttm)
                basis = spot - fut_theo

                formula = 'Cost-of-carry: F = S*exp((r+u-y)*T); basis = S - F.'
                calc_rows = [
                    ('spot', spot),
                    ('risk_free_rate', rf),
                    ('storage_cost', storage),
                    ('convenience_yield', conv_y),
                    ('ttm', ttm),
                    ('theoretical_futures', fut_theo),
                    ('basis', basis),
                    ('annualized_hist_vol', hist_vol),
                ]
                if ptype == 'commodity_option':
                    strike = float(row.get('strike', spot) or spot)
                    iv = float(row.get('implied_vol', hist_vol if pd.notna(hist_vol) else 0.30) or 0.30)
                    otype = 'call' if str(row.get('option_type', 'call')).lower() == 'call' else 'put'
                    opt_px = ta.black_76(spot, strike, ttm, rf, iv, option_type=otype)
                    calc_rows.extend([('option_strike', strike), ('implied_vol', iv), ('black76_option_price', opt_px)])

            elif any(k in ptype for k in ['asian', 'barrier', 'digital', 'lookback', 'compound', 'quanto', 'chooser', 'cliquet']):
                strike = float(row.get('strike', latest_spot) or latest_spot)
                iv = float(row.get('implied_vol', hist_vol if pd.notna(hist_vol) else 0.35) or 0.35)
                exotic = 'vanilla'
                barrier = None
                if 'asian' in ptype:
                    exotic = 'asian_arithmetic'
                elif 'digital' in ptype:
                    exotic = 'digital'
                elif 'lookback' in ptype:
                    exotic = 'lookback'
                elif 'barrier' in ptype:
                    exotic = 'barrier_knock_out'
                    barrier = float(row.get('barrier', latest_spot * 1.2) or latest_spot * 1.2)
                mc = ta.monte_carlo_option(latest_spot, strike, ttm, rf, iv, option_type='call', exotic=exotic, barrier=barrier, paths=20_000)

                formula = f'Monte Carlo valuation for {exotic} payoff: average discounted payoff over simulated paths.'
                calc_rows = [
                    ('spot', latest_spot),
                    ('strike', strike),
                    ('implied_vol', iv),
                    ('ttm', ttm),
                    ('mc_price', mc),
                    ('barrier', barrier),
                    ('hist_vol', hist_vol),
                ]

            else:
                value = qty * (px_cur if pd.notna(px_cur) else latest_spot)
                calc_rows = [('estimated_value', value), ('hist_vol', hist_vol)]

            display(Markdown('### Valuation Formula / Math'))
            display(Markdown(formula))
            display(pd.DataFrame(calc_rows, columns=['metric', 'value']))

            r_slice = risk[risk['position_id'].astype(str) == pid]
            if not r_slice.empty:
                display(Markdown('### Risk Sensitivities (from risk engine)'))
                display(r_slice.head(1).T.rename(columns={r_slice.index[0]: 'value'}))

            p_slice = pnl[pnl['position_id'].astype(str) == pid].sort_values('date')
            if not p_slice.empty:
                display(Markdown('### Position PnL Decomposition (recent)'))
                display(p_slice.tail(10))
                cum = p_slice.set_index('date')['total_pnl'].cumsum()
                plt.figure(figsize=(8, 3.5))
                plt.plot(cum.index, cum.values)
                plt.title('Cumulative Position PnL')
                plt.tight_layout()
                plt.show()

            mkt_sub = market[market['identifier'].astype(str) == ident].sort_values('date')
            if not mkt_sub.empty:
                rv = mkt_sub['return_1d'].rolling(21).std(ddof=0) * np.sqrt(252)
                plt.figure(figsize=(8, 3.5))
                plt.plot(mkt_sub['date'], rv)
                plt.title(f'21d Realized Volatility - {ident}')
                plt.tight_layout()
                plt.show()

    product_dd.observe(_update_positions, names='value')
    pos_dd.observe(_render, names='value')
    _update_positions()
    _render()

    display(Markdown('### Product Deep Dive Explorer'))
    display(widgets.HBox([product_dd, pos_dd]), out)


def section_interactive_tools() -> None:
    if widgets is None:
        print('ipywidgets is not installed; install ipywidgets for interactive tools.')
        return

    display(Markdown('### Options Pricer'))
    spot_w = widgets.FloatText(value=100.0, description='Spot')
    strike_w = widgets.FloatText(value=100.0, description='Strike')
    vol_w = widgets.FloatSlider(value=0.25, min=0.05, max=1.2, step=0.01, description='Vol')
    rate_w = widgets.FloatSlider(value=MASTER_CONFIG['RISK_FREE_RATE'], min=-0.02, max=0.15, step=0.001, description='Rate')
    ttm_w = widgets.FloatSlider(value=0.5, min=0.01, max=2.0, step=0.01, description='TTM')
    typ_w = widgets.Dropdown(options=['call', 'put'], value='call', description='Type')
    out = widgets.Output()

    def _run(*_):
        with out:
            out.clear_output()
            g = ta.black_scholes_merton(spot_w.value, strike_w.value, ttm_w.value, rate_w.value, vol_w.value, typ_w.value)
            display(pd.DataFrame(g, index=[0]).T.rename(columns={0: 'value'}))

    for w in [spot_w, strike_w, vol_w, rate_w, ttm_w, typ_w]:
        w.observe(_run, names='value')

    _run()
    display(widgets.VBox([widgets.HBox([spot_w, strike_w, typ_w]), widgets.HBox([vol_w, rate_w, ttm_w]), out]))


## Section Launcher (One-Notebook Control Panel)

Use this panel to run any section with one click, or run everything in sequence.


In [ ]:
def run_everything() -> None:
    section_equities_and_options()
    section_rates_credit_fx_commodities()
    section_portfolio_risk_and_attribution()
    section_strategy_lab()

if widgets is not None:
    buttons = {
        'Product Deep Dive': widgets.Button(description='Product Deep Dive', button_style='success'),
        'Equities + Options': widgets.Button(description='Equities + Options', button_style='info'),
        'Rates/FX/Credit/Commod': widgets.Button(description='Rates/FX/Credit/Commod', button_style='info'),
        'Portfolio Risk + Attr': widgets.Button(description='Portfolio Risk + Attr', button_style='warning'),
        'Strategy Lab': widgets.Button(description='Strategy Lab', button_style='warning'),
        'Interactive Tools': widgets.Button(description='Interactive Tools', button_style='success'),
        'Run Everything': widgets.Button(description='Run Everything', button_style='danger'),
    }

    out = widgets.Output()

    def wrap(fn):
        def _inner(_):
            with out:
                out.clear_output()
                fn()
        return _inner

    buttons['Product Deep Dive'].on_click(wrap(lambda: section_product_deep_dive()))
    buttons['Equities + Options'].on_click(wrap(lambda: section_equities_and_options()))
    buttons['Rates/FX/Credit/Commod'].on_click(wrap(lambda: section_rates_credit_fx_commodities()))
    buttons['Portfolio Risk + Attr'].on_click(wrap(lambda: section_portfolio_risk_and_attribution()))
    buttons['Strategy Lab'].on_click(wrap(lambda: section_strategy_lab()))
    buttons['Interactive Tools'].on_click(wrap(lambda: section_interactive_tools()))
    buttons['Run Everything'].on_click(wrap(lambda: run_everything()))

    display(
        widgets.VBox([
            widgets.HBox([buttons['Product Deep Dive'], buttons['Equities + Options'], buttons['Rates/FX/Credit/Commod']]),
            widgets.HBox([buttons['Portfolio Risk + Attr'], buttons['Strategy Lab'], buttons['Interactive Tools'], buttons['Run Everything']]),
            out,
        ])
    )
else:
    print('ipywidgets not installed. Run section_*() functions manually.')


## Optional: Keep Modular Notebooks for Deep Dive

If users want deeper theory and worked examples by topic, these remain available:
- `00_data_engine_and_configuration.ipynb`
- `01_multi_asset_product_analytics.ipynb`
- `02_portfolio_risk_pnl_factor_lab.ipynb`
- `03_strategies_dashboards_learning_system.ipynb`

But for most users, this all-in-one notebook is the primary UX.
